# NGC 628 (M74, UGC 1149)  spiral arm detection

In [ ]:
from IPython.display import HTML
from IPython.display import Image
import os
from PIL import Image as ImagePIL
import itertools
import time
from astropy.io import fits
import cv2
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
import math
import copy
from sklearn.metrics import *
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator

from scipy.interpolate import LinearNDInterpolator, RectBivariateSpline, RegularGridInterpolator
from scipy.ndimage import map_coordinates
# https://nbviewer.jupyter.org/github/pierre-haessig/stodynprog/blob/master/stodynprog/linear_interp_benchmark.ipynb

%pylab
%matplotlib inline
# %run ../../utils/load_notebook.py

In [ ]:
# from utils import *

In [ ]:
# from photometry import *

In [ ]:
# from instabilities import *

* [NED](https://ned.ipac.caltech.edu/cgi-bin/objsearch?objname=n628&extend=no&hconst=73&omegam=0.27&omegav=0.73&corr_z=1&out_csys=Equatorial&out_equinox=J2000.0&obj_sort=RA+or+Longitude&of=pre_text&zv_breaker=30000.0&list_limit=5&img_stamp=YES)
* [HYPERLEDA](http://leda.univ-lyon1.fr/ledacat.cgi?o=n628)

In [ ]:
name = 'NGC628'
# gtype = 'SBm' #LEDA
incl = 7.0  #Leroy, Walter08
distance = 8.6 #Mpc, Herrmann et al. 2008
# scale = calc_scale(distance) #kpc/arcsec

data_path = '../../data/ngc628'

sin_i, cos_i = np.sin(incl*np.pi/180.), np.cos(incl*np.pi/180.)
# print 'sin_i={:5.5f}, cos_i={:5.5f}, scale={:5.5f} kpc/arcsec'.format(sin_i, cos_i, scale)

# Литература

Основополагающая работа, где есть также результат для этой галактики

[ARM AND INTERARM STAR FORMATION IN SPIRAL GALAXIE](http://iopscience.iop.org/article/10.1088/0004-637X/725/1/534/pdf) Foyle, Rix, Walter, Leroy (2010)

Описывается все в разделе **2.2**, но без подробностей.

In [ ]:
Image('Foyle2010.png')

Похоже сам алгоритм был предложен и подробнее всего описывается в старой работе 

[Nonaxisymmetric Structures in the Stellar Disks of Galaxies](https://arxiv.org/pdf/astro-ph/9505111.pdf) Rix & Zaritsky (1995)

Интересно там то, как определяли угол (из соотношения T-F). Модель и подробности фитирования - разделы **3.1** и  **3.2**.

$$\frac{\mu(R,\phi)}{<\mu(R)>} = \sum_{m=1}^{\inf}A_m(R)e^{im[\phi-\phi_m(R)]}$$

Делаются полярные бины.

Небольшая критика подхода Foyle et al. из https://academic.oup.com/mnras/article/433/3/1837/1215753:

> For example, Hitschfeld et al. (2009) attributed
all emission above a certain threshold molecular gas surface density
to spiral arms and the remainder to the inner-arm regions, and
Foyle et al. (2010) followed a similar approach but instead used the
surface density of stellar 3.6 µm emission. This approach, however,
has the effect of including emission features (spurs) bridging the
gap between the spiral arms as part of the arms themselves.

[Classifying and modelling spiral structures in hydrodynamic simulations of astrophysical discs](https://arxiv.org/pdf/1802.01364.pdf) Forgan et al. (2018) - упоминают подход Foyle и предлагают свой для анализа гидродинамических экспериментов. Детектируют т.н. spine points и затем friends-of-friends алгоритм.

У них есть опубликованный код https://github.com/dh4gan/tache, но он только для SPH snapshot files (currently sphNG formats only).

http://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/

# DBSCAN

In [ ]:
Image('clustering.png')

In [ ]:
# def radial_data(data,annulus_width=1,working_mask=None,x=None,y=None,rmax=None):
#     """
#     r = radial_data(data,annulus_width,working_mask,x,y)
    
#     A function to reduce an image to a radial cross-section.
    
#     INPUT:
#     ------
#     data   - whatever data you are radially averaging.  Data is
#             binned into a series of annuli of width 'annulus_width'
#             pixels.
#     annulus_width - width of each annulus.  Default is 1.
#     working_mask - array of same size as 'data', with zeros at
#                       whichever 'data' points you don't want included
#                       in the radial data computations.
#       x,y - coordinate system in which the data exists (used to set
#              the center of the data).  By default, these are set to
#              integer meshgrids
#       rmax -- maximum radial value over which to compute statistics
    
#      OUTPUT:
#      -------
#       r - a data structure containing the following
#                    statistics, computed across each annulus:
#           .r      - the radial coordinate used (outer edge of annulus)
#           .mean   - mean of the data in the annulus
#           .std    - standard deviation of the data in the annulus
#           .median - median value in the annulus
#           .max    - maximum value in the annulus
#           .min    - minimum value in the annulus
#           .numel  - number of elements in the annulus
#     """
    
# # 2010-03-10 19:22 IJC: Ported to python from Matlab
# # 2005/12/19 Added 'working_region' option (IJC)
# # 2005/12/15 Switched order of outputs (IJC)
# # 2005/12/12 IJC: Removed decifact, changed name, wrote comments.
# # 2005/11/04 by Ian Crossfield at the Jet Propulsion Laboratory
 
#     import numpy as ny

#     class radialDat:
#         """Empty object container.
#         """
#         def __init__(self): 
#             self.mean = None
#             self.std = None
#             self.median = None
#             self.numel = None
#             self.max = None
#             self.min = None
#             self.r = None

#     #---------------------
#     # Set up input parameters
#     #---------------------
#     data = ny.array(data)
    
#     if working_mask==None:
#         working_mask = ny.ones(data.shape,bool)
    
#     npix, npiy = data.shape
#     if x==None or y==None:
#         x1 = ny.arange(-npix/2.,npix/2.)
#         y1 = ny.arange(-npiy/2.,npiy/2.)
#         x,y = ny.meshgrid(y1,x1)

#     r = abs(x+1j*y)

#     if rmax==None:
#         rmax = r[working_mask].max()

#     #---------------------
#     # Prepare the data container
#     #---------------------
#     dr = ny.abs([x[0,0] - x[0,1]]) * annulus_width
#     radial = ny.arange(rmax/dr)*dr + dr/2.
#     nrad = len(radial)
#     radialdata = radialDat()
#     radialdata.mean = ny.zeros(nrad)
#     radialdata.std = ny.zeros(nrad)
#     radialdata.median = ny.zeros(nrad)
#     radialdata.numel = ny.zeros(nrad)
#     radialdata.max = ny.zeros(nrad)
#     radialdata.min = ny.zeros(nrad)
#     radialdata.r = radial
    
#     #---------------------
#     # Loop through the bins
#     #---------------------
#     for irad in range(nrad): #= 1:numel(radial)
#       minrad = irad*dr
#       maxrad = minrad + dr
#       thisindex = (r>=minrad) * (r<maxrad) * working_mask
#       if not thisindex.ravel().any():
#         radialdata.mean[irad] = ny.nan
#         radialdata.std[irad]  = ny.nan
#         radialdata.median[irad] = ny.nan
#         radialdata.numel[irad] = ny.nan
#         radialdata.max[irad] = ny.nan
#         radialdata.min[irad] = ny.nan
#       else:
#         radialdata.mean[irad] = data[thisindex].mean()
#         radialdata.std[irad]  = data[thisindex].std()
#         radialdata.median[irad] = ny.median(data[thisindex])
#         radialdata.numel[irad] = data[thisindex].size
#         radialdata.max[irad] = data[thisindex].max()
#         radialdata.min[irad] = data[thisindex].min()
    
#     #---------------------
#     # Return with data
#     #---------------------
    
#     return radialdata

In [ ]:
def azimuthalAverage(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    # Calculate the indices from the image
    y, x = np.indices(image.shape)

    if not center:
        center = np.array([(x.max()-x.min())/2.0, (x.max()-x.min())/2.0])

    r = np.hypot(x - center[0], y - center[1])

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    
    # Cumulative sum to figure out sums for each radius bin
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]

    radial_prof = tbin / nr

    return radial_prof

In [ ]:
from scipy import fftpack
import pyfits
import numpy as np
import pylab as py
# import radialProfile

# an astronomical image (left), 
# the 2D power spectrum of the image (middle), 
# and the azimuthally averaged 1D power spectrum (right)
 
image = pyfits.getdata('../../data/ngc628/ngc0628_heracles_mom0.fits')

image = np.squeeze(fits.getdata('../../data/ngc628/ngc0628_heracles_mom0.fits'))
image[np.isnan(image)] = 0
# image.data = np.require(image.data, dtype=np.float16)

im2 = np.resize(np.float32(image.flatten()),(294, 295))
 
# Take the fourier transform of the image.
F1 = fftpack.fft2(im2)
 
# Now shift the quadrants around so that low spatial frequencies are in
# the center of the 2D fourier transformed image.
F2 = fftpack.fftshift( F1 )
 
# Calculate a 2D power spectrum
psd2D = np.abs( F2 )**2
 
# Calculate the azimuthally averaged 1D power spectrum
psd1D = azimuthalAverage(psd2D)
 
# Now plot up both
py.figure(1)
py.clf()
py.imshow( np.log10( image ), cmap=py.cm.Greys)
 
py.figure(2)
py.clf()
py.imshow( np.log10( psd2D ))
 
py.figure(3)
py.clf()
py.semilogy( psd1D )
py.xlabel('Spatial Frequency')
py.ylabel('Power Spectrum')
 
py.show()

In [ ]:
from mpl_toolkits import mplot3d

ax = plt.axes(projection='3d')
ax.plot3D(xline, yline, zline, 'gray')

In [ ]:
fig = plt.figure(figsize=[24,16])
plt.subplot(121)
ax = plt.gca()
fname = '../../data/ngc628/ngc0628_heracles_mom0.fits'
data = fits.getdata(fname)
im = plt.imshow(np.squeeze(data), origin='lower', cmap='jet')

In [ ]:
fig = plt.figure(figsize=[24,16])
plt.subplot(121)
im = plt.imshow(np.squeeze(data), origin='lower', cmap='jet', vmin=3.5, vmax=5)

In [ ]:
data

In [ ]:
fig = plt.figure(figsize=[24,16])
plt.subplot(121)
ax = plt.gca()
fname = '../../data/ngc628/NGC0628_WISE_3.4.fits'
data = fits.getdata(fname)
im = plt.imshow(np.squeeze(data[1100:1500, 1100:1500]), origin='lower', cmap='jet', vmin=0.00002, vmax= 0.00013)

In [ ]:
fig = plt.figure(figsize=[24,16])
plt.subplot(121)
ax = plt.gca()
fname = '../../data/ngc628/NGC0628_Spitzer_3.6.fits.gz'
data = fits.getdata(fname)
im = plt.imshow(np.squeeze(data[2100:2700, 2100:2700]), origin='lower', cmap='jet', vmax=0.00007, vmin=0.000005)
plt.scatter(300, 300, 50, 'b')

In [ ]:
spitzer = np.squeeze(data[2100:2700, 2100:2700])

In [ ]:
spitzer[300][300]

In [ ]:
dist = np.zeros(spitzer.shape)
for i in range(0, 600):
    for j in range(0, 600):
        dist[i][j] = np.sqrt((i-300)**2 + (j-300)**2)

In [ ]:
fig = plt.figure(figsize=[24,16])
plt.subplot(121)
ax = plt.gca()
im = plt.imshow(dist, origin='lower', cmap='jet')
plt.scatter(300, 300, 50, 'b')

In [ ]:
substr = spitzer.copy()
for i in range(0, 600):
    for j in range(0, 600):
        substr[i][j] -= spitzer[300][300]*np.exp(-dist[i][j]/50.)

In [ ]:
fig = plt.figure(figsize=[24,16])
plt.subplot(121)
ax = plt.gca()
im = plt.imshow(np.log10(np.abs(substr)), origin='lower', cmap='jet', vmax=-3.7, vmin=-7)
plt.scatter(300, 300, 50, 'b')
plt.colorbar(im)

In [ ]:
fig = plt.figure(figsize=[24,16])
plt.subplot(121)
ax = plt.gca()
im = plt.imshow(np.log10(np.abs(substr))[200:400, 200:400], origin='lower', cmap='jet', vmax=-4.2, vmin=-4.5)
# plt.scatter(300, 300, 50, 'b')
plt.colorbar(im)

Сравнивать будем с результатами из [The Star Formation Efficiency in Nearby Galaxies: Measuring Where Gas Forms Stars Effectively](http://iopscience.iop.org/article/10.1088/0004-6256/136/6/2782/pdf) Leroy 2008

Цитата про S04 модель, которая объясняет чем у них плохо:
> Inner galaxy disks will tend
to have higher metallicities, which will lower ΣS04. We already
find Σgas > ΣS04 over most inner disks; therefore, missing Z
seems unlikely to seriously bias our results.

In [ ]:
# Image('Leroy2008_maps.png')

In [ ]:
# Image('Leroy2008_results.png')

* [THINGS: THE H i NEARBY GALAXY SURVEY](http://iopscience.iop.org/article/10.1088/0004-6256/136/6/2563/pdf) Walter 2008 - $I_{HI}, V_{HI}, \sigma_{HI}$
* https://arxiv.org/pdf/1612.05272.pdf - velocity dispersion of the HI gas, and early work with the WSRT indicated a value around 10 km/s, with a slight enhancement in star forming regions as found for NGC 628 by Shostak and van der Kruit (1984)
* [THE BIMA SURVEY OF NEARBY GALAXIES. I. THE RADIAL DISTRIBUTION OF CO EMISSION IN SPIRAL GALAXIES](http://iopscience.iop.org/article/10.1086/323221/pdf)-$I_{CO}$
* [THE VIRUS-P EXPLORATION OF NEARBY GALAXIES (VENGA): SURVEY DESIGN, DATA PROCESSING, AND SPECTRAL ANALYSIS METHODS](http://iopscience.iop.org/article/10.1088/0004-6256/145/5/138/pdf) - $V_*, \sigma_*$
* [THE VIRUS-P EXPLORATION OF NEARBY GALAXIES (VENGA): THE XCO GRADIENT IN NGC 628](http://iopscience.iop.org/article/10.1088/0004-637X/764/2/117/pdf) - Данные по $SFR$, $I_{CO}$ из HERACLES, BIMA, CARMS, карта металличности, изучение $X_{CO}$
>We caution against adopting a single XCO value when large changes in gas surface density
or metallicity are present.
* [The JCMT Nearby Galaxies Legacy Survey – IV. Velocity dispersions in the molecular interstellar medium in spiral galaxies](https://ui.adsabs.harvard.edu/#abs/2011MNRAS.410.1409W/abstract)- $\sigma_{CO(3-2)}$
>CO J = 3–2 velocity dispersion for six galaxies which are not members of the Virgo cluster. (a) NGC 628. Contour levels are 2.5, 5 and 10 km s−1
and the colour scale peak is 15 km s−1.
* [The extended law of star formation: the combined role of gas and stars](https://arxiv.org/pdf/1702.02609.pdf) Dib (2016) - **обязательно цитировать**, ибо они делают очень похожее и для той же галактики с теми же данными! Но при этом там непонятная турбулентность, результаты похожи на читерские, пикселей всего 90 (The number of resolution elements in NGC 628 that simultaneously have σ∗ measurements in VENGA as well as measured values of ΣSFR,obs is 91 - ?), звезды у них изотермическая жидкость, т.е. дисперсии во все стороны равны.
* [COMPARING C ii, H i, AND CO DYNAMICS OF NEARBY GALAXIES](https://arxiv.org/pdf/1604.08777.pdf)- есть карта скоростей CII, HI и CO
* [CO(J = 3 − 2) On-the-fly Mapping of the Nearby Spiral Galaxies NGC 628 and NGC 7793: Spatially-resolved CO(J = 3 − 2) Star-formation Law](https://arxiv.org/pdf/1512.04159.pdf)-$I_{CO(3-2)}$
* [HI AND CO VELOCITY DISPERSIONS IN NEARBY GALAXIES](https://arxiv.org/pdf/1511.06006.pdf) - радиальные профили $\sigma_{HI}, \sigma_{CO}$, можно использовать для проверки
* [HERACLES: THE HERA CO LINE EXTRAGALACTIC SURVEY](http://iopscience.iop.org/article/10.1088/0004-6256/137/6/4670/pdf) Leroy 2009 - есть карта $I_{CO(2-1)}$
* [SCALING RELATIONS OF THE PROPERTIES FOR CO RESOLVED STRUCTURES IN NEARBY SPIRAL GALAXIES](http://iopscience.iop.org/article/10.1088/0004-637X/808/1/99/pdf) Leroy 2015 - $\Sigma_{H_2}, \Sigma_{SFR}$ (правда только для верхней части галактики судя по всему)
* [THE CO-TO-H2 CONVERSION FACTOR AND DUST-TO-GAS RATIO ON KILOPARSEC SCALES IN NEARBY GALAXIES](http://iopscience.iop.org/article/10.1088/0004-637X/777/1/5/pdf) - Данные по $\alpha_{CO}, \Sigma_{dust}$
* [DISCOVERY OF RECENT STAR FORMATION IN THE EXTREME OUTER REGIONS OF DISK GALAXIES](http://iopscience.iop.org/article/10.1086/311626/pdf) - $H_{\alpha}$ картинка
* [The JCMT Nearby Galaxies Legacy Survey. VII. Hα imaging and massive star formation properties](https://arxiv.org/pdf/1203.0001.pdf) - картинка $H_{\alpha}$
* [The Westerbork SINGS survey III. Global magnetic field topology](https://www.aanda.org/articles/aa/pdf/2010/06/aa13375-09.pdf) - В случае, если нас заинтересует - есть данные по магнитному полю
* [Hα Kinematics of the SINGS Nearby Galaxies Survey](https://arxiv.org/pdf/astro-ph/0601376.pdf) - $H_{\alpha}$ кривая вращения
* [The JCMT Nearby Galaxies Legacy Survey – VIII. CO data and the LCO(3−2)–LFIR correlation in the SINGS sample](https://oup.silverchair-cdn.com/oup/backfile/Content_public/Journal/mnras/424/4/10.1111_j.1365-2966.2012.21453.x/1/424-4-3050.pdf?Expires=1504015928&Signature=LkroryIoCFcC7KwEtI73WJ5DKK3qzRrM56WiybgKCpcejAtQaaqbqzVcZkEUnYo3u3M8-nsM4AWNjXtjV3nJHRi5FU1dqAQ4VqritmCvXw1hbstOSTNWjsZzPNiD9TAehKbLPAblc6EFF0338NS7vyGp8UUNWq95zIeqA0FIjM4UxtotKVyOfjsMruYY4PjjJCbF8ItUsFFXelMiri2JlKUuVS6Lz2pmfxj8UntbHhqzVBawtdWD4IV0BTLPHN6oB58hwZ~3lVXurPXOkm1C9a-GBaZCQgEucyDCTjIPoJJWnE1fY7z5ouaRL6BsJsdHEe3XTr9F9uW387tcailomQ__&Key-Pair-Id=APKAIUCZBIA4LVPAVW3Q) - есть карта $I_{CO(3-2)}, V_{CO(3-2)}$
* Heidt 2001 https://www.aanda.org/articles/aa/pdf/2001/10/aa10227.pdf - Фотометрия балдж 13-20 arcsec
* http://iopscience.iop.org/article/10.1088/0004-6256/142/1/16/pdf - Фотометрия балдж вроде порядка 15
* [http://www.oulu.fi/astronomy/S4G_PIPELINE4/MAIN/decomp0053.html#entry0003](S4G) - там тоже видно, что балдж не такой большой (точно не скажу, надо вспоминать, там scale есть)
* https://arxiv.org/pdf/1506.01653.pdf - $M_*,SFR, extinction$ maps
* SPITZER изображение http://www.spitzer.caltech.edu/images/2257-sig06-018c-NASA-s-Spitzer-Space-Telescope-View-of-M74
* На заглавной https://guaix.fis.ucm.es/sings есть куча картинок NGC628 в полосах FUV 1528 A,	NUV   2271 A,	B   0.45 μm,	V   0.55 μm,	R   0.66 μm,	I   0.81 μm, IRAC1   3.6 μm,
	IRAC2   4.5 μm,IRAC3   5.8 μm,	IRAC4   8.0 μm,	MIPS1   24 μm,	MIPS2   70 μm. 
* S4G $S^4G$ http://www.oulu.fi/astronomy/S4G_PIPELINE4/MAIN/decomp0053.html#entry0003
* [Parameters of the brightest star formation regions in the two principal spiral arms of NGC 628](https://arxiv.org/pdf/1310.3690.pdf) - спиральные рукава и star-forming regions
* [Late-type galaxies observed with SAURON: two-dimensional stellar and emission-line kinematics of 18 spirals](https://academic.oup.com/mnras/article/367/1/46/1020614/Late-type-galaxies-observed-with-SAURON-two) - странные диспресии и скорости
* [The inner mass distribution of late-type spiral galaxies from SAURON stellar kinematic maps](https://arxiv.org/pdf/1609.08700.pdf) - еще более странные дисперсии и скорости. Есть кривая вращения, но как-то уж очень странно извлеченная.
* [Hα kinematics of the SINGS nearby galaxies survey – I](http://adsabs.harvard.edu/abs/2006MNRAS.367..469D)
* http://iopscience.iop.org/article/10.1088/0004-6256/141/1/24/pdf Vollmer Leroy 2011 - турбулентная модель и кривая вращения
* [THE HIERARCHICAL DISTRIBUTION OF THE YOUNG STELLAR CLUSTERS IN SIX LOCAL STAR FORMING GALAXIES](https://arxiv.org/pdf/1704.06321.pdf) и [PLANETARY NEBULAE IN FACE-ON SPIRAL GALAXIES. III. PLANETARY NEBULA KINEMATICS AND DISK MASS](http://iopscience.iop.org/article/10.1088/0004-637X/705/2/1686/pdf) - есть кривая вращения и много разноцветных кружков YSO
* [HERACLES: THE HERA CO LINE EXTRAGALACTIC SURVEY](http://iopscience.iop.org/article/10.1088/0004-6256/137/6/4670/pdf) HERACLES CO Leroy 2009 - M_H2/M_HI = 0.24, M_H2/M_* = 0.1, (M_H2 + M_HI)/M_* = 0.52, ну и карта уровней CO само собой
* [WHAT IS DRIVING THE H i VELOCITY DISPERSION?](http://iopscience.iop.org/article/10.1088/0004-6256/137/5/4424/pdf) Tamburro 2009 - THINGS velocity resolution of the H i data cubes - 2.6 km/s, H i mass-weighted median of the H i velocity dispersion - 8.0 km/s, LEDA Hubble Type - S?, есть Radial profiles of H i velocity dispersion 
* [THE STAR FORMATION EFFICIENCY IN NEARBY GALAXIES: MEASURING WHERE GAS FORMS STARS EFFECTIVELY](http://iopscience.iop.org/article/10.1088/0004-6256/136/6/2782/pdf) Leroy 2008 - M_B = −20.0, logM* = 10.1, logM_HI = 9.7, logM_H2 = 9.0, SFR=0.807 Msun/yr, $l_*\approx l{SFR} \approx l_{CO}$, таблица профилей $\Sigma_{HI},\Sigma_{H2}, \Sigma_{*}, FUV,24\mu m$
* [THINGS: THE H i NEARBY GALAXY SURVEY](http://iopscience.iop.org/article/10.1088/0004-6256/136/6/2563/pdf) Walter 2008 -  M 74, UGC 1149 RA - 01 36 41.8, Decl. +15 47 00, D-7.3 [Karachentsev et al. 2004], m_B=9.35, PA=20 [Tamburro et al. 2008],  12+log(O/H) - 8.33 [Moustakas 2006], SFR-1.21 [Lee 2006], параметры кубов (их два - NA и RO), $M_{HI} = 3.8\times 10^9$
* [KINGFISH—Key Insights on Nearby Galaxies: A Far-Infrared Survey with Herschel: Survey Description and Image Atlas](http://iopscience.iop.org/article/10.1086/663818/pdf) KENNICUTT 2011 - SAc, size 10.5x9.5 arcmin, dist method SN II, SFR - 0.68, high surface brightness
* [ARM AND INTERARM STAR FORMATION IN SPIRAL GALAXIES](http://iopscience.iop.org/article/10.1088/0004-637X/725/1/534/pdf) Foyle 2010 - изучают спиральный узор, много красивых карт и слов про узор (сколько руковов), но главное
> Taken together these results show that interarm star formation
is significant and that the spiral arms gather the gas into
regions of higher surface densities, which leads to an enhanced
molecular fraction, but they do not “shock trigger” star formation
nor molecular gas formation. Thus, spiral arms act only to
reorganize the material in the disk out of which stars form.
* [REGULATION OF STAR FORMATION RATES IN MULTIPHASE GALACTIC DISKS: A THERMAL/DYNAMICAL EQUILIBRIUM MODEL](http://iopscience.iop.org/article/10.1088/0004-637X/721/2/975/pdf) Ostriker 2010 - довольно важная работа, interstellar medium (ISM) heating by stellar UV plays a key role. By requiring that thermal and (vertical) dynamical equilibrium are simultaneously satisfied within the diffuse gas, and that stars form at a rate proportional to the mass of the self-gravitating component. Получилось неплохое согласие с наблюдениями в 628, но там все азимутально усреднено, масштаб в логарифме и толщина постоянна. Но думаю упомянуть стоит в работе.
* [A HIGH-DISPERSION MOLECULAR GAS COMPONENT IN NEARBY GALAXIES](http://iopscience.iop.org/article/10.1088/0004-6256/146/6/150/pdf) Caldu-Primo 2013 - изучали профили диспресий и есть вывод, который можно использовать чтобы обосновать одинаковую дисперсию для HI и CO (также  Combes & Becquaert, 1997):
> similarity of the CO and H i dispersions suggests that the two
components are well mixed, and are only two different phases
of the same kinematical gas component
* https://arxiv.org/pdf/1606.07077.pdf 2016 - есть CO карта с разрешением 45-60 пк.
* [H I AND CO VELOCITY DISPERSIONS IN NEARBY GALAXIES](http://iopscience.iop.org/article/10.3847/0004-6256/151/1/15/pdf) Mogotsi 2016 - есть средние дисперсии по HI и H2 и соотношение между ними и средняя разница (все азимутально усредненное). Дисперсии очень мало отличаются.
* обязательно надо что-то процитировать из http://iopscience.iop.org/article/10.1088/0004-637X/808/1/99/pdf (большая работа, много упоминаний)
* пыль и $X_{CO}$ также исследовались в http://iopscience.iop.org/article/10.1088/0004-637X/756/2/138/pdf, большая яработа, может быть важна (есть карты пыли) 
* [A REVISED PLANETARY NEBULA LUMINOSITY FUNCTION DISTANCE TO NGC 628 USING MUSE](https://arxiv.org/pdf/1611.09369.pdf) - самые новые данные по расстоянию, хотя точность меньше чем у тех где 8.6

Еще данных:

In [ ]:
# http://iopscience.iop.org/article/10.1088/0004-6256/140/5/1194/pdf - Bigiel2010
# D-7.3, i-7, PA=20, r25 = 4.9 arcmin, type - Sc
# Image('Bigiel2010.png', width=500)

[THE FINE-SCALE STRUCTURE OF THE NEUTRAL INTERSTELLAR MEDIUM IN NEARBY GALAXIES](http://iopscience.iop.org/article/10.1088/0004-6256/141/1/23/pdf) Bagetakos 2011 - изучают HI holes:

> there must be sufficient contrast (at least a 50% drop in
surface brightness) between the hole and its immediate
surroundings & 4. the shape of the hole in pV space must be clearly defined
and adequately described by an ellipse

Также анализируют высоту (средняя - 500 пк ), но там все как обычно по методам. Дыр найдено примерно 100

> NGC 628 is a grand-design Sc spiral galaxy located at a
distance of 7.3 Mpc and is seen almost completely face-on.
We detected 104 holes in its H i layer, with diameters ranging
from 0.24 kpc (the resolution limit) to 2 kpc. Only a small
fraction of these holes are type 3 (≈5%). Several supershells
were detected, however four cases (holes number 24, 45, 47,
and 99) are probably superpositions of two or more holes.

In [ ]:
# Image('holes.png', width=600)

[A TEST OF STAR FORMATION LAWS IN DISK GALAXIES. II. DEPENDENCE ON DYNAMICAL PROPERTIES](http://iopscience.iop.org/article/10.1088/0004-637X/787/1/68/pdf) Suwannajak 2014 - есть кривая вращения и сравнение 6 законов звездообразования (K-S,Constant Molecular,KM05,KMT09,Gas-Ω, GMC Collision law), которые все неплохо согласуются.

In [ ]:
# Image('Suwannajak2014.png')

# Данные

In [ ]:
os.chdir(data_path)

## Расстояния (THINGS/VENGA область)

Рассчитаем покоординатно расстояние от центра галактики.

https://physics.stackexchange.com/questions/224950/how-can-i-convert-right-ascension-and-declination-to-distances

In [ ]:
%%time
cposition= SkyCoord('24.174042  15.783472', frame='fk5', unit='deg', equinox='J2000.0')
dist = np.zeros((55, 155))

for i in range(len(dist)):
    for j in range(len(dist[0])):
        scoord = sgrid[i][j]
        dist[i][j] = cposition.separation(scoord).arcsec

In [ ]:
X,Y = np.meshgrid(np.arange(0.,155,1), np.arange(0.,55,1))
plt.figure(figsize=[18, 6])
quadmesh = plt.pcolormesh(X, Y, dist, cmap=cm.Blues)
plt.colorbar()
quadmesh.cmap.set_under('white')
CS = plt.contour(X, Y, dist, levels=[50, 100, 150, 500, 750])

In [ ]:
# dist_[28][0] + dist_[28][-1], dist_[0][81] + dist_[-1][81]

In [ ]:
# fname = '../../data/ngc628/blanc_fits/NGC0628_map_ha_corr.fits'
# data = fits.getdata(fname)
# plt.figure(figsize=[18, 6])
# quadmesh = plt.pcolormesh(X, Y, np.squeeze(data)[4:-1, 3:-2], cmap='sauron')
# quadmesh.set_clim(vmin=0., vmax=1e-14)
# # quadmesh.cmap.set_under('white')
# plt.colorbar()
# CS = plt.contour(X, Y, dist, levels=[50, 100, 150, 500, 750])

# plt.show()

In [ ]:
np.save('dist.npy', dist)

## Дисперсии $\sigma_{\rm s}$ (VENGA)

$$\sigma^2_{los, \phi} = [\sigma_R^2\sin^2\phi + \sigma^2_{\varphi}\cos^2\phi]\sin^2 i + \sigma_z^2\cos^2 i,$$ where $\phi$ is the angle between the slit and the major axis of the
projected stellar disc (the position angle).

TODO: разрешение куба в пространственном и по скоростям, учеть другой угол и расстояние если надо - у них inclination of i = 8.7 and a distance of 8.6 Mpc (Herrmann et al. 2008).

Как и с SFR, cперва уберем точки и лишнее, применим dilation

VENGA 2013: At the assumed distance of NGC 628 the VENGA spatial resolution corresponds to ∼235 pc (=5.6" FWHM, но пиксели же???)

Fits, который прислал Blanc:

In [ ]:
from astropy.io import fits
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure(figsize=[24,16])
ax = plt.gca()
fname = '../../data/ngc628/blanc_fits/NGC0628_map_sigstar.fits'
data = fits.getdata(fname)
im = plt.imshow(np.squeeze(data), origin='lower', cmap='jet')

plt.title(fname, fontsize=19)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax)

plt.show()

In [ ]:
data = np.squeeze(data)
data = np.nan_to_num(data)
data.shape

Т.е. 60 на 160 pix. Посмотрим в сауроновских цветах и при оригинальном колорбаре, а также нанесем сетку:

In [ ]:
# X,Y = np.meshgrid(np.arange(0.,len(data[0]),1), np.arange(0.,len(data),1))
# Z = data

# # ax1,ax2 = plt.subplots(ncols=2, nrows=1, figsize=[16, 8])
# fig = plt.figure(figsize=[38, 12])
# quadmesh = plt.pcolormesh(X, Y, Z, cmap='sauron')
# quadmesh.set_clim(vmin=6., vmax=80.)

# # CS = plt.contour(X, Y, Z, levels=[5000, 10000, 20000])
# # plt.clabel(CS, inline=1, fontsize=10)

# plt.colorbar()

# ax = plt.gca()

# # Major ticks
# ax.set_xticks(np.arange(0, 160, 1));
# ax.set_yticks(np.arange(0, 60, 1));

# # # Labels for major ticks
# # ax.set_xticklabels(np.arange(1, 11, 1));
# # ax.set_yticklabels(np.arange(1, 11, 1));

# # # Minor ticks
# # ax.set_xticks(np.arange(-.5, 10, 0.5), minor=True);
# # ax.set_yticks(np.arange(-.5, 10, 0.5), minor=True);

# # Gridlines based on minor ticks
# ax.grid(which='major', color='w', linestyle='-', linewidth=2)
# # ax.grid(which='minor', color='w', linestyle='-', linewidth=1)

In [ ]:
# Image('VENGA_sig_los.png', width=1350)

Все очень похоже, хотя они явно снизу убрали часть пикселей (а также слева, справа и сверху). Это согласуется с анализом точек на картинке со статьи, где количество пикселей получилось равным (54L, 154L) (по одному видимо на конце картинки не учли).

In [ ]:
sig_los = Z[4:-1, 3:-2]

In [ ]:
np.save('sig_los.npy', sig_los)

Теперь надо $\sigma_R$ извлечь, используя формулу выше.

Рассчитаем углы $\phi$ и проверим что верно:

In [ ]:
import math

def dotproduct(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def length(v):
    return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
    return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))

In [ ]:
phi = np.zeros(sig_los.shape)

for i in range(len(sig_los[0])):
    for j in range(len(sig_los)):
        x, y  = 813.5*155./1553.-i, 253.5*55./513.-j
        phi[j][i] = angle(np.array([x,y]), np.array([155., 155.*np.sin(22.5*np.pi/180.)]))*180./np.pi

TODO: проверить, почему приходится дополнительно делать вычитание при отрисовке нулевой линии

In [ ]:
X,Y = np.meshgrid(np.arange(0.,len(sig_los[0]),1), np.arange(0.,len(sig_los),1))

plt.figure(figsize=[24, 8])
quadmesh = plt.pcolormesh(X, Y, phi, cmap=cm.jet)
# quadmesh.set_clim(vmin=6.0, vmax=80.)
plt.colorbar()
quadmesh.cmap.set_under('white')
quadmesh.cmap.set_over('white')

CS = plt.contour(X, Y, phi, levels=[45., 90., 135.], aspect='auto')

plt.plot([0., 160.], [0.-4.0, 160.*np.sin(22.5*np.pi/180.)-4.0], '-', lw=3)
plt.scatter([813.5*155./1553.],[253.5*55./513.],[150.])

In [ ]:
sig_r_max = lambda sig, phi_: sig/np.sqrt(sin_i**2 * np.sin(phi_*np.pi/180.)**2 + 0.25*sin_i**2*np.cos(phi_*np.pi/180.)**2 + 0.09*cos_i**2) 
sig_r_min = lambda sig, phi_: sig/np.sqrt(sin_i**2 + 0.49*cos_i**2)

In [ ]:
sig_R_max = np.zeros(sig_los.shape)

for i in range(len(sig_los)):
    for j in range(len(sig_los[0])):
        sig_R_max[i][j] = sig_r_max(sig_los[i][j], phi[i][j])
        
sig_R_min = np.zeros(sig_los.shape)

for i in range(len(sig_los)):
    for j in range(len(sig_los[0])):
        sig_R_min[i][j] = sig_r_min(sig_los[i][j], phi[i][j])

In [ ]:
plt.figure(figsize=[24, 12])
plt.subplot(2,1,1)
quadmesh = plt.pcolormesh(X, Y, sig_R_min, cmap=cm.jet)
quadmesh.set_clim(vmin=1.0, vmax=250.)
plt.colorbar()
quadmesh.cmap.set_under('white')
quadmesh.cmap.set_over('white')
plt.title(r'$\sigma_R^{min}$')

plt.subplot(2,1,2)
quadmesh = plt.pcolormesh(X, Y, sig_R_max, cmap=cm.jet)
quadmesh.set_clim(vmin=1.0, vmax=250.)
plt.colorbar()
quadmesh.cmap.set_under('white')
quadmesh.cmap.set_over('white')
plt.title(r'$\sigma_R^{max}$')
plt.show()

In [ ]:
np.save('sig_R_max.npy', sig_R_max)
np.save('sig_R_min.npy', sig_R_min)

Ошибки:

In [ ]:
# fig = plt.figure(figsize=[24,16])
# ax = plt.gca()
# fname = '../../data/ngc628/blanc_fits/NGC0628_map_siggas_err.fits'
# data = fits.getdata(fname)
# im = plt.imshow(np.squeeze(data), origin='lower', cmap='jet')

# plt.title(fname, fontsize=19)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)

# plt.colorbar(im, cax=cax)

# plt.show()

Подсчитаем относительную ошибку:

In [ ]:
fig = plt.figure(figsize=[24,16])
ax = plt.gca()
fname = '../../data/ngc628/blanc_fits/NGC0628_map_siggas_err.fits'
data_sl = fits.getdata(fname)
rel_sl_err = np.divide(np.squeeze(data_sl), np.squeeze(data))
im = plt.imshow(rel_sl_err, origin='lower', cmap=cm.Blues)

plt.title('Relative error', fontsize=19)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.show()

In [ ]:
np.save('rel_sl_err.npy', rel_sl_err[4:-1, 3:-2])

## Плотность $\Sigma_{\rm HI}$ (THINGS)

Куб данных из THINGS http://www.mpia.de/THINGS/Data.html - дисперсии $\sigma_{HI}$ и плотности $\Sigma_{HI}$.

TODO: про куб - разрешение, почему NA или RO, размеры бима, переводные коэффициенты?, расстояние, H_0 из

http://iopscience.iop.org/article/10.1088/0004-6256/136/6/2563/pdf - масштаб $1.5^{''}/pix$ (а в пк порядка 50 чтоли?)

In [ ]:
# from astropy.io import fits

# fig = plt.figure(figsize=[12,12])
# data = fits.getdata('NGC_628_NA_MOM0_THINGS.FITS')
# plt.imshow(np.squeeze(data), origin='lower', cmap='gist_yarg')
# plt.show()

$$\Sigma_{HI} [M_{sun} pc^{-2}] = 0.020 \cos i \times I_{21cm} [K km s^{-1}]$$

Leroy:**which accounts for inclination and includes a factor of 1.36 to
reflect the presence of helium!** Так что нужно /1.36

из https://arxiv.org/pdf/1609.03179.pdf $$\Sigma_{HI}[M_{sun} {pc}^{-2}] = 8840\times\frac{F_{HI}[Jy / beam \times km/s]}{bmaj\times bmin [arcsec]},$$
where the bmaj and bmin are the major and minor axis size of the beam, respectively. FHI is the flux detected across all velocity channels.

Чтобы перевести в кельвины в Walter написана формула
$$T[K] = 6.07\times 10^5\times S[Jy beam^{-1}]/(B_{maj}\times B_{min}),$$ где $B_{maj},B_{min}$ - размеры бима в секундах

In [ ]:
# заменяем nan на 0.
data = np.squeeze(data)
data = np.nan_to_num(data)

Т.к. мы использовал ```NA``` данные - размеры бима 11.88 и 9.30 arcsec

Не знаю почему, но похоже надо переводить beam в пк.

In [ ]:
f = lambda l: (6.07*2*1000*cos_i*l/(11.88*9.30*calc_scale(distance)**2 * 1000**2))/1.36
Z1 = f(data)

In [ ]:
Sigma_HI = Z1
np.save('Sigma_HI.npy', Sigma_HI)

Проверка и сравнение с данными Лероя дают схожие результаты.

Вырежем теперь cutout и сохраним данные по гриду:

In [ ]:
from astropy.visualization.wcsaxes import SphericalCircle

cutout, wcs = get_cutout('NGC_628_NA_MOM0_THINGS.FITS', size=size)

fig = plt.figure(figsize=[24, 8])
ax = fig.add_subplot(111, projection=cutout.wcs)
im = ax.imshow(cutout.data, origin='lower')

lon = ax.coords[0]
lat = ax.coords[1]
lon.set_major_formatter('dd:mm:ss')
lat.set_major_formatter('dd:mm:ss')

lon.set_ticks(spacing=30. * u.arcsec)
lat.set_ticks(spacing=30. * u.arcsec)

lon.display_minor_ticks(True)
lat.display_minor_ticks(True)
lat.set_minor_frequency(3)
lon.set_minor_frequency(3)

# r = SphericalCircle((24.174042 * u.deg, 15.783472 * u.deg), 1500 * u.arcsec,
#                      edgecolor='yellow', facecolor='none', transform=ax.get_transform(cutout.wcs))
# ax.add_patch(r)
plt.show()

In [ ]:
%%time
cutout = plot_cutout('NGC_628_NA_MOM0_THINGS.FITS', size=size, coordinates=True, figsize=[24, 8], title='hi_density', cmap='sauron')

hi_grid_pixs = []
Sigma_HI_cut = []
for scoord in sgrid.ravel():
    spix = skycoord_to_pixel(scoord, cutout.wcs)
    hi_grid_pixs.append(spix)
    Sigma_HI_cut.append(f(cutout.data[int(spix[1])][int(spix[0])]))
plt.plot(zip(*hi_grid_pixs)[0],  zip(*hi_grid_pixs)[1], '.', ms=3, color='r')
plt.show()

In [ ]:
len(hi_grid_pixs)

Площадка еще раз

In [ ]:
# cutout = plot_cutout('NGC_628_NA_MOM0_THINGS.FITS', size=size, coordinates=True, figsize=[24, 8], title='hi_density', cmap='sauron')

# plot_rectange_from_grid(plt.gca(), hi_grid_pixs)
# plt.show()

In [ ]:
Sigma_HI_cut = np.array(Sigma_HI_cut).reshape((55, 155))

In [ ]:
# plt.figure(figsize=[18, 7])
# X,Y = np.meshgrid(np.arange(0.,155,1), np.arange(0.,55,1))
# Z = Sigma_HI_cut
# quadmesh = plt.pcolormesh(X, Y, Z, cmap=cm.gist_yarg)
# plt.colorbar()
# quadmesh.cmap.set_under('white')

In [ ]:
np.save('Sigma_HI_cut.npy', Sigma_HI_cut)

## Дисперсии $\sigma_{\rm HI}$ (THINGS)

C картинкой в статье совпадает (только в ней вроде убраны с малым S/N точки):

In [ ]:
# data = fits.getdata('NGC_628_NA_MOM2_THINGS.FITS')
# data = np.squeeze(data)
# data = np.nan_to_num(data)

# X,Y = np.meshgrid(np.arange(0.,len(data),1), np.arange(0.,len(data[0]),1))
# Z = data

# # ax1,ax2 = plt.subplots(ncols=2, nrows=1, figsize=[16, 8])
# plt.figure(figsize=[30, 12])
# plt.subplot(1, 2, 1)
# quadmesh = plt.pcolormesh(X, Y, Z, cmap=cm.gist_yarg)
# quadmesh.set_clim(vmin=4000., vmax=30000.)
# plt.colorbar()

# CS = plt.contour(X, Y, Z, levels=[5000, 10000, 20000])
# quadmesh.cmap.set_under('white')

# plt.subplot(1, 2, 2)
# plt.imshow(ImagePIL.open('THINGS_HI_disp.png'))

In [ ]:
c_g = Z/1000.
np.save('c_g.npy', c_g)

Нужная нам область:

In [ ]:
%%time
cutout = plot_cutout('NGC_628_NA_MOM2_THINGS.FITS', size=size, coordinates=True, figsize=[24, 8], title='hi_disp', cmap='sauron')

hi_grid_pixs = []
sigma_HI_cut = []
for scoord in sgrid.ravel():
    spix = skycoord_to_pixel(scoord, cutout.wcs)
    hi_grid_pixs.append(spix)
    sigma_HI_cut.append(cutout.data[int(spix[1])][int(spix[0])]/1000.)
plt.plot(zip(*hi_grid_pixs)[0],  zip(*hi_grid_pixs)[1], '.', ms=3, color='r')
plt.show()

In [ ]:
len(hi_grid_pixs)

Площадка еще раз

In [ ]:
# cutout = plot_cutout('NGC_628_NA_MOM2_THINGS.FITS', size=size, coordinates=True, figsize=[24, 8], title='hi_disp', cmap='sauron')
# plot_rectange_from_grid(plt.gca(), hi_grid_pixs)
# plt.show()

In [ ]:
sigma_HI_cut = np.array(sigma_HI_cut).reshape((55, 155))

In [ ]:
plt.figure(figsize=[18, 7])
X,Y = np.meshgrid(np.arange(0.,155,1), np.arange(0.,55,1))
Z = sigma_HI_cut
quadmesh = plt.pcolormesh(X, Y, Z, cmap=cm.gist_yarg)
plt.colorbar()
quadmesh.cmap.set_under('white')

In [ ]:
np.save('c_g_cut.npy', sigma_HI_cut)

## Кривая вращения и эпициклическая частота

Я разрежу сам руками вдоль большой оси карту скоростей THINGS и сравню ее с литературными.

In [ ]:
fig = plt.figure(figsize=[12,12])
data = fits.getdata('NGC_628_NA_MOM1_THINGS.FITS')
# plt.imshow(np.squeeze(data), origin='lower', cmap='gist_yarg')
# plt.show()

In [ ]:
data = np.squeeze(data)
data = np.nan_to_num(data)

In [ ]:
veloc_norm = lambda l: (l/1000. - 659.1)/sin_i
Z = veloc_norm(data)

In [ ]:
v_HI = Z

In [ ]:
np.save('v_HI.npy', v_HI)

In [ ]:
# https://stackoverflow.com/questions/7878398/how-to-extract-an-arbitrary-line-of-values-from-a-numpy-array
import scipy.ndimage

#-- Extract the line...
# Make a line with "num" points...
x0, y0 = 647, 200 # These are in _pixel_ coordinates!!
x1, y1 = 387, 800
num = 1000
x, y = np.linspace(x0, x1, num), np.linspace(y0, y1, num)

# Extract the values along the line, using cubic interpolation
zi = scipy.ndimage.map_coordinates(v_HI, np.vstack((y,x)))
zi_cub = v_HI[y.astype(np.int), x.astype(np.int)]


x0_, y0_ = 100, 325 # These are in _pixel_ coordinates!!
x1_, y1_ = 923, 695
num = 1000
x_, y_ = np.linspace(x0_, x1_, num), np.linspace(y0_, y1_, num)
zi_ = scipy.ndimage.map_coordinates(v_HI, np.vstack((y_,x_)))

#-- Plot...
fig, axes = plt.subplots(ncols=2, figsize = [30, 12])

axes[0].plot(zi)
axes[0].plot(zi_cub)
axes[0].plot(zi_)
axes[0].set_ylim(-300., 300.)
# axes[0].imshow(v_HI)
# axes[1].axis('image')
X,Y = np.meshgrid(np.arange(0.,len(data),1), np.arange(0.,len(data[0]),1))
quadmesh = plt.pcolormesh(X, Y, v_HI, cmap=cm.Blues)
quadmesh.set_clim(vmin=-300., vmax=300.)
plt.colorbar()
# axes[1].imshow(v_HI[::-1, ])
CS = plt.contour(X, Y, v_HI, levels=[-100., 0., 100.])
axes[1].plot([x0, x1], [y0, y1], 'ro-')
axes[1].plot([x0_, x1_], [y0_, y1_], 'ro-', alpha=0.5)
axes[1].scatter([512.,], [512.,], [30.,], marker='s', color='r')
plt.show()

Сделаем, чтобы расстояния примерно соответствовали нужным (потому как на картинке слева - не верна общая длина):

In [ ]:
# sqrt((647.-387.)**2 + (200.-800.)**2) * 1520./1024.
sqrt((647.-387.)**2 + (200.-800.)**2) * 1.5

In [ ]:
r_g, vel_g = zip(*sorted(zip(np.abs(np.linspace(0., 970.6, 1000)-970.6/2), np.abs(zi))))

In [ ]:
np.save('r_g.npy', r_g)
np.save('vel_g.npy', vel_g)

In [ ]:
herrman2009_vc=[0.002107968676327232, 1.8461367775815631,
0.9211823115540909, 6.761771803145308,
2.389733822727237, 12.903726851637828,
4.225071883580966, 20.273481199323243,
5.876314013368969, 26.4139569715517,
7.162877562152744, 33.17277022216879,
8.630023760875005, 38.08396741894032,
9.915181997207867, 43.61202281783639,
11.930400051774697, 48.51878218581564,
12.666081119812134, 52.820517561782935,
14.316620593374722, 58.34561440815084,
16.150553341777538, 64.48461090411519,
18.16577139634434, 69.39137027209442,
19.999001488521742, 74.91498784219823,
22.197612817928857, 80.4356468597739,
23.846746979040518, 84.72998585442076,
25.67927441499249, 89.63822449866407,
27.87648043194868, 93.92812566451867,
29.891698486515494, 98.8348850324979,
32.27089246586111, 102.50792799622789,
34.83418237627244, 107.41024953541485,
36.84729246216288, 110.47087212581249,
39.22789175395938, 115.37467294126353,
42.522646795055536, 120.88645630125467,
45.44991263024566, 124.5550614361924,
49.1100489085715, 130.06388624365536,
53.49954234890582, 134.3360360943409,
56.975582696165915, 138.6155823263469,
60.2675271123603, 141.66584998289593,
63.3760782537144, 144.10221798984853,
67.39948780059358, 147.76194746720174,
70.50803894194767, 150.19831547415433,
73.7992807019166, 152.63320420484285,
75.99297343774562, 153.84621074139486]

In [ ]:
leroy2011_vc = [5.327946127946127, 1.941747572815558,
6.6599326599326645, 14.5631067961165,
7.991919191919187, 28.155339805825264,
8.65791245791246, 42.718446601941764,
10.655892255892255, 52.42718446601944,
11.321885521885527, 66.01941747572818,
12.653872053872057, 78.64077669902912,
15.983838383838382, 94.17475728155341,
17.315824915824912, 107.76699029126215,
19.313804713804707, 118.44660194174757,
22.643771043771046, 131.06796116504856,
25.97373737373737, 137.86407766990294,
29.96969696969696, 147.5728155339806,
35.96363636363636, 159.2233009708738,
40.625589225589216, 166.99029126213594,
44.62154882154882, 172.81553398058253,
51.947474747474736, 179.61165048543688,
58.6074074074074, 187.37864077669903,
65.93333333333334, 190.29126213592235,
73.92525252525252, 192.23300970873788,
81.91717171717173, 196.11650485436894,
87.9111111111111, 197.0873786407767,
96.56902356902356, 198.05825242718447,
107.22491582491583, 198.05825242718447,
121.2107744107744, 200.97087378640776,
129.86868686868686, 200,
139.85858585858585, 200,
149.1824915824916, 200,
162.5023569023569, 200,
171.82626262626263, 199.02912621359224,
181.81616161616162, 199.02912621359224,
191.8060606060606, 200,
201.12996632996632, 199.02912621359224,
211.7858585858586, 201.94174757281553,
223.77373737373733, 199.02912621359224,
232.43164983164985, 199.02912621359224,
241.08956228956225, 199.02912621359224,
252.41144781144783, 199.02912621359224,
261.7353535353535, 200,
270.39326599326597, 199.02912621359224,
281.7151515151515, 198.05825242718447,
289.0410774410774, 200]

In [ ]:
suwannajak2014_vc = [0.18715761885325177, 30.105909901394284,
0.23961846656211838, 42.149133171496715,
0.2919504178392671, 52.688564738232856,
0.32713914369803954, 63.22369975724507,
0.3966143203935646, 73.76742787170508,
0.4490751681024303, 85.81065114180751,
0.5184214483662384, 94.85058755290123,
0.5707533996433876, 105.39001911963737,
0.6402285763389126, 115.93374723409732,
0.6921738383209095, 121.96180369073448,
0.7956776729897537, 129.5065414939096,
0.8650239532535617, 138.54647790500326,
0.9515134589357461, 147.59071086382096,
1.0204730499044024, 152.11927216481553,
1.124105781004963, 161.16780167135704,
1.210208597391997, 165.7006595200756,
1.3135835356291237, 171.7416056198844,
1.3825431265977786, 176.27016692087903,
1.5374766375217526, 183.82779436722592,
1.675138026595631, 189.87733356248253,
1.7953983973877001, 192.91499280328253,
1.898515542761392, 195.94835549635866,
2.018775913553461, 198.98601473715865,
2.1561795097639056, 202.02797052568263,
2.2764398805559747, 205.06562976648263,
2.4137145803347018, 206.6037938516402,
2.5682614019635226, 209.650046187888,
2.6882639798921577, 209.6801220219553,
2.842681905089261, 211.2225826548368,
2.996970933854648, 211.2612515843519,
3.1513888590517523, 212.8037122172334,
3.30567788781714, 212.84238114674855,
3.4600958130142443, 214.38484177963,
3.631656963629723, 215.93159896023548,
3.7516595415583582, 215.96167479430278,
3.9402350211604977, 216.00893681926573,
4.094524049925885, 216.04760574878088,
4.265956304109648, 216.0905712260199,
4.437646351156847, 219.14112010999162,
4.5919353799222336, 219.17978903950677,
4.746224408687621, 219.21845796902193,
4.831940535779502, 219.23994070764144,
5.003372789963266, 219.28290618488046,
5.140518593310276, 219.3172785666717,
5.311950847494039, 219.3602440439107,
5.449096650841049, 219.39461642570194,
5.603256783174718, 217.92949365185075,
5.706116135684978, 217.95527293819418]

In [ ]:
import scipy.interpolate as inter
fig = plt.figure(figsize=[10, 6])
gas_approx = poly1d(polyfit(r_g, vel_g, deg=5))
test_points = np.linspace(min(r_g), max(r_g), 100)
plt.plot(test_points, gas_approx(test_points), '--', label='approx')

plt.plot(herrman2009_vc[0::2], herrman2009_vc[1::2], '-', label='Herrman+2009')
plt.plot(leroy2011_vc[0::2], leroy2011_vc[1::2], '-', label='Leroy+2011')
plt.plot(map(lambda l: l/calc_scale(9.7), suwannajak2014_vc[0::2]), suwannajak2014_vc[1::2], '-', label='Suwannajak+2014')

gas_approx2 = poly1d(polyfit(leroy2011_vc[0::2], leroy2011_vc[1::2], deg=7))
plt.plot(test_points[:-50], gas_approx2(test_points[:-50]), '--', label='approx2')

plt.plot(r_g, vel_g, 'g.')
# plt.ylim(0, 300)
plt.legend(loc='lower right');

Данные хорошо согласуются (похоже, что две работы фитировали только одну сторону кривой) и извлеченный профиль тоже на них похож (отличия есть, но это все не так существенно).

Эпициклическая частота:

$$\kappa=\sqrt{2}\frac{\vartheta_c}{R}\sqrt{1+\frac{R}{\vartheta_c}\frac{d\vartheta_c}{dR}}$$

In [ ]:
def epicyclicFreq_real(poly_gas, R, resolution):
    '''Честное вычисление эпициклической частоты на расстоянии R для сплайна или полинома'''
    try:
        return sqrt(2.0) * poly_gas(R) * sqrt(1 + R * poly_gas.deriv()(R) / poly_gas(R)) / (R * resolution )
    except:
        return sqrt(2.0) * poly_gas(R) * sqrt(1 + R * poly_gas.derivative()(R) / poly_gas(R)) / (R * resolution )

fig = plt.figure(figsize=[18, 8])
plt.plot(test_points, [epicyclicFreq_real(gas_approx, x, scale) for x in test_points], '-', label='gas_approx')
# plt.plot(test_points, [epicyclicFreq_real(gas_approx3, x, scale) for x in test_points], '-', label='gas_approx3')
plt.plot(test_points[2:-50], [epicyclicFreq_real(gas_approx2, x, scale) for x in test_points[2:-50]], '-', label='gas_approx Leroy')

# plt.plot(r_g_dens[1:], ef, '--')
plt.xlabel('$R, arcsec$')
plt.ylabel('$\kappa,\, km/s/kpc$', fontsize=15)
plt.xlim(0)
plt.axvline(x=15.) #bulge
plt.ylim(0, 1200.)
plt.legend()

Для ошибок используем следующий разброс:

In [ ]:
import scipy.interpolate as inter
fig = plt.figure(figsize=[10, 6])
gas_approx = poly1d(polyfit(r_g, vel_g, deg=5))
test_points = np.linspace(min(r_g), max(r_g), 100)
plt.plot(test_points, gas_approx(test_points), '--', label='approx')

gas_approx_u = gas_approx * 1.07
gas_approx_b = gas_approx * 0.93
plt.fill_between(test_points, gas_approx_u(test_points), gas_approx_b(test_points), color='b', alpha=0.3)

# plt.plot(herrman2009_vc[0::2], herrman2009_vc[1::2], '-', label='Herrman+2009')
# plt.plot(leroy2011_vc[0::2], leroy2011_vc[1::2], '-', label='Leroy+2011')
# plt.plot(map(lambda l: l/calc_scale(9.7), suwannajak2014_vc[0::2]), suwannajak2014_vc[1::2], '-', label='Suwannajak+2014')

gas_approx2 = poly1d(polyfit(leroy2011_vc[0::2], leroy2011_vc[1::2], deg=7))
plt.plot(test_points[:-50], gas_approx2(test_points[:-50]), '--', label='approx2')

plt.plot(r_g, vel_g, 'g.')
# plt.ylim(0, 300)
plt.legend(loc='lower right');

Картинка для статьи

In [ ]:
r_g, vel_g = zip(*sorted(zip(np.abs(np.linspace(0., 980.6, 1000)-990.6/2), zi)))
plt.plot(r_g, vel_g, 'g.')
plt.axvline(x=0.)
plt.xlim(-220, 220)
plt.axhline(y=200.)
plt.axhline(y=0.)
plt.axhline(y=-35.)
plt.axhline(y=-200.)

In [ ]:
import matplotlib as mpl
mpl.style.use('classic')

In [ ]:
fig = plt.figure(figsize=[15, 9])

ax = plt.gca()

r_g, vel_g = zip(*sorted(zip(np.abs(np.linspace(0., 980.6, 1000)-990.6/2), np.abs(zi+35.))))
# plt.plot(r_g, vel_g, 'g.', label='THINGS')

r_g, vel_g = np.array(r_g), np.array(vel_g)
az_av, az_dev = [], []
for dbin in range(0, 210, 10):
    mask = (r_g >= dbin) & (r_g <= dbin + 10)
    anulli = vel_g[mask]
    az_av.append(anulli.mean())
    az_dev.append(np.std(anulli))
az_av, az_dev = np.array(az_av), np.array(az_dev)

# plt.fill_between(np.array(range(0, 210, 10))+5., az_av-az_dev, az_av+az_dev, color='g', alpha=0.3)
plt.errorbar(np.array(range(0, 210, 10))+5., az_av, az_dev, 
             fmt='.', marker='.', mew=2, label = r'$\mathrm{THINGS}$', ms=20, color='b')

# plt.plot(herrman2009_vc[0::2], np.array(herrman2009_vc[1::2]), '-', label='Herrman+2009')
plt.plot(leroy2011_vc[0::2], leroy2011_vc[1::2], ':', label=r'$\mathrm{Vollmer\, et\, al. (2011)}$', lw=5)
plt.plot(map(lambda l: l/calc_scale(9.7), suwannajak2014_vc[0::2]), suwannajak2014_vc[1::2], '-.', 
         label=r'$\mathrm{Suwannajak\, et\, al.\, (2014)}$', lw=5)

vflat = 217
lflat = 0.8 #kpc

vrot = lambda l: vflat*(1 - np.exp(-l/lflat))
plt.plot([l/calc_scale(7.3) for l in np.linspace(0, 10, 100)], [vrot(l) for l in np.linspace(0, 10, 100)], 
         '--', label=r'$\mathrm{Leroy\, et\, al. (2008)}$', lw=4, alpha=0.7)

gas_approx3 = poly1d(polyfit(r_g, vel_g, deg=5))
test_points = np.linspace(min(r_g), max(r_g), 100)
plt.plot(test_points, gas_approx3(test_points), '-', label=r'$v_\mathrm{c}$', lw=5, color='m')

gas_approx_u = gas_approx * 1.07
gas_approx_b = gas_approx * 0.93
plt.fill_between(test_points, gas_approx_u(test_points), gas_approx_b(test_points), color='m', alpha=0.1)

# gas_approx2 = poly1d(polyfit(leroy2011_vc[0::2], leroy2011_vc[1::2], deg=7))
# plt.plot(test_points[:-50], gas_approx2(test_points[:-50]), '--', label='approx2')
plt.plot(test_points, [epicyclicFreq_real(gas_approx3, x, scale) for x in test_points], 
         '-', label=r'$\varkappa$', color='grey', lw=7, alpha=0.3)


plt.xlim(0, 200)
plt.ylim(0, 260)
plt.legend(fontsize=28, bbox_to_anchor=[0.86, 0.32], ncol=2)

plt.xlabel(r'$R,\,\mathrm{arcsec}$', fontsize=45, rotation=0)
plt.ylabel(r'$v,\,\mathrm{km/s}$', fontsize=50)

ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(2))
plt.grid(ls='-', alpha=0.5)

# ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
ax.set_yticklabels(range(0, 350, 50), fontsize=35)
ax.set_xticklabels(range(0, 250, 50), fontsize=35)
# if ind == 1:
#     ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)

ax.tick_params(axis='both', length=7, width=2)
ax.tick_params(axis='both', length=4, width=1, which='minor')

ax2 = ax.twinx()
ax2.set_ylabel(r'$\varkappa,\, \mathrm{km/s/kpc}$', fontsize=50)
ax2.plot([0.0], [-1.])
ax2.set_ylim(0, 260)
ax2.set_yticklabels([], fontsize=25)

plt.savefig(paper_imgs_dir3 +'rotvel.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'rotvel.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'rotvel.pdf', format='pdf', dpi=150, bbox_inches='tight')

plt.show()

## SFR (VENGA, H$\alpha$)

Есть картинка Лероя где больше в центре и не удается восстановить из colorbar правильную шкалу и есть JCMT $H_{\alpha}$ изображение, где видно что оно хорошо накладывается на $HI$ картинку 

Это все хорошо, но мы будем использовать $log(\Sigma_{SFR})$ Dust Corrected H$\alpha$ [$M_{\odot} yr^{-1} kpc^{-1}$] из http://iopscience.iop.org/article/10.1088/0004-637X/764/2/117/pdf.

А точнее FITS, дюбезно предоставленный Бланком

* http://iopscience.iop.org/article/10.1088/0004-637X/737/2/67/pdf
* http://iopscience.iop.org/article/10.1088/0004-637X/741/2/124/pdf

Murphy (2011) - первая ссылка: $$\frac{SFR}{M_{\odot}yr^{-1}} = 5.37\times 10^{-42} \frac{L_{H\alpha}}{erg \times s^{-1}}$$

(хотя например Kennicutt там может до 7.9 коэффециент давать http://iopscience.iop.org/article/10.1088/0004-637X/703/2/1672/pdf)

Проблема пересчитать в том, что я не знаю какие единицы в их фитсе - но похоже erg/s/cm^2.

In [ ]:
Image('VENGA_Halpha_SFR_wbar.png')

А вот как выглядит исходная картинка из другой статьи:

In [ ]:
Image('VENGA_Halpha.png')

In [ ]:
from astropy.io import fits
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure(figsize=[24,16])
ax = plt.gca()
fname = '../../data/ngc628/blanc_fits/NGC0628_map_ha_corr.fits'
data = fits.getdata(fname)
im = plt.imshow(np.squeeze(np.log10(data)), origin='lower', cmap='jet')

plt.title(fname, fontsize=19)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax)

plt.show()

Чтобы перевести в $L$ используем формулу 5 из http://iopscience.iop.org/article/10.1086/305331/fulltext/ и для площадки 86 пк считаем (86 = calc_scale(8.6)*1000*10/4.8)

In [ ]:
fname = '../../data/ngc628/blanc_fits/NGC0628_map_ha_corr.fits'
data = fits.getdata(fname)
data = np.squeeze(data) * (((5.37*4*pi*(3.086*8.6)**2)*1e6)/(0.086)**2)

In [ ]:
# from astropy.io import fits
# from mpl_toolkits.axes_grid1 import make_axes_locatable

# X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))

# fig = plt.figure(figsize=[24,8])
# ax = plt.gca()
# quadmesh2 = ax.pcolormesh(X, Y, np.log10(data)[4:-1, 3:-2], cmap='sauron')
# quadmesh2.set_clim(vmin=-3.5, vmax=-1.)
# plt.colorbar(quadmesh2)

# plt.show()

В принципе похоже.

In [ ]:
np.save('Halpha_SFR_.npy', np.log10(data)[4:-1, 3:-2])
np.save('Halpha_SFR_only.npy', data[4:-1, 3:-2] * (0.086)**2)

In [ ]:
log_Halpha_SFR_ = np.load('Halpha_SFR_.npy')

SFR contours:

In [ ]:
def add_SFR_contours(levels=[-2.], colors=('red', 'blue'), X=None, Y=None):
    if X is None and Y is None:
        X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
    CS = plt.contour(X, Y, log_Halpha_SFR_, levels=levels, colors=colors)

fig = plt.figure(figsize=[24,8])
ax = plt.gca()
quadmesh2 = ax.pcolormesh(X, Y, np.log10(data)[4:-1, 3:-2], cmap='sauron')
quadmesh2.set_clim(vmin=-3.5, vmax=-1.)
plt.colorbar(quadmesh2)

add_SFR_contours()

plt.show()

How to mark stars - two ways, proper and less (есть разница в шкалах, делать для imshow или pcolormesh):

In [ ]:
# def mark_stars(colors=('white'), levels=[-28.1, -28.]):
#     X,Y = np.meshgrid(np.arange(0.5,Nx+0.5,1), np.arange(0.5,Ny+0.5,1))
#     mask = sig_los == 0
    
# #     mask = np.ma.masked_where(sig_los>250, sig_los)
# #     CS = plt.contour(X, Y, mask, levels=[0.99, 1.0], colors=colors, interpolation='none', origin='lower')
#     CS3 = plt.contourf(X, Y, mask, levels, colors=colors, interpolation='none')

# X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))

# fig = plt.figure(figsize=[24,16])    
# ax = fig.add_subplot(211)
# # im = ax.imshow(sig_los, interpolation='none', cmap='jet')
# quadmesh2 = ax.pcolormesh(X, Y, sig_los, cmap=cm.jet)
# mark_stars(colors=('m'), levels=[0.999, 1.001])
# ax.set_title('Way using contourf', fontsize=19)
# ax.set_xlim(-0.5)

# ax = fig.add_subplot(212)
# masked_array = np.ma.array(sig_los, mask=sig_los == 0)
# cmap = matplotlib.cm.jet
# cmap.set_bad('m',1.)
# ax.imshow(masked_array[::-1, :], interpolation='nearest', cmap=cmap)
# ax.set_title('Prover way using set_bad', fontsize=19)
# ax.set_xlim(-0.5)

# fig.subplots_adjust(wspace=0.0, hspace=0.0)
# plt.tight_layout()
# plt.show()

Ошибки:

In [ ]:
fig = plt.figure(figsize=[24,16])
ax = plt.gca()
fname = '../../data/ngc628/blanc_fits/NGC0628_map_ha_corr_err.fits'
data = fits.getdata(fname)
data = np.squeeze(data) * (((5.37*4*pi*(3.086*8.6)**2)*1e6)/(0.086)**2)
im = plt.imshow(np.squeeze(np.log10(data)), origin='lower', cmap='jet')

plt.title(fname, fontsize=19)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax)

plt.show()

In [ ]:
np.save('Halpha_SFR_err.npy', np.log10(data)[4:-1, 3:-2])

## Плотность $\Sigma_{\rm CO}$ (VENGA, HERACLES)

В http://iopscience.iop.org/article/10.1088/0004-637X/764/2/117/pdf выбор из трех возможностей:

* HERACLES, CO(2–1) beam size of $13.6$ and rms noise of ∼22 mK per 2.6 km s−1 channel, which translates into a 1σ limit on the molecular gas surface density of $\Sigma_{H2}\approx 3 M_{\odot} pc^{-2}$
* BIMA-SONG, robust beam size of $6.2$, and rms noise of 51 mJy beam−1 in a 10 km s−1 channel or $\Sigma_{H2}\approx 13 M_{\odot} pc^{-2}$
* CARMA, CO(1–0) has a spatial resolution of $3.6$ and rms noise of ∼20 mJy beam−1 in a 10 km s−1 channel $\Sigma_{H2}\approx 5 M_{\odot} pc^{-2}$

Т.о. лучше HERACLES, поскольку в смысле пространственного разрешения они все приведены к одному
>We
convolve the BIMA-SONG and CARMA maps with a Gaussian
kernel to match the 13.6 arcsec PSF of the HERACLES map, which
corresponds to ∼570 pc at the assumed distance to NGC 0628.

Куб HERACLES и ошибки из https://www.cv.nrao.edu/~aleroy/heracles_data/ (не работает, загрузил из NED)

TODO: учеть другой угол и расстояние - у них For NGC 628, we assume an inclination of i = 8.7 and a distance of 8.6 Mpc (Herrmann et al. 2008).

Теперь поверхностные плотности. 

Leroy делал $$\Sigma_{H_2} = 4.4\cos i \times I_{CO(1-0)}$$ $$\Sigma_{H_2} = 5.5\cos i \times I_{CO(2-1)}$$ для $X_{CO}=2\times10^{20}$

В HERACLES $CO(2-1)$ изначально, но карта в статье уже переведена в $CO(1-0)$, так что используем первую формулу.

Эти же формулы в HERACLES (3) & (4) - **они уже исправлены за наличие гелия!!**

Сравним с 2008м годом:

In [ ]:
H2_raw = ImagePIL.open('Leroy_H2_scaled.png')
colorbar = ImagePIL.open('Leroy_HI_colorbar.png')
cw, ch = colorbar.width, colorbar.height
colorbar = np.array(list(colorbar.getdata())).reshape(ch, cw, 4)[:, 10:11, :].reshape(ch, 4)

In [ ]:
fname = '../../data/ngc628/ngc0628_heracles_mom0.fits'
data = np.squeeze(fits.getdata(fname))
data = data*5.5*cos_i

In [ ]:
%%time
Z = image_map_to_colorbar(H2_raw, colorbar, 33.89, 1.)

In [ ]:
# X,Y = np.meshgrid(np.arange(0.,862,1), np.arange(0.,862,1))

# plt.figure(figsize=[20, 8])
# plt.subplot(1, 2, 1)
# quadmesh = plt.pcolormesh(X, Y, Z*2, cmap=cm.gist_heat_r)
# quadmesh.set_clim(vmin=10.0, vmax=40.)
# plt.colorbar()

# plt.subplot(1, 2, 2)
# X,Y = np.meshgrid(np.arange(0.,len(data[0]),1), np.arange(0.,len(data),1))

# quadmesh = plt.pcolormesh(X, Y, data, cmap=cm.gist_heat_r)
# quadmesh.set_clim(vmin=10.0, vmax=40.)
# plt.colorbar()

Видно, что разница примерно в два раза по оценкам - что-то Leroy сделал не так в своей работе, поскольку позже в [Leroy 2015](http://iopscience.iop.org/article/10.1088/0004-637X/808/1/99/pdf) плотности уже до 40 $M_{\odot}pc^{-2}$ (может ли это быть связано с разрешением?).

Причем это верно и для снятой картинки со статьи.

http://iopscience.iop.org/article/10.1088/0004-6256/137/6/4670/pdf

- NAME_heracles.hans.fits - the data cube
- NAME_heracles_mom0.fits - the integrated intensity map
- NAME_heracles_emom0.fits - uncertainty in the integrated intensity map
- NAME_heracles_mom1.fits - the velocity field

In [ ]:
# from astropy.io import fits
# from mpl_toolkits.axes_grid1 import make_axes_locatable

# fig = plt.figure(figsize=[24,16])
# plt.subplot(121)
# ax = plt.gca()
# fname = '../../data/ngc628/ngc0628_heracles_mom0.fits'
# data = fits.getdata(fname)
# im = plt.imshow(np.squeeze(data), origin='lower', cmap='jet')

# plt.title(fname, fontsize=19)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)

# # plt.subplot(122)
# # ax = plt.gca()
# # fname = '../../data/ngc628/ngc0628_heracles_mom1.fits'
# # data = fits.getdata(fname)
# # im = plt.imshow(np.squeeze(data), origin='lower', cmap='jet')

# # plt.title(fname, fontsize=19)
# # divider = make_axes_locatable(ax)
# # cax = divider.append_axes("right", size="5%", pad=0.05)
# # plt.colorbar(im, cax=cax)
# # plt.show()

Нужная нам область (для перевода из BUNIT = 'K KM/S'/Tmb, integrated over line используем формулу выше, она же (4) в статье о HERACLES):

In [ ]:
%%time
cutout = plot_cutout('../../data/ngc628/ngc0628_heracles_mom0.fits', size=size, coordinates=True, figsize=[24, 8], title='co_density', cmap='sauron')

co_grid_pixs = []
Sigma_CO_cut = []
for scoord in sgrid.ravel():
    spix = skycoord_to_pixel(scoord, cutout.wcs)
    co_grid_pixs.append(spix)
    Sigma_CO_cut.append(cutout.data[int(spix[1])][int(spix[0])]*5.5*cos_i/1.36)
plt.plot(zip(*co_grid_pixs)[0],  zip(*co_grid_pixs)[1], '.', ms=3, color='r')
plt.show()

In [ ]:
len(co_grid_pixs)

Площадка еще раз

In [ ]:
# cutout = plot_cutout('../../data/ngc628/ngc0628_heracles_mom0.fits', size=size, coordinates=True, figsize=[24, 8], title='co_density', cmap='sauron')
# plot_rectange_from_grid(plt.gca(), co_grid_pixs)
# plt.show()

In [ ]:
Sigma_CO_cut = np.array(Sigma_CO_cut).reshape((55, 155))

In [ ]:
#убираем странную область в левом нижнем углу
Sigma_CO_cut[Sigma_CO_cut < 0] = 0

In [ ]:
plt.figure(figsize=[18, 7])
X,Y = np.meshgrid(np.arange(0.,155,1), np.arange(0.,55,1))
Z = Sigma_CO_cut
quadmesh = plt.pcolormesh(X, Y, Z, cmap='jet')
plt.colorbar()
quadmesh.cmap.set_under('white')

In [ ]:
np.save('Sigma_CO_cut.npy', Sigma_CO_cut)

Ошибки:

In [ ]:
# from astropy.io import fits
# from mpl_toolkits.axes_grid1 import make_axes_locatable

# fig = plt.figure(figsize=[24,16])
# plt.subplot(121)
# ax = plt.gca()
# fname = '../../data/ngc628/ngc0628_heracles_mom0.fits'
# data = fits.getdata(fname)
# im = plt.imshow(np.squeeze(data), origin='lower', cmap='jet')

# plt.title(fname, fontsize=19)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)

# plt.subplot(122)
# ax = plt.gca()
# fname = '../../data/ngc628/ngc0628_heracles_emom0.fits'
# data = fits.getdata(fname)
# im = plt.imshow(np.squeeze(data), origin='lower', cmap='jet')

# plt.title(fname, fontsize=19)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)
# plt.show()

In [ ]:
# size = (120, 350)
# cutout = plot_cutout('../../data/ngc628/ngc0628_heracles_mom0.fits', size=size, coordinates=True, figsize=[24, 8], title='co_density', cmap='sauron')
# cutout_err = plot_cutout('../../data/ngc628/ngc0628_heracles_emom0.fits', size=size, coordinates=True, figsize=[24, 8], title='co_density', cmap='sauron')

# Sigma_CO_cut_err = []
# for scoord in sgrid.ravel():
#     spix = skycoord_to_pixel(scoord, cutout.wcs)
#     Sigma_CO_cut_err.append(cutout_err.data[int(spix[1])][int(spix[0])]/cutout.data[int(spix[1])][int(spix[0])])

# Sigma_CO_cut_err = np.array(Sigma_CO_cut_err).reshape((55, 155))

In [ ]:
# fig = plt.figure(figsize=[24,16])
# ax = plt.gca()
# im = plt.imshow(Sigma_CO_cut_err, origin='lower', cmap=cm.Blues)
# im.set_clim(0., .5)

# plt.title('Relative error', fontsize=19)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# cb = plt.colorbar(im, cax=cax)
# plt.show()

# np.save('Sigma_CO_cut_relerr.npy', Sigma_CO_cut_err)

In [ ]:
Sigma_CO_cut_err = np.zeros((55, 155))

## Плотность $\Sigma_{\rm s}$ (S4G)

другая возможность - использовать cleaned stellar fits из http://irsa.ipac.caltech.edu/data/SPITZER/S4G/docs/P5_README.html чтобы восстановить с помощью работы [Querejeta et al. (2014)](https://ui.adsabs.harvard.edu/#abs/2014ASPC..486..143Q/abstract)

Используем ровно ту методику, что Leroy применил в своей работе к S4G данным: $$\Sigma_s = 280\cos i\times I_{3.6}$$

TODO: проверить, что именно к ним, добавить разрешение кубов, размеры бима

Кубы из https://irsa.ipac.caltech.edu/cgi-bin/Atlas/nph-atlas?mission=S4G&hdr_location=\S4GDataPath\&locstr=MESSIER+074&regSize=0.00555556&radius=0.166667&radunits=arcmin&searchregion=off&covers=none&band=A

S4G Pipeline Link: http://www.oulu.fi/astronomy/S4G_PIPELINE4/MAIN/decomp0053.html#entry0003

Ошибок похоже нет

По поводу разрешения - Саша Мосенков пишет:

> Берется средняя 1.66" для всех галактик, отдельно для каждой ты не найдешь. 

In [ ]:
data = fits.getdata('NGC0628.phot.1_nonan.fits')
data = squeeze(data)
X,Y = np.meshgrid(np.arange(0.,len(data[0]),1), np.arange(0.,len(data),1))
Z = data

plt.figure(figsize=[12, 12])
quadmesh = plt.pcolormesh(X, Y, Z, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=0., vmax=1.)

plt.colorbar()

Слудеум Leroy (2008) c его постоянной M/L в $K$: $$\Sigma = 280\times\cos i\times I_{36}$$

Уже проверяли в другом документе по приближенной площадке, что похоже очень на его данные (надеюсь SIP distortion ничего не ломает, его отключили).

In [ ]:
%%time
cutout = plot_cutout('NGC0628.phot.1_nonan.fits', size=size, coordinates=True, figsize=[24, 8], title='star_density', cmap='sauron', vmax=10.)

star_grid_pixs = []
Sigma_s_cut = []
for scoord in sgrid.ravel():
    spix = skycoord_to_pixel(scoord, cutout.wcs)
    star_grid_pixs.append(spix)
    Sigma_s_cut.append(cutout.data[int(spix[1])][int(spix[0])]*280.*cos_i)
plt.plot(zip(*star_grid_pixs)[0],  zip(*star_grid_pixs)[1], '.', ms=1, color='r')
plt.show()

In [ ]:
len(star_grid_pixs)

Площадка еще раз

In [ ]:
# cutout = plot_cutout('NGC0628.phot.1_nonan.fits', size=size, coordinates=True, figsize=[24, 8], title='star_density', cmap='sauron', vmax=10.)
# plot_rectange_from_grid(plt.gca(), star_grid_pixs)
# plt.show()

In [ ]:
X,Y = np.meshgrid(np.arange(0.,len(data[0]),1), np.arange(0.,len(data),1))

fig = plt.figure(figsize=[12,12])

w = WCS('NGC0628.phot.1_nonan.fits', naxis=2)
ax = fig.add_subplot(111, projection=w)
im = ax.imshow(data, origin='lower', vmax=5.)

lon = ax.coords[0]
lat = ax.coords[1]

lon.set_major_formatter('dd:mm:ss')
lat.set_major_formatter('dd:mm:ss')

lon.set_ticks(spacing=60. * u.arcsec)
lat.set_ticks(spacing=60. * u.arcsec)

lon.display_minor_ticks(True)
lat.display_minor_ticks(True)
lat.set_minor_frequency(3)
lon.set_minor_frequency(3)

star_grid_pixs_ = []
for scoord in sgrid.ravel():
    spix = skycoord_to_pixel(scoord, w)
    star_grid_pixs_.append(spix)

plot_rectange_from_grid(plt.gca(), star_grid_pixs_)
plt.show()

In [ ]:
X,Y = np.meshgrid(np.arange(0.,len(data[0]),1), np.arange(0.,len(data),1))

fig = plt.figure(figsize=[24,24])

w = WCS('NGC0628.phot.1_nonan.fits', naxis=2)
ax = fig.add_subplot(111, projection=w)
im = ax.imshow(data, origin='lower', vmax=3., vmin=0.)

plt.xlim(350, 930)
plt.ylim(450, 950)

# lon = ax.coords[0]
# lat = ax.coords[1]

# lon.set_major_formatter('dd:mm:ss')
# lat.set_major_formatter('dd:mm:ss')

ax.set_xticklabels([])
ax.set_yticklabels([])

# lon.display_minor_ticks(False)
# lat.display_minor_ticks(False)
# lat.set_minor_frequency(3)
# lon.set_minor_frequency(3)

plot_rectange_from_grid(plt.gca(), star_grid_pixs_)
plt.show()

In [ ]:
Sigma_s_cut = np.array(Sigma_s_cut).reshape((55, 155))

In [ ]:
plt.figure(figsize=[18, 7])
X,Y = np.meshgrid(np.arange(0.,155,1), np.arange(0.,55,1))
Z = Sigma_s_cut
quadmesh = plt.pcolormesh(X, Y, Z, cmap='magma')
quadmesh.set_clim(vmin=1., vmax=1000.)
plt.colorbar()
quadmesh.cmap.set_under('white')

In [ ]:
np.save('Sigma_s_cut.npy', Sigma_s_cut)

# Все в одном

In [ ]:
Sigma_HI_ = np.load('Sigma_HI_cut.npy')
sig_los = np.load('sig_los.npy')
c_g_ = np.load('c_g_cut.npy')
c_CO_ = np.load('c_CO_.npy')
dist_ = np.load('dist.npy')
Sigma_HERACLES_CO_ = np.load('Sigma_CO_cut.npy')
# Sigma_HERACLES_CO_ = np.nan_to_num(Sigma_HERACLES_CO_)
log_Halpha_SFR_ = np.load('Halpha_SFR_.npy')
# Sigma_s = np.load('Sigma_s.npy')
sig_R_max = np.load('sig_R_max.npy')
sig_R_min = np.load('sig_R_min.npy')
Sigma_s_Leroy_ = np.load('Sigma_s_cut.npy')

X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))

r_g = np.load('r_g.npy')
vel_g = np.load('vel_g.npy')
gas_approx = poly1d(polyfit(r_g, vel_g, deg=5))
def epicyclicFreq_real(poly_gas, R, resolution):
    return sqrt(2.0) * poly_gas(R) * sqrt(1 + R * poly_gas.deriv()(R) / poly_gas(R)) / (R * resolution )

def add_SFR_contours(levels=[-2.], colors=('red', 'blue'), X=None, Y=None, ax=None, **kwargs):
    if X is None and Y is None:
        X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
    if ax is not None:
        CS = ax.contour(X, Y, log_Halpha_SFR_, levels=levels, colors=colors, **kwargs)
    else:
        CS = plt.contour(X, Y, log_Halpha_SFR_, levels=levels, colors=colors, **kwargs)

def mark_stars(colors=('white'), levels=[0.999, 1.001], ax=None):
    Xm,Ym = np.meshgrid(np.arange(0.5,Nx+0.5,1), np.arange(0.5,Ny+0.5,1))
    mask = sig_los == 0
    if ax is not None:
        CS3 = ax.contourf(Xm, Ym, mask, levels, colors=colors, interpolation='none')
    else:
        CS3 = plt.contourf(Xm, Ym, mask, levels, colors=colors, interpolation='none')
        
def add_spirals(ax, colors=['m', 'm'], **kwargs):
    
    tt = np.arange(0.5, 15.5, 0.001)
    delta = -40.*pi/180.
    theta = (tt-delta)
    r0 = 14.95 * (Nx/350.)
    b = 0.33

    yy = r0 * np.exp(tt*b) * np.sin(theta) + 55./2
    xx = r0 * np.exp(tt*b) * np.cos(theta) + 155./2

    delta2 = 140.*pi/180.
    theta2 = (tt-delta2)
    r02 = 20.95 * (Nx/350.)
    b2 = 0.27

    yy2 = r02 * np.exp(tt*b2) * np.sin(theta2) + 55./2
    xx2 = r02 * np.exp(tt*b2) * np.cos(theta2) + 168./2

    spiralA = zip(xx,yy)
    spiralB = zip(xx2,yy2)

    collection1 = LineCollection([spiralA, spiralB], **kwargs)
    ax.add_collection(collection1)

Общая картинка:

In [ ]:
import matplotlib.patches as patches

fig = plt.figure(figsize=[30, 96])
plt.subplot(8, 1, 1)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, Sigma_HI_, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=0., vmax=10.5)
plt.colorbar()
CS = plt.contour(X, Y, dist_, [50., 80., 100.], colors=('g'))
plt.title(r'$\Sigma_{HI}$', fontsize=30)


plt.subplot(8, 1, 2)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
# CS = plt.contour(X, Y, sig_los, levels=[40., 70.], aspect='auto', alpha=0.5, colors='r')
# plt.clabel(CS, inline=1, fontsize=10)
plt.title(r'$\sigma_{s}$', fontsize=30)
quadmesh2 = plt.pcolormesh(X, Y, sig_los, cmap=cm.jet)
quadmesh2.set_clim(vmin=6., vmax=80.5)
plt.colorbar()
CS = plt.contour(X, Y, dist_, [50., 80., 100.], colors=('g'))
mark_stars(colors=('y'), levels=[0.999, 1.001])


#bulge
circle = plt.Circle((512, 512), 10., color='b', fill=False)
plt.gca().add_artist(circle)


plt.subplot(8, 1, 3)
# X,Y = np.meshgrid(xvalues, yvalues)
quadmesh = plt.pcolormesh(X, Y, c_g_, cmap=cm.plasma)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=5., vmax=25.5)
plt.colorbar()
CS = plt.contour(X, Y, dist_, [50., 80., 100.], colors=('g'))
plt.title(r'$\sigma_{HI}$', fontsize=30)

plt.subplot(8, 1, 4)
X_,Y_ = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X_, Y_, c_CO_, cmap=cm.copper)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=.1, vmax=15.1)
plt.colorbar()
plt.title(r'$\sigma_{CO}$', fontsize=30)

plt.subplot(8, 1, 5)
# X,Y = np.meshgrid(xvalues, yvalues)
quadmesh = plt.pcolormesh(X, Y, dist_, cmap=cm.ocean)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
# quadmesh.set_clim(vmin=.1, vmax=15.1)
plt.colorbar()
CS = plt.contour(X, Y, dist_, [50., 80., 100.], colors=('g'))
plt.title(r'$R$', fontsize=30)

plt.subplot(8, 1, 6)
# X,Y = np.meshgrid(xvalues, yvalues)
quadmesh = plt.pcolormesh(X, Y, Sigma_HERACLES_CO_, cmap=cm.jet)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=0., vmax=45.19)
plt.colorbar()
CS = plt.contour(X, Y, dist_, [50., 80., 100.], colors=('g'))
plt.title(r'$\Sigma_{CO}$', fontsize=30)

plt.subplot(8, 1, 7)
# X,Y = np.meshgrid(xvalues, yvalues)
quadmesh = plt.pcolormesh(X, Y, log_Halpha_SFR_, cmap=cm.jet)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=-3.5, vmax=-1.)
plt.colorbar()
CS = plt.contour(X, Y, dist_, [50., 80., 100.], colors=('g'))
plt.title(r'$\log H_{\alpha}$', fontsize=30)
add_SFR_contours()

plt.subplot(8, 1, 8)
# X,Y = np.meshgrid(xvalues, yvalues)
quadmesh = plt.pcolormesh(X, Y, Sigma_s_Leroy_, cmap=cm.jet)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=1, vmax=1000.)
plt.colorbar()
CS = plt.contour(X, Y, dist_, [50., 80., 100.], colors=('g'))
plt.title(r'$\Sigma_{s}$', fontsize=30)

# X,Y = np.meshgrid(np.arange(0.,len(c_g),1), np.arange(0.,len(c_g[0]),1))
# CS = plt.contour(X[470:560, 380:650], Y[470:560, 380:650], c_g[470:560, 380:650], levels=[5., 10., 20.], aspect='auto', alpha=0.5)

fig.subplots_adjust(wspace=0.1, hspace=0.05)

plt.savefig('scaled_maps_column.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
print Sigma_HI_.shape
print sig_los.shape
print c_g_.shape
print c_CO_.shape
print dist_.shape
print Sigma_HERACLES_CO_.shape
print log_Halpha_SFR_.shape
print sig_R_max.shape
print sig_R_min.shape
print Sigma_s_Leroy_.shape

# Результаты

## $Q$

Было 1553x513≈800000 pix, стало 155x55=8525 т.е. в тысячу раз быстрее.

TODO: можно ли оценить эффект толщины?

TODO: cold phase

[A UNIVERSAL, LOCAL STAR FORMATION LAW IN GALACTIC CLOUDS, NEARBY GALAXIES, HIGH-REDSHIFT DISKS, AND STARBURSTS](http://iopscience.iop.org/article/10.1088/0004-637X/745/1/69/pdf) Krumholz 2012 - что две фазы и $Q$ важно в первой.

In [ ]:
%%time

G = 4.32
Qeff_max = np.zeros(dist_.shape)
Qeff_min = np.zeros(dist_.shape)
bark = np.zeros(dist_.shape)
bark_max = np.zeros(dist_.shape)
Qgs = np.zeros(dist_.shape)
Qss_min = np.zeros(dist_.shape)
Qss_max = np.zeros(dist_.shape)

step = 0
errors = 0
for i, row in enumerate(dist_):
    for j, col in enumerate(dist_[0]):
        step+=1
        if step % 1000 == 0:
            print step
        c = c_g_[i][j]
        Sigma = 1.36*(Sigma_HI_[i][j] + Sigma_HERACLES_CO_[i][j])
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        Sigma_s_ = Sigma_s_Leroy_[i][j]
        s_R_max = sig_R_max[i][j]
        s_R_min = sig_R_min[i][j]
        Qg = kappa * c / (math.pi * G * Sigma)
        Qs_max = kappa * s_R_max / (3.36 * G * Sigma_s_)
        Qs_min = kappa * s_R_min / (3.36 * G * Sigma_s_)
        
#         print c, Sigma, kappa, Sigma_s_, s_R_max, s_R_min, Qs_min, Qg, c/s_R_min
        
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_min, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
        except ValueError:
            errors+=1
            qeff = 0.01
            root_for_max = -1.
        Qeff_min[i][j] = 1./qeff
        bark[i][j] = root_for_max
        
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_max, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
        except ValueError:
            errors+=1
            qeff = 0.01
            root_for_max = -1.
        Qeff_max[i][j] = 1./qeff
        bark_max[i][j] = root_for_max
        
        Qgs[i][j] = Qg
        Qss_min[i][j] = Qs_min
        Qss_max[i][j] = Qs_max
        
print 'Errors:{}'.format(errors)

In [ ]:
np.save('Qeff_min.npy', Qeff_min)
np.save('bark.npy', bark)
np.save('Qeff_max.npy', Qeff_max)
np.save('bark_max.npy', bark)
np.save('Qgs.npy', Qgs)
np.save('Qss_min.npy', Qss_min)
np.save('Qss_max.npy', Qss_max)

In [ ]:
Qeff_min = np.load('Qeff_min.npy')
bark = np.load('bark.npy')
Qeff_max = np.load('Qeff_max.npy')
bark = np.load('bark_max.npy')
Qgs = np.load('Qgs.npy')
Qss_min = np.load('Qss_min.npy')
Qss_max = np.load('Qss_max.npy')

X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))

In [ ]:
# TODO: понять, откуда берутся контуры внутри звезд и как их убрать

plt.figure(figsize=[30, 54])

plt.subplot(6,1,1)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, Qeff_min, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
# CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,min}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((512, 512), 10., color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
# circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10) #7px approx 15arcsec
# plt.gca().add_artist(circle)
add_SFR_contours(colors=('red'))

circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='m', fill=False, lw=6, ls='--')
plt.gca().add_artist(circle2)

add_spirals(plt.gca(), colors=['m', 'm'])

plt.subplot(6,1,2)
quadmesh = plt.pcolormesh(X, Y, Qeff_max, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_max, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,max}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours(colors=('red'))

plt.subplot(6,1,3)
quadmesh = plt.pcolormesh(X, Y, Qgs, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'$Q_{g}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(colors=('red'))

plt.subplot(6,1,4)
quadmesh = plt.pcolormesh(X, Y, bark, cmap=cm.gray)
quadmesh.set_clim(vmin=0., vmax=2.)
plt.colorbar()
plt.title(r'$\bar{k}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])

plt.subplot(6,1,5)
quadmesh = plt.pcolormesh(X, Y, Qss_min, cmap=cm.hot)
quadmesh.set_clim(vmin=1.5, vmax=5.)
plt.colorbar()
plt.title(r'$Q_{s}^{min}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])

plt.subplot(6,1,6)
quadmesh = plt.pcolormesh(X, Y, Qss_max, cmap=cm.hot)
quadmesh.set_clim(vmin=2.5, vmax=8.)
plt.colorbar()
plt.title(r'$Q_{s}^{max}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])

# plt.savefig('result_maps_column2.png', format='png', bbox_inches='tight')
# plt.show()

### k в кпк

$\bar{k}$ в масштабах кпк 
${\displaystyle \bar{k}\equiv\frac{k\,\sigma_{\mathrm{s}}}{\kappa}}$:

In [ ]:
%%time
G = 4.32
lambda_kpc = np.zeros(dist_.shape)

for i, row in enumerate(dist_):
    if i%100 == 0:
        print i
    for j, col in enumerate(dist_[0]):
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        s_R_min = sig_R_min[i][j]
        lambda_kpc[i][j] = 2*math.pi*s_R_min/(bark[i][j]*kappa)

In [ ]:
np.save('lambda_kpc.npy', lambda_kpc)

In [ ]:
lambda_kpc = np.load('lambda_kpc.npy')

In [ ]:
plt.figure(figsize=[30, 9])
X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
quadmesh = plt.pcolormesh(X, Y, lambda_kpc, cmap=cm.gray)
quadmesh.set_clim(vmin=0., vmax=3.)
plt.colorbar()
CS = plt.contour(X, Y, lambda_kpc, levels=[0.75, 1.4], colors=('g', 'b'))
plt.title(r'$\lambda\:[kpc]$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)

plt.plot([10, 10+1./calc_scale(8.6) * (Nx/350.)], [45, 45], '-', lw=3, color='r')
plt.text(12, 46, '1 kpc', color='r', fontsize=20)

plt.plot([12, 12 + 1.66 * (Nx/350.)], [40, 40], '-', lw=3, color='r')
plt.text(12, 41, '1.66 arcsec S4G', color='r', fontsize=20)

plt.plot([12, 12 + 13.6 * (Nx/350.)], [35, 35], '-', lw=3, color='r')
plt.text(12, 36, '13.6 arcsec HERA', color='r', fontsize=20)

plt.plot([12, 12 + 6 * (Nx/350.)], [30, 30], '-', lw=3, color='r')
plt.text(12, 31, '6.0 arcsec THINGS', color='r', fontsize=20)

plt.plot([12, 12 + 5.6 * (Nx/350.)], [25, 25], '-', lw=3, color='r')
plt.text(12, 26, '5.6 arcsec VENGA', color='r', fontsize=20)

In [ ]:
# fig= plt.figure(figsize=[30,10])

# ax = plt.gca()
# q_level = 3.
# h_level = -2.15
# ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
# CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.3)
# # CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.3)
# #         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
# add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
# CS2 = ax.contour(X, Y, lambda_kpc, levels=[1.2], colors=('grey'), alpha=0.3)
# CS3 = ax.contourf(X, Y, lambda_kpc, levels=[0., 1.2], colors=('none'), hatches=['o'], alpha=0.)
# mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

# plt.show()

In [ ]:
# plt.figure(figsize=[30, 9])
# X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
# # quadmesh = plt.pcolormesh(X, Y, lambda_kpc, cmap=cm.gray)
# # quadmesh.set_clim(vmin=0., vmax=3.)
# # plt.colorbar()
# CS = plt.contourf(X, Y, lambda_kpc, levels=[0., 0.5, 0.75, 1., 1.5, 2.0, 2.5, 3.0], colors=(cm.rainbow(np.linspace(0,1,8))), alpha=0.4)
# plt.colorbar()
# plt.title(r'$\lambda\:[kpc]$', fontsize=30)
# mark_stars(colors=('y'), levels=[0.999, 1.001])
# add_SFR_contours(levels=[-2.15], colors=('red'), X=None, Y=None)
# plt.plot([10, 10+1./calc_scale(8.6) * (Nx/350.)], [45, 45], '-', lw=3, color='r')
# plt.text(12, 46, '1 kpc', color='r', fontsize=20)

In [ ]:
plt.figure(figsize=[30, 9])
X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
# quadmesh = plt.pcolormesh(X, Y, lambda_kpc, cmap=cm.gray)
# quadmesh.set_clim(vmin=0., vmax=3.)
# plt.colorbar()
CS = plt.contourf(X, Y, lambda_kpc, levels=[0., 1., 2.0, 3.0], colors=(cm.rainbow(np.linspace(0,1,8))), alpha=0.4)
plt.colorbar()
plt.title(r'$\lambda\:[kpc]$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(levels=[-2.15], colors=('red'), X=None, Y=None, linewidths=(2.))
plt.plot([10, 10+1./calc_scale(8.6) * (Nx/350.)], [45, 45], '-', lw=3, color='r')
plt.text(12, 46, '1 kpc', color='r', fontsize=20)

Картинка для статьи

In [ ]:
import matplotlib as mpl
mpl.style.use('classic')

In [ ]:
fig = plt.figure(figsize=[30, 9])

ax = plt.gca()

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))

# CS = plt.contourf(X, Y, lambda_kpc, levels=[0., 1., 2.0, 3.0], 
#             colors=(cm.rainbow(np.linspace(0,1,8))), alpha=0.4)
# plt.colorbar()
# plt.title(r'$\lambda\:[kpc]$', fontsize=30)
# mark_stars(colors=('y'), levels=[0.999, 1.001])
   
masked_array = np.ma.array(lambda_kpc, mask=sig_los == 0)
# imcmap = matplotlib.cm.gray_r
# imcmap.set_bad('y')
# quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=0., vmax=3., interpolation='none')
# quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
# quadmesh.set_clim(vmin=0.0, vmax=3.)
CS = plt.contourf(X, Y, masked_array, levels=[0., 1., 2.0, 3.0], 
                  colors=(cm.rainbow(np.linspace(0,1,8))), alpha=0.4)
# plt.colorbar()

cbar = plt.colorbar(CS, pad=0.01)
cbar.ax.tick_params(labelsize=20) 

add_SFR_contours(levels=[h_level], colors=('black'), X=None, Y=None, linewidths=(2.7))
ax.set_facecolor("white")
    
ax.set_xlim(0, Nx-1)
ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
ax.tick_params(axis='both', length=8, width=3)
ax.tick_params(axis='both', length=4, width=2, which='minor')
ax.xaxis.set_minor_locator(AutoMinorLocator(3))
ax.yaxis.set_minor_locator(AutoMinorLocator(2))

ax.text(4, 46.5, r'$\lambda,\,\mathrm{kpc}$', fontsize=60, color='black')
plt.plot([6, 6+1./calc_scale(8.6) * (Nx/350.)], [6, 6], '-', lw=4, color='black')

# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

plt.savefig(paper_imgs_dir3 +'lambda.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'lambda.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'lambda.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=[30, 9])

ax = plt.gca()

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
  
masked_array = np.ma.array(lambda_kpc, mask=sig_los == 0)
CS = plt.contourf(X, Y, masked_array, levels=[0., 1., 2.0, 3.0], 
                  colors=(cm.rainbow(np.linspace(0,1,5))), alpha=0.4)
cbar = plt.colorbar(CS, pad=0.01)
cbar.ax.tick_params(labelsize=20) 

add_SFR_contours(levels=[h_level], colors=('black'), X=None, Y=None, linewidths=(3.7))
ax.set_facecolor("white")
    
ax.set_xlim(0, Nx-1)
ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
ax.tick_params(axis='both', length=8, width=3)
ax.tick_params(axis='both', length=4, width=2, which='minor')
ax.xaxis.set_minor_locator(AutoMinorLocator(3))
ax.yaxis.set_minor_locator(AutoMinorLocator(2))

ax.text(4, 46.5, r'$\lambda,\,\mathrm{kpc}$', fontsize=60, color='black')
plt.plot([6, 6+1./calc_scale(8.6) * (Nx/350.)], [6, 6], '-', lw=4, color='black')

# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

plt.savefig(paper_imgs_dir3 +'lambda2.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'lambda2.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'lambda2.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

### Ошибки

Для $\mathrm{HI}$ ошибку буду считать равной 1 $M_{\odot}/kpc^2$, ибо это предел THINGS.

In [ ]:
rel_sl_err = np.load('rel_sl_err.npy')
gas_approx_u = gas_approx * 1.07
gas_approx_b = gas_approx * 0.93
Halpha_SFR_log_err = np.load('Halpha_SFR_err.npy')
Sigma_CO_cut_relerr = np.load('Sigma_CO_cut_relerr.npy')

log_Halpha_SFR_u = np.log10(np.power(10., log_Halpha_SFR_) + np.power(10., Halpha_SFR_log_err))
log_Halpha_SFR_b = np.log10(np.power(10., log_Halpha_SFR_) - np.power(10., Halpha_SFR_log_err))

In [ ]:
%%time

G = 4.32
Qeff_max_u = np.zeros(dist_.shape)
Qeff_min_u = np.zeros(dist_.shape)
Qgs_u = np.zeros(dist_.shape)

Qeff_max_b = np.zeros(dist_.shape)
Qeff_min_b = np.zeros(dist_.shape)
Qgs_b = np.zeros(dist_.shape)

step = 0
errors = 0
for i, row in enumerate(dist_):
    for j, col in enumerate(dist_[0]):
        step+=1
        if step % 1000 == 0:
            print step
        c = c_g_[i][j]
        Sigma = 1.36*(Sigma_HI_[i][j] + Sigma_HERACLES_CO_[i][j])
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        Sigma_s_ = Sigma_s_Leroy_[i][j]
        s_R_max = sig_R_max[i][j]
        s_R_min = sig_R_min[i][j]
        
        kappa_u = epicyclicFreq_real(gas_approx_u, dist_[i][j], scale)
        Sigma_u = 1.36*(Sigma_HI_[i][j] + 1. + Sigma_HERACLES_CO_[i][j] * (Sigma_CO_cut_relerr[i][j] + 1.))
        Qg_u = kappa_u * c / (math.pi * G * Sigma_u)
        Qs_max_u = kappa_u * s_R_max * (rel_sl_err[i][j] + 1.) / (3.36 * G * Sigma_s_)
        Qs_min_u = kappa_u * s_R_min * (rel_sl_err[i][j] + 1.) / (3.36 * G * Sigma_s_)
        
        kappa_b = epicyclicFreq_real(gas_approx_b, dist_[i][j], scale)
        Sigma_b = 1.36*(max(Sigma_HI_[i][j] - 1., 0.) + Sigma_HERACLES_CO_[i][j] * (-Sigma_CO_cut_relerr[i][j] + 1.))
        Qg_b = kappa_b * c / (math.pi * G * Sigma_b)
        Qs_max_b = kappa_b * s_R_max * (-rel_sl_err[i][j] + 1.) / (3.36 * G * Sigma_s_)
        Qs_min_b = kappa_b * s_R_min * (-rel_sl_err[i][j] + 1.) / (3.36 * G * Sigma_s_)
                
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_min_u, Qg_u, c/(s_R_min*(rel_sl_err[i][j] + 1.)), np.arange(0.01, 50., 1.))
            root_for_max, qeff2 = findInvKinemQeffBrentq(Qs_min_b, Qg_b, c/(s_R_min*(-rel_sl_err[i][j] + 1.)), np.arange(0.01, 50., 1.))
            Qeff_min_u[i][j] = 1./qeff
            Qeff_min_b[i][j] = 1./qeff2
        except ValueError:
            errors+=1
            qeff = 0.01
            root_for_max = -1.
        
        
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_max_u, Qg_u, c/(s_R_min*(rel_sl_err[i][j] + 1.)), np.arange(0.01, 50., 1.))
            root_for_max, qeff2 = findInvKinemQeffBrentq(Qs_max_b, Qg_b, c/(s_R_min*(-rel_sl_err[i][j] + 1.)), np.arange(0.01, 50., 1.))
            Qeff_max_u[i][j] = 1./qeff        
            Qeff_max_b[i][j] = 1./qeff2
        except ValueError:
            errors+=1
            qeff = 0.01
            root_for_max = -1.
        
        Qgs_u[i][j] = Qg_u
        Qgs_b[i][j] = Qg_b

        
print 'Errors:{}'.format(errors)

In [ ]:
X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
fig = plt.figure(figsize=[50, 27])

plt.subplot(3,2,1)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, Qeff_min_u, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_min_u, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,min}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((512, 512), 10., color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
# circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10) #7px approx 15arcsec
# plt.gca().add_artist(circle)
plt.contour(X, Y, log_Halpha_SFR_b, levels=[-2], colors=['r'])

plt.subplot(3,2,3)
quadmesh = plt.pcolormesh(X, Y, Qeff_max_u, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_max_u, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,max}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
plt.contour(X, Y, log_Halpha_SFR_b, levels=[-2], colors=['r'])

plt.subplot(3,2,5)
quadmesh = plt.pcolormesh(X, Y, Qgs_u, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qgs_u, levels=[2.5], colors=('g'))
plt.title(r'$Q_{g}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
plt.contour(X, Y, log_Halpha_SFR_b, levels=[-2], colors=['r'])

plt.subplot(3,2,2)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, Qeff_min_b, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_min_b, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,min}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((512, 512), 10., color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
# circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10) #7px approx 15arcsec
# plt.gca().add_artist(circle)
plt.contour(X, Y, log_Halpha_SFR_u, levels=[-2], colors=['r'])

plt.subplot(3,2,4)
quadmesh = plt.pcolormesh(X, Y, Qeff_max_b, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_max_b, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,max}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
plt.contour(X, Y, log_Halpha_SFR_u, levels=[-2], colors=['r'])

plt.subplot(3,2,6)
quadmesh = plt.pcolormesh(X, Y, Qgs_b, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qgs_b, levels=[2.5], colors=('g'))
plt.title(r'$Q_{g}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
plt.contour(X, Y, log_Halpha_SFR_u, levels=[-2], colors=['r'])

fig.subplots_adjust(wspace=0.0, hspace=0.0);

Не сильно меняется.

### SFR vs Q

In [ ]:
mask = (sig_los != 0.) & (dist_ > 15.)

# create data
x = log_Halpha_SFR_[mask].ravel()
y = np.log10(Qeff_min[mask].ravel())
cmm = (cm.rainbow(np.linspace(0,1,len(x))))

# Make the plot
plt.figure(figsize=[14, 12])
# plt.hexbin(x, y, gridsize=100, bins='log', cmap='plasma', vmin=0.2, vmax=1.5)
sc = plt.scatter(x, y, c=[l for l in dist_[mask].ravel()], cmap='plasma', vmin=15, vmax=120)
plt.ylim(-0.5, 1.)
plt.xlim(-3.5, -1.0)
plt.colorbar(sc)
plt.axhline(y=np.log10(3.), ls='--', color='g')
plt.axvline(x=-2.3, ls='--', color='g')
# kslaw = 1.48
# kslaw_bias = 5.
# plt.plot([-4., -2.], [-4.*kslaw+kslaw_bias, -2.*kslaw+kslaw_bias], '--', lw=20, color='g')
plt.xlabel('log(Sigma_SFR)')
plt.ylabel('log(Q_eff)')
plt.show()

### проверка параметров

In [ ]:
def examine_pixel(row, col):
    print 'pixels x={}, y={}'.format(col, row)
    print '''
    Sigma_HI_ = {:3.2f}
    sig_los = {:3.2f}
    c_g_ = {:3.2f}
    dist_ = {:3.2f}
    Sigma_HERACLES_CO_ = {:3.2f}
    log_Halpha_SFR_ = {:3.2f}
    sig_R_max = {:3.2f}
    sig_R_min = {:3.2f}
    Sigma_s_Leroy_ = {:3.2f}
    Qeff_min = {:3.2f}
    bark = {:3.2f}
    Qeff_max = {:3.2f}
    bark = {:3.2f}
    Qgs = {:3.2f}
    Qss_min = {:3.2f}
    Qss_max = {:3.2f}'''.format(
        Sigma_HI_[row][col],
        sig_los[row][col],
        c_g_[row][col],
        dist_[row][col],
        Sigma_HERACLES_CO_[row][col],
        log_Halpha_SFR_[row][col],
        sig_R_max[row][col],
        sig_R_min[row][col],
        Sigma_s_Leroy_[row][col],
        Qeff_min[row][col],
        bark[row][col],
        Qeff_max[row][col],
        bark[row][col],
        Qgs[row][col],
        Qss_min[row][col],
        Qss_max[row][col])

In [ ]:
examine_pixel(10, 16)

### азимутальное усреднение

In [ ]:
def azimuthal_average(Q, step=5, vmax=120, Qmax=100, vmin=0):
    az_av = []
    az_dev = []
    az_max = []
    az_min = []
    for dbin in range(vmin, vmax, step):
        mask = (dist_ >= dbin) & (dist_ <= dbin + step)
        anulli = Q[mask]
        mask = (anulli < Qmax) & (anulli > 0) #not count nans
        anulli = anulli[mask]
        az_av.append(anulli.sum()/mask.sum())
        az_dev.append(np.std(anulli))
        az_max.append(np.max(anulli) if len(anulli) != 0 else np.nan)
        az_min.append(np.min(anulli) if len(anulli) != 0 else np.nan)
    return az_av, az_dev, az_max, az_min

In [ ]:
for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    label = ['Qeff_min', 'Qeff_max', 'Qgs'][ind]
    color = ['r', 'g', 'm'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=5, vmax=120)
    plt.plot(np.array(range(5, 120, 5))+2.5, az_av, ':', label=label, color=color)
    plt.fill_between(np.array(range(5, 120, 5))+2.5, 
                     np.array(az_av)-np.array(az_dev), 
                     np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)
#     plt.fill_between(np.array(range(5, 120, 5))+2.5, 
#                      np.array(az_min), 
#                      np.array(az_max), alpha=0.1, color=color)

plt.grid()
plt.legend()
plt.axvline(x=55., alpha=0.3)
plt.ylim(0, 12.5);

In [ ]:
Image('Leroy2008_results.png')

In [ ]:
# plt.figure(figsize=[30, 54])

# plt.subplot(6,1,1)
# # X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
# quadmesh = plt.pcolormesh(X, Y, Qeff_min, cmap=cm.gist_yarg)
# # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
# quadmesh.set_clim(vmin=2.0, vmax=5.)
# plt.colorbar()
# CS = plt.contour(X, Y, dist_, [40., 45., 55, 60, 65, 90], colors=('g'))
# plt.title(r'$Q_{eff,min}$', fontsize=30)
# # add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
# mark_stars(colors=('y'), levels=[0.999, 1.001])
# #bulge
# # circle = plt.Circle((512, 512), 10., color='b', fill=False, lw=10)
# # plt.gca().add_artist(circle)
# # circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10) #7px approx 15arcsec
# # plt.gca().add_artist(circle)
# # add_SFR_contours(colors=('red'))

# # circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='m', fill=False, lw=6, ls='--')
# # plt.gca().add_artist(circle2)

# # add_spirals(plt.gca(), colors=['m', 'm'])

# # plt.savefig('result_maps_column2.png', format='png', bbox_inches='tight')
# # plt.show()

усредненные данные

In [ ]:
fig = plt.figure(figsize=[16, 7])

ax = plt.subplot(121)

for ind, Q in enumerate([Sigma_HI_, Sigma_HERACLES_CO_, Sigma_s_Leroy_]):
    label = ['Sigma_HI_', 'Sigma_HERACLES_CO_', 'Sigma_s_Leroy_'][ind]
    color = ['r', 'g', 'm'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=5, vmax=120, Qmax=1000)
    plt.plot(np.array(range(0, 120, 5))+2.5, az_av, ':', label=label, color=color)
    plt.fill_between(np.array(range(0, 120, 5))+2.5, 
                     np.array(az_av)-np.array(az_dev), 
                     np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)
#     plt.fill_between(np.array(range(5, 120, 5))+2.5, 
#                      np.array(az_min), 
#                      np.array(az_max), alpha=0.1, color=color)

plt.grid()
plt.legend()
plt.axvline(x=55., alpha=0.3)
ax.set_yscale("log", nonposy='clip')
plt.ylim(1, 1000)

ax = plt.subplot(122)

for ind, Q in enumerate([c_g_, sig_R_max, sig_R_min]):
    label = ['c_g_', 'sig_R_max', 'sig_R_min'][ind]
    color = ['r', 'g', 'm'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=5, vmax=120, Qmax=100000)
    plt.plot(np.array(range(0, 120, 5))+2.5, az_av, ':', label=label, color=color)
    plt.fill_between(np.array(range(0, 120, 5))+2.5, 
                     np.array(az_av)-np.array(az_dev), 
                     np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)

plt.grid()
plt.legend()
plt.axvline(x=55., alpha=0.3)
ax.set_yscale("log", nonposy='clip')
plt.ylim(0)

In [ ]:
import matplotlib as mpl
mpl.style.use('classic')

In [ ]:
fig = plt.figure(figsize=[20, 7])

ax = plt.subplot(121)
ax1 = ax

for ind, Q in enumerate([Sigma_HI_, Sigma_HERACLES_CO_, Sigma_s_Leroy_]):
    label = [r'$\mathrm{HI}$', r'$\mathrm{H_2}$', r'$\mathrm{star}$'][ind]
    color = ['r', 'g', 'm'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=5, vmax=145, Qmax=1000)
    plt.plot(np.array(range(0, 145, 5))+2.5, az_av, '--', label=label, color=color,lw=4)
    if label != r'$\mathrm{star}$':
        plt.fill_between(np.array(range(0, 145, 5))+2.5, 
                         np.array(az_av)-np.array(az_dev), 
                         np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)

az_av, az_dev, az_max, az_min = azimuthal_average(Sigma_s_Leroy_, step=5, vmax=55, Qmax=2000, vmin=0)
plt.fill_between(np.array(range(0, 55, 5))+2.5, 
                         np.array(az_av)-np.array(az_dev), 
                         np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)
az_av, az_dev, az_max, az_min = azimuthal_average(Sigma_s_Leroy_, step=5, vmax=145, Qmax=500, vmin=50)
plt.fill_between(np.array(range(50, 145, 5))+2.5, 
                         np.array(az_av)-np.array(az_dev), 
                         np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)

plt.grid()
plt.legend(fontsize=25, loc=(0.14, 0.87), ncol=3)
plt.axvline(x=55., alpha=0.3, color='grey', lw=4)
ax.set_yscale("log", nonposy='clip')
plt.ylim(1, 1000)
plt.xlim(0, 140)

az_av, az_dev, az_max, az_min = azimuthal_average(Sigma_HERACLES_CO_grad, step=5, vmax=145, Qmax=1000)
plt.plot(np.array(range(0, 145, 5))+2.5, az_av, '-', label=label, color='g',lw=2)


ax = plt.subplot(122)
ax2 = ax

for ind, Q in enumerate([c_g_, sig_R_max, sig_R_min]):
    label = [r'$c_\mathrm{g}$', r'$\sigma_R^\mathrm{max}$', r'$\sigma_R^\mathrm{min}$'][ind]
    color = ['r', 'g', 'm'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=5, vmax=145, Qmax=100000)
    plt.plot(np.array(range(0, 145, 5))+2.5, az_av, '--', label=label, color=color, lw=4)
    plt.fill_between(np.array(range(0, 145, 5))+2.5, 
                     np.array(az_av)-np.array(az_dev), 
                     np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)

plt.grid()
plt.legend(fontsize=27, loc="upper left", bbox_to_anchor=[0, 1], ncol=3)
plt.axvline(x=55., alpha=0.3, color='grey', lw=4)
ax.set_yscale("log", nonposy='clip')
plt.ylim(8)
plt.xlim(0, 140)

plt.axhline(y=11.)
sigmaR_leroy = lambda l: sqrt(2*pi*4.32*2.3*l/7.3)/0.6
plt.plot(np.array(range(0, 140, 5))+2.5,
         [sigmaR_leroy(_) for _ in azimuthal_average(Sigma_s_Leroy_, step=5, vmax=140, Qmax=1000)[0]], 'o-', ms=9)

ax1.set_xlabel(r'$R,\,{\mathrm{arcsec}}$', fontsize=30, rotation=0)
ax2.set_xlabel(r'$R,\,{\mathrm{arcsec}}$', fontsize=30, rotation=0)

ax1.set_ylabel(r'$\Sigma, \mathrm{M_{\odot}/{pc}^2}$', fontsize=35)
ax2.set_ylabel(r'$\sigma,\,\mathrm{km/s}$', fontsize=35)

# ax1.set_yticklabels(ax1.yaxis.get_major_ticks(), fontsize=20)
zed = [tick.label.set_fontsize(25) for tick in ax1.yaxis.get_major_ticks()]
ax1.tick_params(direction='in', pad=7)
zed = [tick.label.set_fontsize(25) for tick in ax2.yaxis.get_major_ticks()]
ax2.tick_params(direction='in', pad=7)

ax1.set_xticklabels(np.arange(0, 160, 20), fontsize=25)
ax2.set_xticklabels(np.arange(0, 160, 20), fontsize=25)

# ax1.set_title(r'$\mathrm{accuracy\,\,and\,\,recall}$', fontsize=25)
# ax2.set_title(r'$F_1(\mathrm{Q_{eff}^{min}}, \log \Sigma_{\mathrm{SFR}})$', fontsize=25)
# ax3.set_title(r'$F_1$', fontsize=25)

# plt.savefig(paper_imgs_dir3 +'data.eps', format='eps', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'data.png', format='png', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'data.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# plt.figure(figsize=[30, 54])

# plt.subplot(6,1,1)
# # X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
# quadmesh = plt.pcolormesh(X, Y, Sigma_s_Leroy_, cmap=cm.gist_yarg)
# # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
# quadmesh.set_clim(vmin=2.0, vmax=300.)
# plt.colorbar()
# CS = plt.contour(X, Y, dist_, [80., 100.], colors=('g'))
# # plt.title(r'$Q_{eff,min}$', fontsize=30)
# # add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
# mark_stars(colors=('y'), levels=[0.999, 1.001])
# #bulge
# # circle = plt.Circle((512, 512), 10., color='b', fill=False, lw=10)
# # plt.gca().add_artist(circle)
# # circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10) #7px approx 15arcsec
# # plt.gca().add_artist(circle)
# # add_SFR_contours(colors=('red'))

# # circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='m', fill=False, lw=6, ls='--')
# # plt.gca().add_artist(circle2)

# # add_spirals(plt.gca(), colors=['m', 'm'])

# # plt.savefig('result_maps_column2.png', format='png', bbox_inches='tight')
# # plt.show()

In [ ]:
%%time

G = 4.32
Qeff_maxp = np.zeros(23)
Qeff_minp = np.zeros(23)
Qgsp = np.zeros(23)

Sigma_HI_p = azimuthal_average(Sigma_HI_, step=5, vmax=120, Qmax=100000)[0]
Sigma_HERACLES_CO_p = azimuthal_average(Sigma_HERACLES_CO_, step=5, vmax=120, Qmax=100000)[0]
Sigma_s_Leroy_p = azimuthal_average(Sigma_s_Leroy_, step=5, vmax=120, Qmax=100000)[0]

c_g_p = azimuthal_average(c_g_, step=5, vmax=120, Qmax=100000)[0]
sig_R_maxp = azimuthal_average(sig_R_max, step=5, vmax=120, Qmax=100000)[0]
sig_R_minp = azimuthal_average(sig_R_min, step=5, vmax=120, Qmax=100000)[0]

errors = 0
for i, ddd in enumerate(range(5, 120, 5)):
    c = c_g_p[i]
    Sigma = 1.36*(Sigma_HI_p[i] + Sigma_HERACLES_CO_p[i])
    kappa = epicyclicFreq_real(gas_approx, ddd, scale)
    Sigma_s_ = Sigma_s_Leroy_p[i]
    s_R_max = sig_R_maxp[i]
    s_R_min = sig_R_minp[i]
    Qg = kappa * c / (math.pi * G * Sigma)
    Qs_max = kappa * s_R_max / (3.36 * G * Sigma_s_)
    Qs_min = kappa * s_R_min / (3.36 * G * Sigma_s_)

    try:
        root_for_max, qeff = findInvKinemQeffBrentq(Qs_min, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
    except ValueError:
        errors+=1
        qeff = 0.01
        root_for_max = -1.
    Qeff_minp[i] = 1./qeff

    try:
        root_for_max, qeff = findInvKinemQeffBrentq(Qs_max, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
    except ValueError:
        errors+=1
        qeff = 0.01
        root_for_max = -1.
    Qeff_maxp[i] = 1./qeff
    
    Qgsp[i] = Qg
        
print 'Errors:{}'.format(errors)

In [ ]:
leroy2008_Qg = [5.09433962264151, 15.576454802025873,
15.283018867924527, 13.611036963126917,
24.339622641509433, 12.846552938231323,
35.09433962264151, 12.12500728944212,
44.15094339622642, 11.011361082737285,
53.77358490566037, 8.738212344285598,
65.66037735849056, 7.34697879896336,
74.71698113207547, 6.672180435969322,
86.60377358490565, 6.419946083430621,
95.09433962264151, 6.297427552090032,
105.84905660377358, 6.059360315074425,
115.47169811320755, 5.609885211588141,
125.09433962264153, 5.397809747241713,
134.7169811320755, 5.830292944888797,
145.47169811320757, 5.943723218279731,
154.52830188679246, 5.830292944888797,
165.28301886792454, 5.397809747241713]

leroy2008_Q2f = [4.528301886792454, 4.902036820592469,
15.283018867924527, 4.997407564736587,
24.905660377358487, 4.997407564736587,
35.09433962264151, 4.451799177013175,
44.716981132075475, 3.9657593773822617,
54.33962264150943, 3.601516059263063,
65.66037735849056, 3.14708120871953,
74.71698113207547, 2.8580310690685367,
85.47169811320755, 2.803488279369447,
94.52830188679245, 2.5955293333801652,
105.28301886792454, 2.4497475889846094,
114.9056603773585, 2.3121538919078386,
124.52830188679245, 2.224745488389373,
135.28301886792454, 2.224745488389373,
144.9056603773585, 2.224745488389373,
155.0943396226415, 2.224745488389373,
164.7169811320755, 2.1822883483601814]

In [ ]:
fig = plt.figure(figsize=[12, 6])

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    label = ['Qeff_min', 'Qeff_max', 'Qgs'][ind]
    color = ['r', 'b', 'm'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=5, vmax=120)
    plt.plot(np.array(range(5, 120, 5))+2.5, az_av, ':', label=label, color=color)
    
    Qp = [Qeff_minp, Qeff_maxp, Qgsp][ind]
    plt.plot(np.array(range(5, 120, 5))+2.5, Qp, '-', color=color)
#     plt.fill_between(np.array(range(5, 120, 5))+2.5, 
#                      np.array(az_av)-np.array(az_dev), 
#                      np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)

plt.plot(leroy2008_Qg[::2], leroy2008_Qg[1::2], 'o-', label='Qg_leroy', color='black')
plt.plot(leroy2008_Q2f[::2], leroy2008_Q2f[1::2], 'o-', label='Q2f_leroy', color='g')

# plt.plot(rrrr, Qgsl, '--', color='black', label='Qg_leroy(recr)')
# plt.plot(rrrr, Qeff_minl, '--', color='g', label='Q2f_leroy(recr)')

plt.grid()
plt.legend(fontsize=18)
plt.axvline(x=55., alpha=0.3)
plt.ylim(0, 12.5)
plt.xlim(0, 150)

# plt.plot(np.array(range(5, 120, 5))+2.5, Qgsp, '-', label='Qgsp', color='m')
# plt.plot(np.array(range(5, 120, 5))+2.5, Qeff_minp, '-', label='Qeff_minp', color='r')
# plt.plot(np.array(range(5, 120, 5))+2.5, Qeff_maxp, '-', label='Qeff_maxp', color='g')

plt.xlabel(r'$R,arcsec$', fontsize=18);

In [ ]:
fig = plt.figure(figsize=[12, 8])

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    label = [r'$\mathrm{Q_{eff}^{min}}$', r'$\mathrm{Q_{eff}^{max}}$', r'$\mathrm{Q_{g}}$'][ind]
    color = ['r', 'g', 'b'][ind]
    ls = ['--',':','-.'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=10, vmax=145)
    plt.plot(np.array(range(0, 145, 10))+5., az_av, label=label, color=color, lw=5,ls=ls)
    plt.fill_between(np.array(range(0, 145, 10))+5., 
                     np.array(az_av)-np.array(az_dev), 
                     np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)
#     plt.fill_between(np.array(range(5, 120, 5))+2.5, 
#                      np.array(az_min), 
#                      np.array(az_max), alpha=0.1, color=color)

plt.grid()
plt.legend(fontsize=30)
# plt.axvline(x=55., alpha=0.3)
plt.ylim(0, 12.5)
plt.xlim(10, 140.)

plt.plot(leroy2008_Qg[::2], leroy2008_Qg[1::2], 'o-', label='Qg_leroy', color='black', lw=4, ms=10)
plt.plot(leroy2008_Q2f[::2], leroy2008_Q2f[1::2], 's-', label='Q2f_leroy', color='black', lw=4, ms=10)


ax1 = plt.gca()

ax1.set_xlabel(r'$R,\,{\mathrm{arcsec}}$', fontsize=30, rotation=0)
ax1.set_ylabel(r'$Q$', fontsize=40, rotation=0)
# ax1.set_yticklabels(ax1.yaxis.get_major_ticks(), fontsize=20)
zed = [tick.label.set_fontsize(25) for tick in ax1.yaxis.get_major_ticks()]
ax1.tick_params(direction='in', pad=7)
ax1.set_xticklabels(np.arange(0, 160, 20), fontsize=25)

plt.plot([55., 55.], [0., 1.75], alpha=0.3, color='grey', lw=4)

ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(AutoMinorLocator(2))

# plt.savefig(paper_imgs_dir3 +'qaz.eps', format='eps', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'qaz.png', format='png', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'qaz.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# -------- ---- ----- ----- ------ -------
#                     Sig                 
#                     maHI  SigmaH Sig    
#          r (        (Msun 2 (Msu ma* (Ms
# Name     kpc) r.n   /pc2) n/pc2) un/pc2)
# -------- ---- ----- ----- ------ -------
leroy_dens_table_raw  = '''NGC  628  0.2  0.02  1.60  22.70  1209.4
NGC  628  0.5  0.05  2.10  20.20   557.8
NGC  628  0.9  0.08  2.60  16.10   313.6
NGC  628  1.2  0.12  3.10  12.70   231.9
NGC  628  1.6  0.15  3.70  11.40   194.3
NGC  628  1.9  0.19  4.60  11.10   163.5
NGC  628  2.3  0.22  5.30  11.10   143.9
NGC  628  2.7  0.25  5.80  10.60   123.5
NGC  628  3.0  0.29  6.10   8.90   107.5
NGC  628  3.4  0.32  6.50   7.20   151.0
NGC  628  3.7  0.36  7.30   6.20    81.6
NGC  628  4.1  0.39  7.90   5.90    68.0
NGC  628  4.4  0.42  8.10   5.40    61.6
NGC  628  4.8  0.46  7.90   4.30    48.3
NGC  628  5.1  0.49  8.20   3.10    41.8
NGC  628  5.5  0.53  8.50   2.10    37.0
NGC  628  5.8  0.56  8.60   1.20    33.2
NGC  628  6.2  0.59  8.60   0.00    37.0
NGC  628  6.5  0.63  8.80   0.00    52.9
NGC  628  6.9  0.66  8.80   0.00    19.5
NGC  628  7.3  0.69  8.60   0.00    18.9
NGC  628  7.6  0.73  8.20   0.00    18.7
NGC  628  8.0  0.76  7.60   0.00    12.9
NGC  628  8.3  0.80  7.10   0.00    17.6
NGC  628  8.7  0.83  6.70   0.00    17.0
NGC  628  9.0  0.86  6.50   0.00    10.8
NGC  628  9.4  0.90  6.00   0.00    8.0
NGC  628  9.7  0.93  5.20   0.00    7.5
NGC  628 10.1  0.97  4.50   0.00    5.0
NGC  628 10.4  1.00  4.10   0.00    4.1
NGC  628 10.8  1.03  3.90   0.00    3.6
NGC  628 11.1  1.07  3.90   0.00    3.9
NGC  628 11.5  1.10  4.00   0.00    4.4
NGC  628 11.9  1.13  4.30   0.00    9.5
NGC  628 12.2  1.17  4.60   0.00    5.8'''

In [ ]:
lspl = [' '.join(l.split()).split(' ') for l in leroy_dens_table_raw.split('\n')]
lspl[0]

In [ ]:
plt.plot([float(l[2])/calc_scale(7.3) for l in lspl], [float(l[5]) for l in lspl], 'o-', color='m',  label='H2')
plt.plot([float(l[2])/calc_scale(7.3) for l in lspl], [float(l[6]) for l in lspl], 'o-', color='r',  label='*')
plt.plot([float(l[2])/calc_scale(7.3) for l in lspl], [float(l[4]) for l in lspl], 'o-', color='b',  label='HI')
plt.legend()
ax = plt.gca()
ax.set_yscale("log", nonposy='clip')
plt.grid()

In [ ]:
fig = plt.figure(figsize=[16, 7])

ax = plt.subplot(111)

for ind, Q in enumerate([Sigma_HI_, Sigma_HERACLES_CO_, Sigma_s_Leroy_]):
    label = ['Sigma_HI_', 'Sigma_HERACLES_CO_', 'Sigma_s_Leroy_'][ind]
    color = ['r', 'g', 'm'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=5, vmax=140, Qmax=100000)
    plt.plot(np.array(range(0, 140, 5))+2.5, az_av, ':', label=label, color=color)
    plt.fill_between(np.array(range(0, 140, 5))+2.5, 
                     np.array(az_av)-np.array(az_dev), 
                     np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)
#     plt.fill_between(np.array(range(5, 120, 5))+2.5, 
#                      np.array(az_min), 
#                      np.array(az_max), alpha=0.1, color=color)

plt.plot([float(l[2])/calc_scale(7.3) for l in lspl], [float(l[5]) for l in lspl], 'o-', color='g',  label='H2')
plt.plot([float(l[2])/calc_scale(7.3) for l in lspl], [float(l[6]) for l in lspl], 'o-', color='m',  label='*')
plt.plot([float(l[2])/calc_scale(7.3) for l in lspl], [float(l[4]) for l in lspl], 'o-', color='r',  label='HI')

# plt.plot([float(l[2])/calc_scale(7.3) for l in lspl], [float(l[4])/1.34 for l in lspl], 's-', color='r',  label='HI')


plt.grid()
plt.legend()
plt.axvline(x=55., alpha=0.3)
ax.set_yscale("log", nonposy='clip')
plt.ylim(1, 1000)
plt.xlim(0, 140)

ax1 = plt.gca()

ax1.set_xlabel(r'$R,\,{\mathrm{arcsec}}$', fontsize=30, rotation=0)

ax1.set_ylabel(r'$\Sigma, \mathrm{M_{\odot}/{pc}^2}$', fontsize=35)

# ax1.set_yticklabels(ax1.yaxis.get_major_ticks(), fontsize=20)
zed = [tick.label.set_fontsize(25) for tick in ax1.yaxis.get_major_ticks()]
ax1.tick_params(direction='in', pad=7)

ax1.set_xticklabels(np.arange(0, 140, 20), fontsize=25)

In [ ]:
vflat = 217
lflat = 0.8 #kpc
ls = 2.3 #kpc

vrot = lambda l: vflat*(1 - np.exp(-l/lflat))
vrotder = lambda l: vflat*(np.exp(-l/lflat)/lflat)

test_points  = np.linspace(0, 150., 100)

def epic(poly_gas, R, resolution):
    '''Честное вычисление эпициклической частоты на расстоянии R для сплайна или полинома'''
    return sqrt(2.0) * vrot(R) * sqrt(1 + R * vrotder(R) / poly_gas(R)) / (R * resolution )


fig = plt.figure(figsize=[18, 8])
plt.plot(test_points, [epic(vrot, x, scale) for x in test_points], '-', label='gas_approx_Leroy')
plt.plot(test_points, [epicyclicFreq_real(gas_approx, x, scale) for x in test_points], '-', label='gas_approx_our')

# plt.plot(r_g_dens[1:], ef, '--')
plt.xlabel('$R, arcsec$')
plt.ylabel('$\kappa,\, km/s/kpc$', fontsize=15)
plt.xlim(0)
plt.axvline(x=15.) #bulge
plt.ylim(0, 1200.)
plt.legend()

In [ ]:
sigmaR_leroy = lambda l: sqrt(2*pi*4.32*2.3*l/7.3)/0.6

In [ ]:
%%time

rrrr = [float(l[2])/calc_scale(7.3) for l in lspl]

G = 4.32
Qeff_maxl = np.zeros(len(rrrr))
Qeff_minl = np.zeros(len(rrrr))
Qgsl = np.zeros(len(rrrr))

Sigma_HI_l = [float(l[4]) for l in lspl]
Sigma_HERACLES_CO_l = [float(l[5]) for l in lspl]
Sigma_s_Leroy_l = [float(l[6]) for l in lspl]

c_g_l = [11.]*len(rrrr)
sig_R_maxl = [sigmaR_leroy(l) for l in Sigma_s_Leroy_l]
sig_R_minl = [sigmaR_leroy(l) for l in Sigma_s_Leroy_l]

errors = 0
for i, ddd in enumerate(rrrr):
    c = c_g_l[i]
    Sigma = (Sigma_HI_l[i] + Sigma_HERACLES_CO_l[i])
    kappa = epic(vrot, ddd, scale)
    Sigma_s_ = Sigma_s_Leroy_l[i]
    s_R_max = sig_R_maxl[i]
    s_R_min = sig_R_minl[i]
    Qg = kappa * c / (math.pi * G * Sigma)
    Qs_max = kappa * s_R_max / (3.36 * G * Sigma_s_)
    Qs_min = kappa * s_R_min / (3.36 * G * Sigma_s_)

    try:
        root_for_max, qeff = findInvKinemQeffBrentq(Qs_min, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
    except ValueError:
        errors+=1
        qeff = 0.01
        root_for_max = -1.
    Qeff_minl[i] = 1./qeff

    try:
        root_for_max, qeff = findInvKinemQeffBrentq(Qs_max, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
    except ValueError:
        errors+=1
        qeff = 0.01
        root_for_max = -1.
    Qeff_maxl[i] = 1./qeff
    
    Qgsl[i] = Qg
        
print 'Errors:{}'.format(errors)

In [ ]:
fig = plt.figure(figsize=[12, 6])

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    label = ['Qeff_min', 'Qeff_max', 'Qgs'][ind]
    color = ['r', 'b', 'm'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=5, vmax=120)
#     plt.plot(np.array(range(5, 120, 5))+2.5, az_av, ':', label=label, color=color)
    
    Qp = [Qeff_minl, Qeff_maxl, Qgsl][ind]
#     plt.plot(rrrr, Qp, '--', color=color)
#     plt.fill_between(np.array(range(5, 120, 5))+2.5, 
#                      np.array(az_av)-np.array(az_dev), 
#                      np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)

plt.plot(leroy2008_Qg[::2], leroy2008_Qg[1::2], 'o-', label='Qg_leroy', color='black')
plt.plot(leroy2008_Q2f[::2], leroy2008_Q2f[1::2], 'o-', label='Q2f_leroy', color='g')

plt.plot(rrrr, Qgsl, '--', color='black', label='Qg_leroy(recr)')
plt.plot(rrrr, Qeff_minl, '--', color='g', label='Q2f_leroy(recr)')

plt.grid()
plt.legend(fontsize=18)
plt.axvline(x=55., alpha=0.3)
plt.ylim(0.1, 100.)
plt.xlim(0)

ax = plt.gca()
ax.set_yscale("log", nonposy='clip')

plt.axhline(y=2., color='grey', alpha=0.15, lw=5)
plt.axhline(y=3., color='grey', alpha=0.15, lw=5)

# plt.plot(np.array(range(5, 120, 5))+2.5, Qgsp, '-', label='Qgsp', color='m')
# plt.plot(np.array(range(5, 120, 5))+2.5, Qeff_minp, '-', label='Qeff_minp', color='r')
# plt.plot(np.array(range(5, 120, 5))+2.5, Qeff_maxp, '-', label='Qeff_maxp', color='g')

plt.xlabel(r'$R,arcsec$', fontsize=18);

А теперь как у Лероя (E1):

$$\sigma = \frac{\sigma_{rms}}{\sqrt{N_{pix,ring}/N_{pix, beam}}}$$,

при этом если брать beam = 13.6 то np.sqrt(np.pi*(13.6/2.11)^2)= 11.5, а если beam = 5.6, то 4.7

In [ ]:
def azimuthal_average(Q, step=5, vmax=120, Qmax=100, vmin=0, beam_radius=13.6/2):
    az_av = []
    az_dev = []
    az_max = []
    az_min = []
    for dbin in range(vmin, vmax, step):
        mask = (dist_ >= dbin) & (dist_ <= dbin + step)
        anulli = Q[mask]
        mask = (anulli < Qmax) & (anulli > 0) #not count nans
        anulli = anulli[mask]
        az_av.append(np.mean(anulli))
        beam_pix = np.pi*(beam_radius/2.11)**2
        az_dev.append(np.std(anulli)/np.sqrt(mask.sum()/beam_pix)) #see (E1) in Leroy et al. (2008)
#         az_max.append(np.max(anulli) if len(anulli) != 0 else np.nan)
        az_max.append(mask.sum())
        az_min.append(np.min(anulli) if len(anulli) != 0 else np.nan)
    return az_av, az_dev, az_max, az_min

In [ ]:
fig = plt.figure(figsize=[12, 8])

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    label = [r'$\mathrm{Q_{eff}^{min}}$', r'$\mathrm{Q_{eff}^{max}}$', r'$\mathrm{Q_{g}}$'][ind]
    color = ['r', 'g', 'b'][ind]
    ls = ['--',':','-.'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=10, vmax=145, beam_radius=13.6/2)
    plt.plot(np.array(range(0, 145, 10))+5., az_av, label=label, color=color, lw=5,ls=ls)
    plt.fill_between(np.array(range(0, 145, 10))+5., 
                     np.array(az_av)-np.array(az_dev), 
                     np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)
#     plt.fill_between(np.array(range(5, 120, 5))+2.5, 
#                      np.array(az_min), 
#                      np.array(az_max), alpha=0.1, color=color)

plt.grid()
plt.legend(fontsize=30)
# plt.axvline(x=55., alpha=0.3)
plt.ylim(0, 12.5)
plt.xlim(10, 140.)

plt.plot(leroy2008_Qg[::2], leroy2008_Qg[1::2], 'o-', label='Qg_leroy', color='black', lw=4, ms=10)
plt.plot(leroy2008_Q2f[::2], leroy2008_Q2f[1::2], 's-', label='Q2f_leroy', color='black', lw=4, ms=10)


ax1 = plt.gca()

ax1.set_xlabel(r'$R,\,{\mathrm{arcsec}}$', fontsize=30, rotation=0)
ax1.set_ylabel(r'$Q$', fontsize=40, rotation=0)
# ax1.set_yticklabels(ax1.yaxis.get_major_ticks(), fontsize=20)
zed = [tick.label.set_fontsize(25) for tick in ax1.yaxis.get_major_ticks()]
ax1.tick_params(direction='in', pad=7)
ax1.set_xticklabels(np.arange(0, 160, 20), fontsize=25)

plt.plot([55., 55.], [0., 1.75], alpha=0.3, color='grey', lw=4)

ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(AutoMinorLocator(2))

plt.savefig(paper_imgs_dir3 +'qaz2.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'qaz2.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'qaz2.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=[12, 8])

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    label = [r'$\mathrm{Q_{eff}^{min}}$', r'$\mathrm{Q_{eff}^{max}}$', r'$\mathrm{Q_{g}}$'][ind]
    color = ['r', 'g', 'b'][ind]
    ls = ['--',':','-.'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=10, vmax=145, beam_radius=5.6/2)
    plt.plot(np.array(range(0, 145, 10))+5., az_av, label=label, color=color, lw=5,ls=ls)
    plt.fill_between(np.array(range(0, 145, 10))+5., 
                     np.array(az_av)-np.array(az_dev), 
                     np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)
#     plt.fill_between(np.array(range(5, 120, 5))+2.5, 
#                      np.array(az_min), 
#                      np.array(az_max), alpha=0.1, color=color)

plt.grid()
plt.legend(fontsize=30)
# plt.axvline(x=55., alpha=0.3)
plt.ylim(0, 12.5)
plt.xlim(10, 140.)

plt.plot(leroy2008_Qg[::2], leroy2008_Qg[1::2], 'o-', label='Qg_leroy', color='black', lw=4, ms=10)
plt.plot(leroy2008_Q2f[::2], leroy2008_Q2f[1::2], 's-', label='Q2f_leroy', color='black', lw=4, ms=10)


ax1 = plt.gca()

ax1.set_xlabel(r'$R,\,{\mathrm{arcsec}}$', fontsize=30, rotation=0)
ax1.set_ylabel(r'$Q$', fontsize=40, rotation=0)
# ax1.set_yticklabels(ax1.yaxis.get_major_ticks(), fontsize=20)
zed = [tick.label.set_fontsize(25) for tick in ax1.yaxis.get_major_ticks()]
ax1.tick_params(direction='in', pad=7)
ax1.set_xticklabels(np.arange(0, 160, 20), fontsize=25)

plt.plot([55., 55.], [0., 1.75], alpha=0.3, color='grey', lw=4)

ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(AutoMinorLocator(2))

plt.savefig(paper_imgs_dir3 +'qaz3.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'qaz3.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'qaz3.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=[12, 8])

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    label = [r'$\mathrm{Q_{eff}^{min}}$', r'$\mathrm{Q_{eff}^{max}}$', r'$\mathrm{Q_{g}}$'][ind]
    color = ['r', 'g', 'b'][ind]
    ls = ['--',':','-.'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=10, vmax=145, beam_radius=13.6/2)
    plt.plot(np.array(range(0, 145, 10))+5., az_max, label=label, color=color, lw=5,ls=ls)
#     plt.fill_between(np.array(range(0, 145, 10))+5., 
#                      np.array(az_av)-np.array(az_dev), 
#                      np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)
#     plt.fill_between(np.array(range(5, 120, 5))+2.5, 
#                      np.array(az_min), 
#                      np.array(az_max), alpha=0.1, color=color)

plt.grid()
plt.legend(fontsize=30)
# plt.axvline(x=55., alpha=0.3)
plt.ylim(0, 1000.5)
plt.xlim(10, 140.)

# plt.plot(leroy2008_Qg[::2], leroy2008_Qg[1::2], 'o-', label='Qg_leroy', color='black', lw=4, ms=10)
# plt.plot(leroy2008_Q2f[::2], leroy2008_Q2f[1::2], 's-', label='Q2f_leroy', color='black', lw=4, ms=10)

plt.plot(np.array(range(0, 145, 10))+5., 
         [np.pi*(_/2.11)**2 - np.pi*((_-10.)/2.11)**2 for _ in np.array(range(10, 155, 10))], '.-')

# ax1 = plt.gca()

# ax1.set_xlabel(r'$R,\,{\mathrm{arcsec}}$', fontsize=30, rotation=0)
# ax1.set_ylabel(r'$Q$', fontsize=40, rotation=0)
# # ax1.set_yticklabels(ax1.yaxis.get_major_ticks(), fontsize=20)
# zed = [tick.label.set_fontsize(25) for tick in ax1.yaxis.get_major_ticks()]
# ax1.tick_params(direction='in', pad=7)
# ax1.set_xticklabels(np.arange(0, 160, 20), fontsize=25)

# plt.plot([55., 55.], [0., 1.75], alpha=0.3, color='grey', lw=4)

# ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
# ax1.yaxis.set_minor_locator(AutoMinorLocator(2))

# plt.savefig(paper_imgs_dir3 +'qaz.eps', format='eps', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'qaz.png', format='png', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'qaz.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# TODO: понять, откуда берутся контуры внутри звезд и как их убрать

plt.figure(figsize=[30, 27])

plt.subplot(3,1,1)
quadmesh = plt.pcolormesh(X, Y, Qeff_min, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
# CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,min}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((512, 512), 10., color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
# circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10) #7px approx 15arcsec
# plt.gca().add_artist(circle)
# add_SFR_contours(colors=('red'))
CS = plt.contour(X, Y, dist_, [50., 80., 100.], colors=('g'))

# circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='m', fill=False, lw=6, ls='--')
# plt.gca().add_artist(circle2)

# add_spirals(plt.gca(), colors=['m', 'm'])

plt.subplot(3,1,2)
quadmesh = plt.pcolormesh(X, Y, Qeff_max, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
# CS = plt.contour(X, Y, Qeff_max, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,max}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
# add_SFR_contours(colors=('red'))
CS = plt.contour(X, Y, dist_, [50., 80., 100.], colors=('g'))

plt.subplot(3,1,3)
quadmesh = plt.pcolormesh(X, Y, Qgs, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
# CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'$Q_{g}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
# add_SFR_contours(colors=('red'))
CS = plt.contour(X, Y, dist_, [50., 80., 100.], colors=('g'))

усореднение $\Sigma$ и $\sigma$ - как у Лероя и для них уже Q

In [ ]:
def azimuthal_average(Q, step=5, vmax=120, Qmax=100, vmin=0, beam_radius=5.6/2):
    az_av = []
    az_dev = []
    az_max = []
    az_min = []
    for dbin in range(vmin, vmax, step):
        mask = (dist_ >= dbin) & (dist_ <= dbin + step)
        anulli = Q[mask]
        mask = (anulli < Qmax) & (anulli > 0) #not count nans
        anulli = anulli[mask]
        az_av.append(np.mean(anulli))
        beam_pix = np.pi*(beam_radius/2.11)**2
        az_dev.append(np.std(anulli)/np.sqrt(mask.sum()/beam_pix)) #see (E1) in Leroy et al. (2008)
#         az_max.append(np.max(anulli) if len(anulli) != 0 else np.nan)
        az_max.append(mask.sum())
        az_min.append(np.min(anulli) if len(anulli) != 0 else np.nan)
    return az_av, az_dev, az_max, az_min

In [ ]:
fig = plt.figure(figsize=[20, 7])

ax = plt.subplot(121)
ax1 = ax

for ind, Q in enumerate([Sigma_HI_, Sigma_HERACLES_CO_, Sigma_s_Leroy_]):
    label = [r'$\mathrm{HI}$', r'$\mathrm{H_2}$', r'$\mathrm{star}$'][ind]
    color = ['r', 'g', 'm'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=5, vmax=145, Qmax=1000)
    plt.plot(np.array(range(0, 145, 5))+2.5, az_av, '--', label=label, color=color,lw=4)
    if label != r'$\mathrm{star}$':
        plt.fill_between(np.array(range(0, 145, 5))+2.5, 
                         np.array(az_av)-3*np.array(az_dev), 
                         np.array(az_av)+3*np.array(az_dev), alpha=0.1, color=color)

az_av, az_dev, az_max, az_min = azimuthal_average(Sigma_s_Leroy_, step=5, vmax=55, Qmax=2000, vmin=0)
plt.fill_between(np.array(range(0, 55, 5))+2.5, 
                         np.array(az_av)-3*np.array(az_dev), 
                         np.array(az_av)+3*np.array(az_dev), alpha=0.1, color=color)
az_av, az_dev, az_max, az_min = azimuthal_average(Sigma_s_Leroy_, step=5, vmax=145, Qmax=500, vmin=50)
plt.fill_between(np.array(range(50, 145, 5))+2.5, 
                         np.array(az_av)-3*np.array(az_dev), 
                         np.array(az_av)+3*np.array(az_dev), alpha=0.1, color=color)

plt.grid()
plt.legend(fontsize=25, loc=(0.14, 0.87), ncol=3)
plt.axvline(x=55., alpha=0.3, color='grey', lw=4)
ax.set_yscale("log", nonposy='clip')
plt.ylim(1, 1000)
plt.xlim(0, 140)

az_av, az_dev, az_max, az_min = azimuthal_average(Sigma_HERACLES_CO_grad, step=5, vmax=145, Qmax=1000)
plt.plot(np.array(range(0, 145, 5))+2.5, az_av, '-', label=label, color='g',lw=2)


ax = plt.subplot(122)
ax2 = ax

for ind, Q in enumerate([c_g_, sig_R_max, sig_R_min]):
    label = [r'$c_\mathrm{g}$', r'$\sigma_R^\mathrm{max}$', r'$\sigma_R^\mathrm{min}$'][ind]
    color = ['r', 'g', 'm'][ind]
    az_av, az_dev, az_max, az_min = azimuthal_average(Q, step=5, vmax=145, Qmax=100000)
    plt.plot(np.array(range(0, 145, 5))+2.5, az_av, '--', label=label, color=color, lw=4)
    plt.fill_between(np.array(range(0, 145, 5))+2.5, 
                     np.array(az_av)-3*np.array(az_dev), 
                     np.array(az_av)+3*np.array(az_dev), alpha=0.1, color=color)

plt.grid()
plt.legend(fontsize=27, loc="upper left", bbox_to_anchor=[0, 1], ncol=3)
plt.axvline(x=55., alpha=0.3, color='grey', lw=4)
ax.set_yscale("log", nonposy='clip')
plt.ylim(8)
plt.xlim(0, 140)

plt.axhline(y=11.)
sigmaR_leroy = lambda l: sqrt(2*pi*4.32*2.3*l/7.3)/0.6
plt.plot(np.array(range(0, 140, 5))+2.5,
         [sigmaR_leroy(_) for _ in azimuthal_average(Sigma_s_Leroy_, step=5, vmax=140, Qmax=1000)[0]], 'o-', ms=9)

ax1.set_xlabel(r'$R,\,{\mathrm{arcsec}}$', fontsize=30, rotation=0)
ax2.set_xlabel(r'$R,\,{\mathrm{arcsec}}$', fontsize=30, rotation=0)

ax1.set_ylabel(r'$\Sigma, \mathrm{M_{\odot}/{pc}^2}$', fontsize=35)
ax2.set_ylabel(r'$\sigma,\,\mathrm{km/s}$', fontsize=35)

# ax1.set_yticklabels(ax1.yaxis.get_major_ticks(), fontsize=20)
zed = [tick.label.set_fontsize(25) for tick in ax1.yaxis.get_major_ticks()]
ax1.tick_params(direction='in', pad=7)
zed = [tick.label.set_fontsize(25) for tick in ax2.yaxis.get_major_ticks()]
ax2.tick_params(direction='in', pad=7)

ax1.set_xticklabels(np.arange(0, 160, 20), fontsize=25)
ax2.set_xticklabels(np.arange(0, 160, 20), fontsize=25)

ax2.set_ylim(9., 500)

ax1.tick_params(axis='both', color='black', length=7, width=1)
ax1.tick_params(axis='both', color='black', length=4, width=1, which='minor')

ax2.tick_params(axis='both', color='black', length=7, width=1)
ax2.tick_params(axis='both', color='black', length=4, width=1, which='minor')

# ax1.set_title(r'$\mathrm{accuracy\,\,and\,\,recall}$', fontsize=25)
# ax2.set_title(r'$F_1(\mathrm{Q_{eff}^{min}}, \log \Sigma_{\mathrm{SFR}})$', fontsize=25)
# ax3.set_title(r'$F_1$', fontsize=25)

plt.savefig(paper_imgs_dir3 +'data_3sig.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'data_3sig.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'data_3sig.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
G = 4.32
nnn = len(range(0, 145, 5))
Qeff_maxp = np.zeros(nnn)
Qeff_minp = np.zeros(nnn)
Qgsp = np.zeros(nnn)

Qeff_maxpu = np.zeros(nnn)
Qeff_minpu = np.zeros(nnn)
Qgspu = np.zeros(nnn)

Qeff_maxpd = np.zeros(nnn)
Qeff_minpd = np.zeros(nnn)
Qgspd = np.zeros(nnn)

Sigma_HI_p, Sigma_HI_e = azimuthal_average(Sigma_HI_, step=5, vmax=145, Qmax=100000)[:2]
Sigma_HERACLES_CO_p, Sigma_HERACLES_CO_e = azimuthal_average(Sigma_HERACLES_CO_, step=5, vmax=145, Qmax=100000)[:2]
Sigma_s_Leroy_p, Sigma_s_Leroy_e = azimuthal_average(Sigma_s_Leroy_, step=5, vmax=145, Qmax=100000)[:2]

c_g_p, c_g_e = azimuthal_average(c_g_, step=5, vmax=145, Qmax=100000)[:2]
sig_R_maxp, sig_R_maxe = azimuthal_average(sig_R_max, step=5, vmax=145, Qmax=100000)[:2]
sig_R_minp, sig_R_mine = azimuthal_average(sig_R_min, step=5, vmax=145, Qmax=100000)[:2]

errors = 0
for i, ddd in enumerate(range(0, 145, 5)):
    c = c_g_p[i]
    Sigma = 1.36*(Sigma_HI_p[i] + Sigma_HERACLES_CO_p[i])
    Sigma_e = 1.36*(Sigma_HI_e[i] + Sigma_HERACLES_CO_e[i])
    kappa = epicyclicFreq_real(gas_approx, ddd, scale)
    Sigma_s_ = Sigma_s_Leroy_p[i]
    Sigma_s_e = Sigma_s_Leroy_e[i]
    s_R_max = sig_R_maxp[i]
    s_R_min = sig_R_minp[i]
    Qg = kappa * c / (math.pi * G * Sigma)
    Qs_max = kappa * s_R_max / (3.36 * G * Sigma_s_)
    Qs_min = kappa * s_R_min / (3.36 * G * Sigma_s_)
    
    Qgu = kappa * (c + c_g_e[i]) / (math.pi * G * (Sigma-Sigma_e))
    Qs_maxu = kappa * (s_R_max + sig_R_maxe[i]) / (3.36 * G * (Sigma_s_ - Sigma_s_e))
    Qs_minu = kappa * (s_R_min + sig_R_mine[i]) / (3.36 * G * (Sigma_s_ - Sigma_s_e))
    
    Qgd = kappa * (c - c_g_e[i]) / (math.pi * G * (Sigma+Sigma_e))
    Qs_maxd = kappa * (s_R_max - sig_R_maxe[i]) / (3.36 * G * (Sigma_s_ + Sigma_s_e))
    Qs_mind = kappa * (s_R_min - sig_R_mine[i]) / (3.36 * G * (Sigma_s_ + Sigma_s_e))

    try:
        root_for_max, qeff = findInvKinemQeffBrentq(Qs_min, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
    except ValueError:
        errors+=1
        qeff = 0.01
        root_for_max = -1.
    Qeff_minp[i] = 1./qeff

    try:
        root_for_max, qeff = findInvKinemQeffBrentq(Qs_max, Qg, c/s_R_max, np.arange(0.01, 50., 1.))
    except ValueError:
        errors+=1
        qeff = 0.01
        root_for_max = -1.
    Qeff_maxp[i] = 1./qeff
    
    Qgsp[i] = Qg
    
    #########################################
    try:
        root_for_max, qeff = findInvKinemQeffBrentq(Qs_minu, Qgu, 
                                                    (c + c_g_e[i])/(s_R_min+ sig_R_mine[i]), 
                                                    np.arange(0.01, 50., 1.))
    except ValueError:
        errors+=1
        qeff = 0.01
        root_for_max = -1.
    Qeff_minpu[i] = 1./qeff

    try:
        root_for_max, qeff = findInvKinemQeffBrentq(Qs_maxu, Qgu, 
                                                    (c+ c_g_e[i])/(s_R_max+ sig_R_maxe[i]), 
                                                    np.arange(0.01, 50., 1.))
    except ValueError:
        errors+=1
        qeff = 0.01
        root_for_max = -1.
    Qeff_maxpu[i] = 1./qeff
    
    Qgspu[i] = Qgu
    
    #########################################
    try:
        root_for_max, qeff = findInvKinemQeffBrentq(Qs_mind, Qgd, 
                                                    (c - c_g_e[i])/(s_R_min - sig_R_mine[i]), 
                                                    np.arange(0.01, 50., 1.))
    except ValueError:
        errors+=1
        qeff = 0.01
        root_for_max = -1.
    Qeff_minpd[i] = 1./qeff

    try:
        root_for_max, qeff = findInvKinemQeffBrentq(Qs_maxd, Qgd, 
                                                    (c - c_g_e[i])/(s_R_max - sig_R_maxe[i]), 
                                                    np.arange(0.01, 50., 1.))
    except ValueError:
        errors+=1
        qeff = 0.01
        root_for_max = -1.
    Qeff_maxpd[i] = 1./qeff
    
    Qgspd[i] = Qgd
        
print 'Errors:{}'.format(errors)

In [ ]:
fig = plt.figure(figsize=[12, 8])

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    label = [r'$\mathrm{Q_{eff}^{min}}$', r'$\mathrm{Q_{eff}^{max}}$', r'$\mathrm{Q_{g}}$'][ind]
    color = ['r', 'g', 'b'][ind]
    ls = ['--',':','-.'][ind]
    az_av = [Qeff_minp, Qeff_maxp, Qgsp][ind]
    az_dev = [Qeff_minpu, Qeff_maxpu, Qgspu][ind]
    az_dev2 = [Qeff_minpd, Qeff_maxpd, Qgspd][ind]
    plt.plot(np.array(range(0, 145, 5))+2.5, az_av, label=label, color=color, lw=5,ls=ls)
#     plt.fill_between(np.array(range(0, 145, 10))+5., 
#                      np.array(az_av)-np.array(az_dev), 
#                      np.array(az_av)+np.array(az_dev), alpha=0.1, color=color)
    plt.fill_between(np.array(range(0, 145, 5))+2.5, 
                     np.array(az_dev), 
                     np.array(az_dev2), alpha=0.1, color=color)

plt.grid()
plt.legend(fontsize=30)
# plt.axvline(x=55., alpha=0.3)
plt.ylim(0, 12.5)
plt.xlim(10, 140.)

plt.plot(leroy2008_Qg[::2], leroy2008_Qg[1::2], 'o-', label='Qg_leroy', color='black', lw=4, ms=10)
plt.plot(leroy2008_Q2f[::2], leroy2008_Q2f[1::2], 's-', label='Q2f_leroy', color='black', lw=4, ms=10)


ax1 = plt.gca()

ax1.set_xlabel(r'$R,\,{\mathrm{arcsec}}$', fontsize=30, rotation=0)
ax1.set_ylabel(r'$Q$', fontsize=40, rotation=0)
# ax1.set_yticklabels(ax1.yaxis.get_major_ticks(), fontsize=20)
zed = [tick.label.set_fontsize(25) for tick in ax1.yaxis.get_major_ticks()]
ax1.tick_params(direction='in', pad=7)
ax1.set_xticklabels(np.arange(0, 160, 20), fontsize=25)

plt.plot([55., 55.], [0., 1.75], alpha=0.3, color='grey', lw=4)

ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(AutoMinorLocator(2))

plt.savefig(paper_imgs_dir3 +'qaz_1sig.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'qaz_1sig.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'qaz_1sig.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

## как повлияет на результат использование изменяющегося $X_{\rm CO}$

Используя модель из Leroy (2008) предполагалось $X_{\rm CO} = 2.0\times10^{20} \mathrm{cm^{-2}K^{-1}km^{-1}s}$.

В работе http://iopscience.iop.org/article/10.1088/0004-637X/764/2/117/pdf был найден градиент этой величины для двух случаев степенного закона $N=1$ и $N=1.5$: 

 $$log(X_{CO}) = 20.09(0.05)\, +\, 0.06(0.02)R\:[kpc]\: N=1$$
 $$log(X_{CO}) = 20.00(0.06)\, +\, 0.08(0.02)R\:[kpc]\: N=1.5$$

Была еще также работа Aniano (2012), где нет никакого градиента. Они объясняют плохость этой работы тем, что 

> Inspecting their Figure 4, it is evident that the results of their measurements of XCO in NGC 628 are subject to strong systematic uncertainties, and show large changes (from an increasing to a flat, or even decreasing, radial trend) depending on which subset of the Spitzer+Herschel data is used and the spatial resolution at which the modeling is conducted. Given the large systematic uncertainties in their measurement of XCO for NGC 628, we refrain from conducting a detailed comparison to their results.

In [ ]:
Image('alpha_CO_variations_VENGA.png', width=600)

Посмотрим на разброс:

In [ ]:
# log(X_CO) = 20.09(0.05) + 0.06(0.02)R[kpc] N=1
# log(X_CO) = 20.00(0.06) + 0.08(0.02)R[kpc] N=1.5

X_CO_10 = []
X_CO_15 = []
for r in range(8):
    X_CO_10.append((np.power(10., 0.09 + 0.06*r), np.power(10., 0.04 + 0.04*r), np.power(10., 0.14 + 0.08*r)))
    X_CO_15.append((np.power(10., 0.0 + 0.08*r), np.power(10., -0.06 + 0.06*r), np.power(10., 0.06 + 0.1*r)))
    
plt.plot(range(8), zip(*X_CO_10)[0], '-', color='b')
plt.fill_between(range(8), zip(*X_CO_10)[1], zip(*X_CO_10)[2], color='b', alpha=0.3)
plt.plot(range(8), zip(*X_CO_15)[0], '-', color='r')
plt.fill_between(range(8), zip(*X_CO_15)[1], zip(*X_CO_15)[2], color='r', alpha=0.3)
plt.ylabel(r'$X_{CO}$')
plt.xlabel(r'$R[kpc]$')

Видно, что хотя ошибки довольно заметны, средние похожи друг на друга. Используем для $N=1$ (поскольку они именно ее обсуждают в выводах). Надо также не забыть, что у них использованы расстояния 8.6 Мпк.

In [ ]:
%%time

G = 4.32
Qeff_max = np.zeros(dist_.shape)
Qeff_min = np.zeros(dist_.shape)
bark = np.zeros(dist_.shape)
bark_max = np.zeros(dist_.shape)
Qgs = np.zeros(dist_.shape)
Qss_min = np.zeros(dist_.shape)
Qss_max = np.zeros(dist_.shape)
Sigma_HERACLES_CO_grad = np.zeros(dist_.shape)

step = 0
for i, row in enumerate(dist_):
    for j, col in enumerate(dist_[0]):
        if step%1000 == 0:
            print step
        step+=1
        c = c_g_[i][j]
        r = dist_[i][j]
        rkpc = r*calc_scale(8.6)
        X_CO_10 = np.power(10., 0.09 + 0.06*rkpc)
        Sigma_HERACLES_CO_grad[i][j] = X_CO_10*Sigma_HERACLES_CO_[i][j]/2.0
        Sigma = 1.36*(Sigma_HI_[i][j] + X_CO_10*Sigma_HERACLES_CO_[i][j]/2.0)
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        Sigma_s_ = Sigma_s_Leroy_[i][j]
        s_R_max = sig_R_max[i][j]
        s_R_min = sig_R_min[i][j]
        Qg = kappa * c / (math.pi * G * Sigma)
        Qs_max = kappa * s_R_max / (3.36 * G * Sigma_s_)
        Qs_min = kappa * s_R_min / (3.36 * G * Sigma_s_)
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_min, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
        except ValueError:
            qeff = 0.01
            root_for_max = -1.
        Qeff_min[i][j] = 1./qeff
        bark[i][j] = root_for_max
        
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_max, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
        except ValueError:
            qeff = 0.01
            root_for_max = -1.
        Qeff_max[i][j] = 1./qeff
        bark_max[i][j] = root_for_max
        
        Qgs[i][j] = Qg
        Qss_min[i][j] = Qs_min
        Qss_max[i][j] = Qs_max

In [ ]:
plt.figure(figsize=[30, 54])

plt.subplot(6,1,1)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, Qeff_min, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,min}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours(levels=[-34.5], colors=('red'))


plt.subplot(6,1,2)
quadmesh = plt.pcolormesh(X, Y, Qeff_max, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_max, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,max}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours(levels=[-34.5], colors=('red'))


plt.subplot(6,1,4)
quadmesh = plt.pcolormesh(X, Y, bark, cmap=cm.gray)
quadmesh.set_clim(vmin=0., vmax=2.)
plt.colorbar()
plt.title(r'$\bar{k}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])

plt.subplot(6,1,3)
quadmesh = plt.pcolormesh(X, Y, Qgs, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'$Q_{g}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours(levels=[-34.5], colors=('red'))


plt.subplot(6,1,5)
quadmesh = plt.pcolormesh(X, Y, Sigma_HERACLES_CO_, cmap=cm.hot)
quadmesh.set_clim(vmin=10., vmax=45.)
plt.colorbar()
CS = plt.contour(X, Y, Sigma_HERACLES_CO_, levels=[15., 25.], colors=('b', 'g'))
plt.title(r'$\Sigma_{\rm CO}}$', fontsize=30)
mark_stars(colors=('b'), levels=[0.999, 1.001])


plt.subplot(6,1,6)
quadmesh = plt.pcolormesh(X, Y, Sigma_HERACLES_CO_grad, cmap=cm.hot)
quadmesh.set_clim(vmin=10., vmax=45.)
plt.colorbar()
CS = plt.contour(X, Y, Sigma_HERACLES_CO_grad, levels=[15., 25.], colors=('b', 'g'))
plt.title(r'$\Sigma_{\rm CO}^{X_{CO}}$', fontsize=30)
mark_stars(colors=('b'), levels=[0.999, 1.001])

# plt.savefig('result_maps_column3.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=[30, 54])

plt.subplot(6,1,5)
quadmesh = plt.pcolormesh(X, Y, Sigma_HERACLES_CO_, cmap=cm.hot)
quadmesh.set_clim(vmin=10., vmax=45.)
plt.colorbar()
CS = plt.contour(X, Y, Sigma_HERACLES_CO_, levels=[15., 25.], colors=('b', 'g'))
plt.title(r'$\Sigma_{\rm CO}}$', fontsize=30)
mark_stars(colors=('b'), levels=[0.999, 1.001])


plt.subplot(6,1,6)
quadmesh = plt.pcolormesh(X, Y, Sigma_HERACLES_CO_grad, cmap=cm.hot)
quadmesh.set_clim(vmin=10., vmax=45.)
plt.colorbar()
CS = plt.contour(X, Y, Sigma_HERACLES_CO_grad, levels=[15., 25.], colors=('b', 'g'))
plt.title(r'$\Sigma_{\rm CO}^{\delta X_{CO}}$', fontsize=30)
mark_stars(colors=('b'), levels=[0.999, 1.001])

# plt.savefig('result_maps_column3.png', format='png', bbox_inches='tight')
plt.show()

В целом ничего особо не поменялось, поскольку отличия не такие большие, как видно из двух последних карт. Периферия и центр изменились на картах $Q$, но там мы ничего особо и не могли померять.

## сравнение с уровнями $\Sigma_{H2}$

In [ ]:
SFR = np.nan_to_num(log_Halpha_SFR_.ravel())
SFR[SFR > -0.5] = -1000.
SFR = np.power(10., SFR)
print SFR.sum(), len(SFR), mean(SFR), max(SFR)

In [ ]:
(175.92+157.3)*(53.82+ 55.42)*scale*scale #площадь

С этим что-то не сходится у меня

In [ ]:
plt.figure(figsize=[30, 18])
plt.subplot(2,1,1)
X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
quadmesh = plt.pcolormesh(X, Y, Sigma_HERACLES_CO_, cmap=cm.jet)
quadmesh.set_clim(vmin=0., vmax=45.)
plt.colorbar()
CS = plt.contour(X, Y, Sigma_HERACLES_CO_, levels=[15., 30.], colors=('g', 'm'), linestyles='dashed')
plt.title(r'$\Sigma_{CO}$', fontsize=30)
#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours()

plt.subplot(2,1,2)
X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
quadmesh = plt.pcolormesh(X, Y, Sigma_HERACLES_CO_grad, cmap=cm.jet)
quadmesh.set_clim(vmin=0., vmax=45.)
plt.colorbar()
CS = plt.contour(X, Y, Sigma_HERACLES_CO_grad, levels=[15., 30.], colors=('g', 'm'), linestyles='dashed')
plt.title(r'$\Sigma_{CO}$', fontsize=30)
#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours()

В целом очень похоже, но как будто сдвинуто, что странно. 

## shear (Hunter 1998)

Hunter et al. (1998), 'competition with shear' unstable according to Leroy: $$\frac{\alpha_A\sigma_g A}{G\pi\Sigma_g} < 1,$$ где $A = 0.5(\frac{v}{r}-\frac{dv}{dr})$ постоянная Оорта. Hunter et al. (1998) предложили $\alpha_A \approx 2.5$.

In [ ]:
def oort_a(r, gas_vel, scale):
    try:
        return 0.5 * (gas_vel(r)/r - gas_vel.deriv()(r))/(scale)
    except:
        return 0.5 * (gas_vel(r)/r - gas_vel.derivative()(r))/(scale)
    
def Sigma_crit_A(r, gas_vel, alpha, sound_vel, Sigma_g, scale):
    G = 4.32
    return (alpha * sound_vel * oort_a(r, gas_vel, scale)) / (np.pi * G * Sigma_g)

In [ ]:
test_points = np.linspace(0, 300., 100)
plt.plot(test_points, [oort_a(x, gas_approx, scale) for x in test_points], '-', label='poly')
plt.ylim(0, 50)
plt.xlabel('$R, arcsec$')
plt.ylabel('$d\Omega/dr$', fontsize=15);

In [ ]:
%%time

shear = np.zeros(dist_.shape)

for i, row in enumerate(dist_):
    if i%100 == 0:
        print i
    for j, col in enumerate(dist_[0]):
        c = c_g_[i][j]
        Sigma_g = 1.36*(Sigma_HI_[i][j] + Sigma_HERACLES_CO_[i][j])
        shear[i][j] = Sigma_crit_A(dist_[i][j], gas_approx, 2.5, c, Sigma_g, scale)

In [ ]:
plt.figure(figsize=[30, 9])
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
quadmesh = plt.pcolormesh(X, Y, shear, cmap=cm.jet)
quadmesh.set_clim(vmin=0., vmax=3.)
plt.colorbar()
CS = plt.contour(X, Y, shear, levels=[1.], colors=('g', 'm'), linestyles='dashed')

plt.title('Shear', fontsize=30)

#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours()

Т.о. shear ничего не может объяснить или требуется другая величина $\alpha_A$

## SFR через $\lambda$

https://arxiv.org/pdf/1702.02609.pdf (и http://iopscience.iop.org/article/10.1086/431734/pdf), но там плохо тем, что формула не подходит ибо турбулентность (я посмотрел первоисточники [Krumholz 2005](http://iopscience.iop.org/article/10.1086/431734/pdf) - вроде можно использовать формулу) $$SFR=\epsilon_{eff}(\frac{\pi^3}{2})^{0.5}G\lambda^2\frac{\Sigma_g^2}{\sigma_g}(1+\frac{\Sigma_*}{\Sigma_g}\frac{\sigma_g}{\sigma_*})^{0.5}$$ и затем $\Sigma_{SFR} = SFR/Area$ и еще проследить, чтобы $\lambda$ была в правильных единицах.

In [ ]:
bark = np.load('bark.npy')
lambda_kpc = np.load('lambda_kpc.npy')

In [ ]:
0.008*sqrt(pi**3)/sqrt(2)

In [ ]:
3.24/3.17

In [ ]:
((175.92+157.3)/1553)*scale, ((53.82+55.42)/513)*scale

In [ ]:
#  X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
((175.92+157.3)/1553)*((53.82+55.42)/513)*(scale**2)

Проверим точку из Dib грубо:

In [ ]:
G = 4.32
e_ff = 0.008
multipl = e_ff*sqrt(pi**3)/sqrt(2)*1.0221*G/1000.
area = 0.5

(multipl*(0.8**2)*(42.**2)*np.sqrt(2)/10.)/area, np.power(10., -1.3)

Да, очень похоже. Значит и у нас должно быть верно.

In [ ]:
%%time

G = 4.32
Sigma_lambda = np.zeros(dist_.shape)
e_ff = 0.008

for i, row in enumerate(dist_):
    if i%100 == 0:
        print i
    for j, col in enumerate(dist_[0]):
        c = c_g_[i][j]
        Sigma = 1.36*(Sigma_HI_[i][j] + Sigma_HERACLES_CO_[i][j])
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        Sigma_s_ = Sigma_s_Leroy_[i][j]
        s_R_max = sig_R_max[i][j]
        s_R_min = sig_R_min[i][j]
        lambd = lambda_kpc[i][j]
        multipl = e_ff*sqrt(pi**3)/sqrt(2.)*1.0221*G/1000.
#         area = 5.723/100000.
        #TODO: какое значение тут?
#         area = 5.723
        area = 0.086**2
        Sigma_lambda[i][j] = (multipl*(lambd**2)*(Sigma**2)*np.sqrt(1 + (Sigma_s_/s_R_min)*(c/Sigma))/c)/area

In [ ]:
fig = plt.figure(figsize=[30, 18])

plt.subplot(2,1,1)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, log_Halpha_SFR_, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=-3.5, vmax=-1.)
plt.colorbar()
# CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
plt.title(r'$SFR_{obs}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])


plt.subplot(2,1,2)
quadmesh = plt.pcolormesh(X, Y, np.log10(Sigma_lambda), cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=-2., vmax=2.)
# quadmesh.set_clim(vmin=10.0, vmax=2000.)
plt.colorbar()
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(levels=[-2.1])

fig.subplots_adjust(wspace=0.01, hspace=0.01)
plt.show()

In [ ]:
mask = sig_los != 0

In [ ]:
plt.figure(figsize=[16, 11])
plt.scatter(np.power(10., log_Halpha_SFR_[mask]).ravel(), Sigma_lambda[mask].ravel(), s=1., c=np.nan_to_num(dist_[mask].ravel()), cmap='plasma')
plt.xlim(0., 0.02)
plt.ylim(0., 0.40)
plt.colorbar()
plt.show()

In [ ]:
# create data
# x = np.nan_to_num(log_Halpha_SFR_[mask].ravel())
# y = np.nan_to_num(np.log10(Sigma_lambda[mask].ravel()))

x = log_Halpha_SFR_[mask].ravel()
y = np.log10(Sigma_lambda[mask].ravel())
 
# Make the plot
plt.figure(figsize=[14, 12])
plt.hexbin(x, y, gridsize=100, bins='log', cmap='inferno')
plt.ylim(-4, 0.)
plt.xlim(-4, 0)
plt.colorbar()
law = 1.
law_bias = 0.
plt.plot([-4., -1.], [-4.*law+law_bias, -1.*law+law_bias], '--', lw=10, color='g')
plt.xlabel('log(Sigma_SFR)')
plt.ylabel('log(Sigma_lambda)')
plt.show()

**Kennicutt–Schmidt law**

Со звездами формирует странные непонятные кластеры. Интересно. Возможно это же причина "нитей" на картинке после этой.

In [ ]:
Sigma_g = 1.36*(Sigma_HI_ + Sigma_HERACLES_CO_)

In [ ]:
plt.figure(figsize=[16, 11])
plt.scatter(np.power(10., log_Halpha_SFR_)[mask].ravel(), np.power(Sigma_g, 1.4)[mask].ravel(), s=1., c=np.nan_to_num(dist_[mask].ravel()), cmap='plasma')
plt.xlim(0., 0.02)
plt.colorbar()
plt.show()

In [ ]:
Sigma_g = 1.36*(Sigma_HI_ + Sigma_HERACLES_CO_)
fs=20

fig = plt.figure(figsize=[20, 16])

plt.subplot(2,2,1)
# create data
x = np.nan_to_num(log_Halpha_SFR_[mask].ravel())
y = np.nan_to_num(np.log10(Sigma_g[mask].ravel()))
 
# Make the plot
plt.hexbin(x, y, gridsize=100, bins='log', cmap='inferno', vmin=0.2, vmax=1.5)
plt.ylim(0.2/1.4, 2.5/1.4)
plt.xlim(-3.5, -1.0)
plt.colorbar()
kslaw = 1.48
kslaw_bias = 5.
plt.plot([-4., -2.], [-4.*kslaw+kslaw_bias, -2.*kslaw+kslaw_bias], '--', lw=20, color='g')
plt.xlabel('log(Sigma_SFR)', fontsize=fs)
plt.ylabel('log(Sigma_g)', fontsize=fs)

plt.subplot(2,2,2)
# create data
x = np.nan_to_num(log_Halpha_SFR_[mask].ravel())
y = np.nan_to_num(np.log10(Sigma_HI_[mask].ravel()))
 
# Make the plot
plt.hexbin(x, y, gridsize=100, bins='log', cmap='inferno', vmin=0.2, vmax=1.5)
plt.ylim(-0.5, 1.)
plt.xlim(-3.5, -1.0)
plt.colorbar()
# kslaw = 1.48
# kslaw_bias = 5.
# plt.plot([-4., -2.], [-4.*kslaw+kslaw_bias, -2.*kslaw+kslaw_bias], '--', lw=20, color='g')
plt.xlabel('log(Sigma_SFR)', fontsize=fs)
plt.ylabel('log(Sigma_HI)', fontsize=fs)

plt.subplot(2,2,3)
# create data
x = np.nan_to_num(log_Halpha_SFR_[mask].ravel())
y = np.nan_to_num(np.log10(Sigma_HERACLES_CO_[mask].ravel()))
 
# Make the plot
plt.hexbin(x, y, gridsize=100, bins='log', cmap='inferno', vmin=0.2, vmax=1.5)
plt.ylim(0.2/1.4, 1.6)
plt.xlim(-3.5, -1.0)
plt.colorbar()
# kslaw = 1.48
# kslaw_bias = 5.
# plt.plot([-4., -2.], [-4.*kslaw+kslaw_bias, -2.*kslaw+kslaw_bias], '--', lw=20, color='g')
plt.xlabel('log(Sigma_SFR)', fontsize=fs)
plt.ylabel('log(Sigma_CO)', fontsize=fs)

plt.subplot(2,2,4)
# create data
x = np.nan_to_num(log_Halpha_SFR_[mask].ravel())
y = np.nan_to_num(np.log10(Sigma_s_Leroy_[mask].ravel()))
 
# Make the plot
plt.hexbin(x, y, gridsize=100, bins='log', cmap='inferno', vmin=0.2, vmax=1.5)
plt.ylim(1.1, 3.5)
plt.xlim(-3.5, -1.0)
plt.colorbar()
# kslaw = 1.48
# kslaw_bias = 5.
# plt.plot([-4., -2.], [-4.*kslaw+kslaw_bias, -2.*kslaw+kslaw_bias], '--', lw=20, color='g')
plt.xlabel('log(Sigma_SFR)', fontsize=fs)
plt.ylabel('log(Sigma_s)', fontsize=fs)

fig.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

Все как и должно быть - видно подобие K-S, видно что с атомарным не коррелирует и что он молекуляризируется при 10ке, видно что со звездами коррелирует.

In [ ]:
Sigma_g = 1.36*(Sigma_HI_ + Sigma_HERACLES_CO_)
fs=20

fig = plt.figure(figsize=[20, 16])

plt.subplot(2,2,1)
# create data
x = np.nan_to_num(np.log10(Sigma_s_Leroy_[mask].ravel()))
y = np.nan_to_num(np.log10(Sigma_HI_[mask].ravel()))
 
# Make the plot
plt.hexbin(x, y, gridsize=100, bins='log', cmap='inferno', vmin=0.2, vmax=1.5)
plt.ylim(0.2/1.4, 1.)
plt.xlim(1, 3.5)
plt.colorbar()
# kslaw = 1.48
# kslaw_bias = 5.
# plt.plot([-4., -2.], [-4.*kslaw+kslaw_bias, -2.*kslaw+kslaw_bias], '--', lw=20, color='g')
plt.xlabel('log(Sigma_s)', fontsize=fs)
plt.ylabel('log(Sigma_HI)', fontsize=fs)

plt.subplot(2,2,2)
# create data
x = np.nan_to_num(np.log10(Sigma_s_Leroy_[mask].ravel()))
y = np.nan_to_num(np.log10(Sigma_HERACLES_CO_[mask].ravel()))
 
# Make the plot
plt.hexbin(x, y, gridsize=100, bins='log', cmap='inferno', vmin=0.2, vmax=1.5)
plt.ylim(0.2/1.4, 2.5/1.4)
plt.xlim(1, 3.5)
plt.colorbar()
# kslaw = 1.48
# kslaw_bias = 5.
# plt.plot([-4., -2.], [-4.*kslaw+kslaw_bias, -2.*kslaw+kslaw_bias], '--', lw=20, color='g')
plt.xlabel('log(Sigma_s)', fontsize=fs)
plt.ylabel('log(Sigma_CO)', fontsize=fs)

plt.subplot(2,2,3)
x = np.nan_to_num(np.power(10., log_Halpha_SFR_[mask]).ravel())
y = np.nan_to_num(Sigma_g[mask].ravel())
z = np.divide(x, y)
z_ = np.power(Sigma_s_Leroy_[mask].ravel(), 0.5)
 
# Make the plot
plt.hexbin(np.log10(z), np.log10(z_), gridsize=100, bins='log', cmap='inferno', vmin=0.2, vmax=1.5)
# plt.ylim(0.2/1.4, 1.)
# plt.xlim(0.2, 1.5)
plt.colorbar()
plt.xlabel('log(Sigma_SFR/Sigma_g)', fontsize=fs)
plt.ylabel('log(Sigma_s^{0.5})', fontsize=fs)
plt.title('Ostriker 2010')

plt.subplot(2,2,4)
 
# Make the plot
plt.hexbin(np.log10(z), np.log10(Sigma_s_Leroy_[mask].ravel()), gridsize=100, bins='log', cmap='inferno', vmin=0.2, vmax=1.5)
# plt.ylim(0.2/1.4, 1.)
# plt.xlim(0.2, 1.5)
plt.colorbar()
plt.xlabel('log(Sigma_SFR/Sigma_g)', fontsize=fs)
plt.ylabel('log(Sigma_s)', fontsize=fs)

fig.subplots_adjust(wspace=0.1, hspace=0.2)
plt.show()

In [ ]:
x = np.nan_to_num(np.power(10., log_Halpha_SFR_[mask]).ravel())
y = np.nan_to_num(Sigma_g[mask].ravel())
z = np.divide(x, y)
z_ = np.power(Sigma_s_Leroy_[mask].ravel(), 0.5)
 
# Make the plot
plt.hexbin(np.log10(z), np.log10(z_), gridsize=50, cmap='inferno')
# plt.ylim(0.2/1.4, 1.)
# plt.xlim(0.2, 1.5)
plt.colorbar()
plt.xlabel('log(Sigma_SFR/Sigma_g)', fontsize=fs)
plt.ylabel('log(Sigma_s^{0.5})', fontsize=fs)
plt.title('Ostriker 2010')

картинка для статьи

In [ ]:
from scipy.optimize import curve_fit

fig, axes = plt.subplots(figsize=[21, 7], ncols=3, sharey=True)

ax1 = plt.subplot(131)

mask = (sig_los != 0.) & (dist_ > 15.) & (dist_ < 140.)

Sigma_g = 1.36*(Sigma_HI_ + Sigma_HERACLES_CO_)
fs=45

x = np.nan_to_num(log_Halpha_SFR_[mask].ravel())
y = np.nan_to_num(np.log10(Sigma_g[mask].ravel()))

# hb = plt.hexbin(x, y, gridsize=50, bins='log', cmap='inferno')
hb = plt.hexbin(y[(y>0.2)&(y<1.7)], x[(y>0.2)&(y<1.7)], gridsize=50, cmap=cm.inferno_r, vmin=1, vmax=45)
hb.cmap.set_under('white')

def func(x, beta):
    return 1.48*x + beta

guesses = [-4.]
popt,pcov = curve_fit(func,y,x,p0=guesses)

print popt,pcov

def func(x, beta, alpha):
    return alpha*x + beta

guesses = [-4., 1.48]
popt,pcov = curve_fit(func,y,x,p0=guesses)

print popt,pcov


plt.xlim(0.2/1.4, 1.7)
plt.ylim(-3.5, -1.0)
# plt.colorbar()
kslaw = 1.48
kslaw_bias = -4.13
plt.plot([0.5, 1.5], [0.5*kslaw+kslaw_bias, 1.5*kslaw+kslaw_bias], '-', lw=7, color='g')
plt.ylabel(r'$\log \Sigma_\mathrm{SFR}$', fontsize=fs)
plt.xlabel(r'$\log \Sigma_\mathrm{g}$', fontsize=fs)

ax2 = plt.subplot(132)

x = np.nan_to_num(log_Halpha_SFR_[mask & (Sigma_HERACLES_CO_ > 0.01)].ravel())
y = np.nan_to_num(np.log10(1.36*Sigma_HERACLES_CO_[mask & (Sigma_HERACLES_CO_ > 0.01)].ravel()))
# y[y < -3] = -3.
 
# Make the plot
hb = ax2.hexbin(y[(y>0.2)&(y<1.7)], x[(y>0.2)&(y<1.7)], gridsize=50, cmap=cm.inferno_r, vmin=1, vmax=45)
hb.cmap.set_under('white')
print corrcoef(x,y)[0][1]
plt.xlim(0.2/1.4, 1.7)
plt.ylim(-3.5, -1.0)
# plt.colorbar()
# plt.xlabel(r'$\log \Sigma_\mathrm{SFR}$', fontsize=fs)
plt.xlabel(r'$\log \Sigma_\mathrm{H_2}$', fontsize=fs)
plt.yticks([])

kslaw = 1.
kslaw_bias = -3.3
plt.plot([0.3, 1.5], [0.3*kslaw+kslaw_bias, 1.5*kslaw+kslaw_bias], '--', lw=7, color='b')

ax3 = plt.subplot(133)


# x = np.nan_to_num(np.power(10., log_Halpha_SFR_[mask]).ravel())
x = np.nan_to_num(log_Halpha_SFR_[mask].ravel())
y = np.nan_to_num(Sigma_g[mask].ravel())
# z = np.divide(x, y)
z_ = np.power(Sigma_s_Leroy_[mask].ravel(), 0.5)
z = np.multiply(z_, y)
 
# Make the plot
hb = plt.hexbin(np.log10(z), x, gridsize=50, cmap=cm.inferno_r, vmin=1, vmax=45)

hb.cmap.set_under('white')

def func(x, beta):
    return 1*x + beta

guesses = [-4.5]
popt,pcov = curve_fit(func,np.log10(z), x,p0=guesses)

print popt,pcov

# print np.polyfit(np.log10(z), x, 1)
# print corrcoef(np.log10(z), np.log10(z_))[0][1]
# plt.ylim(0.2/1.4, 1.6)
# plt.xlim(1.2)
# plt.ylim(-3.5, -1.0)
# plt.colorbar()
# plt.xlabel(r'$\log \Sigma_\mathrm{SFR}$', fontsize=fs)
plt.xlabel(r'$\log (\Sigma_\mathrm{g}\Sigma_\mathrm{s}^{0.5})$', fontsize=fs)
# plt.title('Ostriker 2010')
plt.yticks([])

kslaw = 1.
kslaw_bias = -4.59
plt.plot([1.5, 2.5], [1.5*kslaw+kslaw_bias, 2.5*kslaw+kslaw_bias], '-', lw=7, color='g')


cbar_ax = fig.add_axes([0.90, 0.10, 0.02, 0.8])
fig.colorbar(hb, cax=cbar_ax)
cbar_ax.set_yticklabels(range(5, 50, 5), fontsize=25)
plt.ylabel(r'$\mathrm{counts}$', fontsize=50)
# plt.gca().yaxis.set_label_coords(1.75, 0.5)

zed = [tick.label.set_fontsize(25) for tick in ax1.yaxis.get_major_ticks()]
zed = [tick.label.set_fontsize(25) for tick in ax2.yaxis.get_major_ticks()]
zed = [tick.label.set_fontsize(25) for tick in ax3.yaxis.get_major_ticks()]

zed = [tick.label.set_fontsize(25) for tick in ax1.xaxis.get_major_ticks()]
zed = [tick.label.set_fontsize(25) for tick in ax2.xaxis.get_major_ticks()]
zed = [tick.label.set_fontsize(25) for tick in ax3.xaxis.get_major_ticks()]

# ax1.set_xlabel(r'$\log \Sigma_{\mathrm{SFR}}$', fontsize=25, rotation=0)
# ax2.set_xlabel(r'$\log \Sigma_{\mathrm{SFR}}$', fontsize=25, rotation=0)
# ax3.set_xlabel(r'$\log \Sigma_{\mathrm{SFR}}$', fontsize=25, rotation=0)

# ax2.set_ylabel(r'$\mathrm{Q_{eff}^{min}}$', fontsize=30, rotation=0)
# ax2.yaxis.labelpad = -75

# ax1.set_xticklabels(np.arange(-3.5, -0.5, 0.5), fontsize=20)
# ax1.tick_params(direction='in', pad=7)
# ax2.set_xticklabels(np.arange(-3.5, -0.5, 0.5), fontsize=20)
# ax2.tick_params(direction='in', pad=7)
# ax3.set_xticklabels(np.arange(-3.5, -0.5, 0.5), fontsize=20)
# ax3.tick_params(direction='in', pad=7)

# ax1.set_yticklabels(np.arange(0., 1.2, 0.2), fontsize=20)
# ax2.set_yticklabels(np.arange(1., 4.5, 0.5), fontsize=20)
# ax3.set_yticklabels(np.arange(0., 1.2, 0.2), fontsize=20)

# ax1.set_title(r'$\mathrm{accuracy\,\,and\,\,recall}$', fontsize=25)
# ax2.set_title(r'$F_1(\mathrm{Q_{eff}^{min}}, \log \Sigma_{\mathrm{SFR}})$', fontsize=25)
# ax3.set_title(r'$F_1$', fontsize=25)

fig.subplots_adjust(wspace=0.01, hspace=0.0)

plt.savefig(paper_imgs_dir3 +'ks.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'ks.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'ks.pdf', format='pdf', dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
mask = (sig_los != 0.) & (dist_ > 15.) & (dist_ < 140.) & (Qeff_min < 5)

x = np.nan_to_num(log_Halpha_SFR_[mask].ravel())
y = np.nan_to_num(Qeff_min[mask].ravel())

 
# Make the plot
plt.hexbin(x, y, gridsize=50, cmap='inferno')
plt.ylim(0., 5.)
# plt.xlim(0.2, 1.5)
plt.colorbar()
plt.xlabel('log(Sigma_SFR)', fontsize=fs)
plt.ylabel('Qeff_min', fontsize=fs)

## $\sigma_R$ через толщину (e.g. Leroy 2008)

$\sigma_R = \sigma_Z/0.6 = \frac{1}{0.6}\sqrt{\frac{2\pi G l_s\Sigma_s}{7.3}}$, где $l_s=2.3\:kpc$ у Leroy. 

In [ ]:
sigmaR_leroy = lambda l: sqrt(2*pi*4.32*2.3*l/7.3)/0.6

In [ ]:
plt.plot(np.linspace(10, 1000., 10),[sigmaR_leroy(_) for _ in np.linspace(10, 1000., 10)], '.-')
plt.xlim(0., 1000.);

In [ ]:
%%time

G = 4.32
Qeff_max = np.zeros(dist_.shape)
Qeff_min = np.zeros(dist_.shape)
bark = np.zeros(dist_.shape)
bark_max = np.zeros(dist_.shape)
Qgs = np.zeros(dist_.shape)
Qss_min = np.zeros(dist_.shape)
Qss_max = np.zeros(dist_.shape)

for i, row in enumerate(dist_):
    if i%100 == 0:
        print i
    for j, col in enumerate(dist_[0]):
        c = c_g_[i][j]
        Sigma = 1.36*(Sigma_HI_[i][j] + Sigma_HERACLES_CO_[i][j])
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        Sigma_s_ = Sigma_s_Leroy_[i][j]
        s_R_max = sigmaR_leroy(Sigma_s_)
        Qg = kappa * c / (math.pi * G * Sigma)
        Qs_max = kappa * s_R_max / (3.36 * G * Sigma_s_)       
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_max, Qg, c/s_R_max, np.arange(0.01, 50., 1.))
        except ValueError:
            qeff = 0.01
            root_for_max = -1.
        Qeff_max[i][j] = 1./qeff
        bark_max[i][j] = root_for_max
        
        Qgs[i][j] = Qg
        Qss_max[i][j] = Qs_max

In [ ]:
np.save('Qeff_min_Leroy.npy', Qeff_max)

In [ ]:
plt.figure(figsize=[30, 36])

plt.subplot(4,1,1)
quadmesh = plt.pcolormesh(X, Y, Qeff_max, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_max, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,max}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours(levels=[-34.5], colors=('red'))


plt.subplot(4,1,2)
quadmesh = plt.pcolormesh(X, Y, bark_max, cmap=cm.gray)
quadmesh.set_clim(vmin=0., vmax=2.)
plt.colorbar()
plt.title(r'$\bar{k}_{max}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])

plt.subplot(4,1,3)
quadmesh = plt.pcolormesh(X, Y, Qgs, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'$Q_{g}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(levels=[-34.5], colors=('red'))

plt.subplot(4,1,4)
quadmesh = plt.pcolormesh(X, Y, Qss_max, cmap=cm.hot)
quadmesh.set_clim(vmin=2.5, vmax=8.)
plt.colorbar()
plt.title(r'$Q_{s}^{max}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])

# plt.savefig('result_maps_column5.png', format='png', bbox_inches='tight')
plt.show()

## $\sigma_R$ через производную

$${\displaystyle \sigma_{\varphi}^{2}/\sigma_{R}^{2}=0.5\left(1+\frac{R}{\bar{v}_{\varphi}}\frac{d\bar{v}_{\varphi}}{dR}\right)}$$

In [ ]:
def sigPhi_to_sigR_real(R):
        return 0.5 * (1 + R*gas_approx.deriv()(R) / gas_approx(R))

test_points = np.linspace(0.1, 300., 100)
plt.plot(test_points, [sigPhi_to_sigR_real(R) for R in test_points], 'd-', color='green')
plt.axhline(y=0.3)
plt.axhline(y=0.7)
plt.xlabel('$R$')
plt.ylabel(r"$\sigma_{\varphi}^2/\sigma_{R}^2$")
plt.ylim(0)
plt.show()

In [ ]:
def dotproduct(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def length(v):
    return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
    return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))

phi = np.zeros(sig_los.shape)

for i in range(len(sig_los[0])):
    for j in range(len(sig_los)):
        x, y  = 813.5-i, 253.5-j
        phi[j][i] = angle(np.array([x,y]), np.array([1553., 1553*np.sin(22.5*np.pi/180.)-60.]))*180./np.pi

$$\sigma^2_{los, \phi} = [\sigma_R^2\sin^2\phi + \sigma^2_{\varphi}\cos^2\phi]\sin^2 i + \sigma_z^2\cos^2 i,$$

In [ ]:
sig_r_max = lambda sig, phi_, R: sig/np.sqrt(sin_i**2 * np.sin(phi_*np.pi/180.)**2 + sigPhi_to_sigR_real(R)*sin_i**2*np.cos(phi_*np.pi/180.)**2 + 0.09*cos_i**2) 
sig_r_min = lambda sig, phi_, R: sig/np.sqrt(sin_i**2 * np.sin(phi_*np.pi/180.)**2 + sigPhi_to_sigR_real(R)*sin_i**2*np.cos(phi_*np.pi/180.)**2 + 0.49*cos_i**2) 

In [ ]:
%%time
sig_R_max = np.zeros(sig_los.shape)

for i in range(len(sig_los)):
    for j in range(len(sig_los[0])):
        sig_R_max[i][j] = sig_r_max(sig_los[i][j], phi[i][j], dist_[i][j])
        
sig_R_min = np.zeros(sig_los.shape)

for i in range(len(sig_los)):
    for j in range(len(sig_los[0])):
        sig_R_min[i][j] = sig_r_min(sig_los[i][j], phi[i][j], dist_[i][j])

In [ ]:
%%time

G = 4.32
Qeff_max = np.zeros(dist_.shape)
Qeff_min = np.zeros(dist_.shape)
bark = np.zeros(dist_.shape)
bark_max = np.zeros(dist_.shape)
Qgs = np.zeros(dist_.shape)
Qss_min = np.zeros(dist_.shape)
Qss_max = np.zeros(dist_.shape)

for i, row in enumerate(dist_):
    if i%100 == 0:
        print i
    for j, col in enumerate(dist_[0]):
        c = c_g_[i][j]
        Sigma = 1.36*(Sigma_HI_[i][j] + Sigma_HERACLES_CO_[i][j])
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        Sigma_s_ = Sigma_s_Leroy_[i][j]
        s_R_max = sig_R_max[i][j]
        s_R_min = sig_R_min[i][j]
        Qg = kappa * c / (math.pi * G * Sigma)
        Qs_max = kappa * s_R_max / (3.36 * G * Sigma_s_)
        Qs_min = kappa * s_R_min / (3.36 * G * Sigma_s_)
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_min, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
        except ValueError:
            qeff = 0.01
            root_for_max = -1.
        Qeff_min[i][j] = 1./qeff
        bark[i][j] = root_for_max
        
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_max, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
        except ValueError:
            qeff = 0.01
            root_for_max = -1.
        Qeff_max[i][j] = 1./qeff
        bark_max[i][j] = root_for_max
        
        Qgs[i][j] = Qg
        Qss_min[i][j] = Qs_min
        Qss_max[i][j] = Qs_max

In [ ]:
np.save('Qeff_min_deriv.npy', Qeff_min)

In [ ]:
plt.figure(figsize=[30, 54])

plt.subplot(6,1,1)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, Qeff_min, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,min}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours(levels=[-34.5], colors=('red'))


plt.subplot(6,1,2)
quadmesh = plt.pcolormesh(X, Y, Qeff_max, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_max, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,max}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours(levels=[-34.5], colors=('red'))


plt.subplot(6,1,4)
quadmesh = plt.pcolormesh(X, Y, bark, cmap=cm.gray)
quadmesh.set_clim(vmin=0., vmax=2.)
plt.colorbar()
plt.title(r'$\bar{k}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])


plt.subplot(6,1,3)
quadmesh = plt.pcolormesh(X, Y, Qgs, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'$Q_{g}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(levels=[-34.5], colors=('red'))


plt.subplot(6,1,5)
quadmesh = plt.pcolormesh(X, Y, Qss_min, cmap=cm.hot)
quadmesh.set_clim(vmin=1.5, vmax=5.)
plt.colorbar()
plt.title(r'$Q_{s}^{min}$', fontsize=30)
mark_stars(colors=('b'), levels=[0.999, 1.001])

plt.subplot(6,1,6)
quadmesh = plt.pcolormesh(X, Y, Qss_max, cmap=cm.hot)
quadmesh.set_clim(vmin=2.5, vmax=8.)
plt.colorbar()
plt.title(r'$Q_{s}^{max}$', fontsize=30)
mark_stars(colors=('b'), levels=[0.999, 1.001])


# plt.savefig('result_maps_column5.png', format='png', bbox_inches='tight')
plt.show()

In [ ]:
sig_R_max_ = np.load('sig_R_max.npy')
sig_R_min_ = np.load('sig_R_min.npy')

Отличия минимальны, да и сами оценки $\sigma_R$ отличаются слабо:

In [ ]:
plt.figure(figsize=[30, 9])
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
quadmesh = plt.pcolormesh(X, Y, sig_R_max_-sig_R_max, cmap=cm.jet)
quadmesh.set_clim(vmin=0., vmax=8.)
plt.colorbar()
# CS = plt.contour(X, Y, shear, levels=[1., 2.], colors=('g', 'm'), linestyles='dashed')

plt.title('delta sig_R_max', fontsize=30)

# add_SFR_contours(colors=('b', 'g'))
mark_stars()

In [ ]:
l1 = lambda sig, phi_, R: sig/np.sqrt(sin_i**2 * np.sin(phi_*np.pi/180.)**2 + 0.25*sin_i**2*np.cos(phi_*np.pi/180.)**2 + 0.09*cos_i**2) 
l2 = lambda sig, phi_, R: sig/np.sqrt(sin_i**2 * np.sin(phi_*np.pi/180.)**2 + sigPhi_to_sigR_real(R)*sin_i**2*np.cos(phi_*np.pi/180.)**2 + 0.09*cos_i**2) 

test_points = np.linspace(0.1, 300., 100)
plt.plot(test_points, [l1(_, 0., R) for _, R in zip(sig_los[35, :], test_points)], 'd-', color='green')
plt.plot(test_points, [l2(_, 0., R) for _, R in zip(sig_los[35, :], test_points)], 'd-', color='r')
# plt.axhline(y=0.3)
# plt.axhline(y=0.7)
# plt.xlabel('$R$')
# plt.ylabel(r"$\sigma_{\varphi}^2/\sigma_{R}^2$")
plt.ylim(0)
plt.show()

Да, разница минимальна.

## $\sigma_R$ в случае равновесного SVE

Как в случае Dib 2016.

In [ ]:
%%time

G = 4.32
Qeff_max = np.zeros(dist_.shape)
Qeff_min = np.zeros(dist_.shape)
bark = np.zeros(dist_.shape)
bark_max = np.zeros(dist_.shape)
Qgs = np.zeros(dist_.shape)
Qss_min = np.zeros(dist_.shape)
Qss_max = np.zeros(dist_.shape)

for i, row in enumerate(dist_):
    if i%100 == 0:
        print i
    for j, col in enumerate(dist_[0]):
        c = c_g_[i][j]
        Sigma = 1.36*(Sigma_HI_[i][j] + Sigma_HERACLES_CO_[i][j])
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        Sigma_s_ = Sigma_s_Leroy_[i][j]
        s_R_max = sig_los[i][j]
        s_R_min = sig_los[i][j]
        Qg = kappa * c / (math.pi * G * Sigma)
        Qs_max = kappa * s_R_max / (3.36 * G * Sigma_s_)
        Qs_min = kappa * s_R_min / (3.36 * G * Sigma_s_)
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_min, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
        except ValueError:
            qeff = 0.01
            root_for_max = -1.
        Qeff_min[i][j] = 1./qeff
        bark[i][j] = root_for_max
        
        try:
            root_for_max, qeff = findInvKinemQeffBrentq(Qs_max, Qg, c/s_R_min, np.arange(0.01, 50., 1.))
        except ValueError:
            qeff = 0.01
            root_for_max = -1.
        Qeff_max[i][j] = 1./qeff
        bark_max[i][j] = root_for_max
        
        Qgs[i][j] = Qg
        Qss_min[i][j] = Qs_min
        Qss_max[i][j] = Qs_max

In [ ]:
np.save('Qeff_min_isotherm.npy', Qeff_min)

In [ ]:
plt.figure(figsize=[30, 27])

plt.subplot(3,1,1)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, Qeff_min, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_min, levels=[1., 2.5], colors=('m','g'))
plt.title(r'$Q_{eff,min}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10)
plt.gca().add_artist(circle)
add_SFR_contours()

plt.subplot(3,1,3)
quadmesh = plt.pcolormesh(X, Y, bark, cmap=cm.gray)
quadmesh.set_clim(vmin=0., vmax=2.)
plt.colorbar()
plt.title(r'$\bar{k}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])

plt.subplot(3,1,2)
quadmesh = plt.pcolormesh(X, Y, Qgs, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'$Q_{g}$', fontsize=30)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours()

# plt.savefig('result_maps_column4.png', format='png', bbox_inches='tight')
plt.show()

Видно, что сильно перегрето выходит.

## cравнение $\sigma_R$

In [ ]:
Qeff_min = np.load('Qeff_min.npy')
Qeff_min_Leroy = np.load('Qeff_min_Leroy.npy')
Qeff_min_deriv = np.load('Qeff_min_deriv.npy')
Qeff_min_isotherm = np.load('Qeff_min_isotherm.npy')
Qeff_min_romeo = np.load('Qeff_min_romeo.npy')

In [ ]:
fig = plt.figure(figsize=[30, 45])

for ind, Qdat in enumerate([Qeff_min, Qeff_min_Leroy, Qeff_min_deriv, Qeff_min_isotherm, Qeff_min_romeo]):
    plt.subplot(5,1,ind+1)
    # X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
    quadmesh = plt.pcolormesh(X, Y, Qdat, cmap=cm.gist_yarg)
    # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
    quadmesh.set_clim(vmin=2.0, vmax=4.)
    plt.colorbar()
    # CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
    plt.title('{}'.format(['Qeff_min', 'Qeff_min_Leroy', 'Qeff_min_deriv', 'Qeff_min_isotherm', 'Qeff_min_romeo'][ind]), fontsize=30)
    # add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
    mark_stars(colors=('y'), levels=[0.999, 1.001])
    #bulge
    # circle = plt.Circle((512, 512), 10., color='b', fill=False, lw=10)
    # plt.gca().add_artist(circle)
    # circle = plt.Circle((Nx/2, Ny/2), 7., color='b', fill=False, lw=10) #7px approx 15arcsec
    # plt.gca().add_artist(circle)
    add_SFR_contours(colors=('red'))

    circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='m', fill=False, lw=6, ls='--')
    plt.gca().add_artist(circle2)

    add_spirals(plt.gca(), colors=['m', 'm'])
    
fig.subplots_adjust(wspace=0.1, hspace=0.1)

## статистика

По корреляции [THE DISKMASS SURVEY. VIII. ON THE RELATIONSHIP BETWEEN DISK STABILITY AND STAR FORMATION](http://iopscience.iop.org/article/10.1088/0004-637X/785/1/43/pdf)  и там коэффициент порядка $0.53 \pm 0.19$ (что похоже на наше).

Без звезд

In [ ]:
mask = sig_los != 0. 

Qeff_min = np.load('Qeff_min.npy')
bark = np.load('bark.npy')
Qeff_max = np.load('Qeff_max.npy')
bark = np.load('bark_max.npy')
Qgs = np.load('Qgs.npy')
Qss_min = np.load('Qss_min.npy')
Qss_max = np.load('Qss_max.npy')

X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))

https://stats.stackexchange.com/questions/73646/how-do-i-test-that-two-continuous-variables-are-independent

In [ ]:
np.count_nonzero(np.isnan(log_Halpha_SFR_.ravel())), log_Halpha_SFR_.ravel().shape

In [ ]:
np.count_nonzero(np.isnan(Qeff_min.ravel())), Qeff_min.ravel().shape

In [ ]:
print np.corrcoef(Qeff_min[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1]
# print np.corrcoef(Qss_min[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1]
print np.corrcoef(Qeff_max[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1]
# print np.corrcoef(Qss_max[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1]
print np.corrcoef(np.nan_to_num(Qgs[mask].ravel()), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1]
# TODO: не учитывать звезды

Посмотрим, как меняется радиально:

In [ ]:
corr_coeffs = []
for radius in range(15, 150, 10):
    tmp = []
    mask = (sig_los != 0.) & (dist_ < radius)
    tmp.append(np.corrcoef(Qeff_min[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1])
    tmp.append(np.corrcoef(Qeff_max[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1])
    tmp.append(np.corrcoef(Qgs[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1])
    corr_coeffs.append(tmp)
    
plt.plot(range(15, 150, 10), zip(*corr_coeffs)[0], 'o-', label='Qmin')
plt.plot(range(15, 150, 10), zip(*corr_coeffs)[1], 'o-', label='Qmax')
plt.plot(range(15, 150, 10), zip(*corr_coeffs)[2], 'o-', label='Qg')
plt.legend();

Без балджа:

In [ ]:
corr_coeffs = []
for radius in range(15, 150, 10):
    tmp = []
    mask = (sig_los != 0.) & (dist_ < radius) & (dist_ > 15.)
    tmp.append(np.corrcoef(Qeff_min[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1])
    tmp.append(np.corrcoef(Qeff_max[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1])
    tmp.append(np.corrcoef(Qgs[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1])
    corr_coeffs.append(tmp)
    
plt.plot(range(15, 150, 10), zip(*corr_coeffs)[0], 'o-', label='Qmin')
plt.plot(range(15, 150, 10), zip(*corr_coeffs)[1], 'o-', label='Qmax')
plt.plot(range(15, 150, 10), zip(*corr_coeffs)[2], 'o-', label='Qg')
plt.legend();

Теперь сделаем нечто экзотическое - будем брать разные границы $Q$ и разные силы $H\alpha$ сигнала и смотреть между ними корреляцию. Пожалуй еще все ограничим $R<130$, ибо там все начинает портиться.

In [ ]:
fig = plt.figure(figsize=[30, 5])

plt.subplot(1,3,1)
corr_coeffs = np.zeros((50,50))
for i, qlim in enumerate(np.linspace(0.5, 5., 50)):
    for j, hlim in enumerate(np.linspace(-3.5, -1., 50)):
        mask = (sig_los != 0.) & (dist_ > 15.) & (Qeff_min < qlim) & (log_Halpha_SFR_ > hlim)
        if len(Qeff_min[mask].ravel()) > 10:
            corr_coeffs[j][i] = np.corrcoef(Qeff_min[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1]
#         print i, j, corr_coeffs[i][j]

xx, yy = np.meshgrid(np.linspace(0.5, 5., 50), np.linspace(-3.5, -1., 50))
quadmesh = plt.pcolormesh(xx, yy, corr_coeffs, cmap='sauron')
quadmesh.set_clim(vmin=-0.5, vmax=0.5)
plt.colorbar()
plt.title(r'$Q_{min}$', fontsize=30)

plt.subplot(1,3,2)
corr_coeffs = np.zeros((50,50))
for i, qlim in enumerate(np.linspace(0.5, 5., 50)):
    for j, hlim in enumerate(np.linspace(-3.5, -1., 50)):
        mask = (sig_los != 0.) & (dist_ > 15.) & (Qeff_max < qlim) & (log_Halpha_SFR_ > hlim)
        if len(Qeff_max[mask].ravel()) > 10:
            corr_coeffs[j][i] = np.corrcoef(Qeff_max[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1]
#         print i, j, corr_coeffs[i][j]

xx, yy = np.meshgrid(np.linspace(0.5, 5., 50), np.linspace(-3.5, -1., 50))
quadmesh = plt.pcolormesh(xx, yy, corr_coeffs, cmap='sauron')
quadmesh.set_clim(vmin=-0.5, vmax=0.5)
plt.colorbar()
plt.title(r'$Q_{max}$', fontsize=30)

plt.subplot(1,3,3)
corr_coeffs = np.zeros((50,50))
for i, qlim in enumerate(np.linspace(0.5, 5., 50)):
    for j, hlim in enumerate(np.linspace(-3.5, -1., 50)):
        mask = (sig_los != 0.) & (dist_ > 15.) & (Qgs < qlim) & (log_Halpha_SFR_ > hlim)
        if len(Qgs[mask].ravel()) > 10:
            corr_coeffs[j][i] = np.corrcoef(Qgs[mask].ravel(), np.nan_to_num(log_Halpha_SFR_[mask].ravel()))[0][1]
#         print i, j, corr_coeffs[i][j]

xx, yy = np.meshgrid(np.linspace(0.5, 5., 50), np.linspace(-3.5, -1., 50))
quadmesh = plt.pcolormesh(xx, yy, corr_coeffs, cmap='sauron')
quadmesh.set_clim(vmin=-0.5, vmax=0.5)
plt.colorbar()
plt.title(r'$Q_{g}$', fontsize=30);

$\chi^2$ https://astropython.blogspot.ru/2012/02/computing-chi-squared-and-reduced-chi.html & http://www.astroml.org/book_figures/chapter4/fig_chi2_eval.html

In [ ]:
def chisqg(ydata,ymod,sd=None):  
    """  
    Returns the chi-square error statistic as the sum of squared errors between  
    Ydata(i) and Ymodel(i). If individual standard deviations (array sd) are supplied,   
    then the chi-square error statistic is computed as the sum of squared errors  
    divided by the standard deviations.     Inspired on the IDL procedure linfit.pro.  
    See http://en.wikipedia.org/wiki/Goodness_of_fit for reference.  

    x,y,sd assumed to be Numpy arrays. a,b scalars.  
    Returns the float chisq with the chi-square statistic.  

    Rodrigo Nemmen  
    http://goo.gl/8S1Oo  
    """  
    # Chi-square statistic (Bevington, eq. 6.9)  
    if sd==None:  
        chisq=numpy.sum((ydata-ymod)**2)  
    else:  
        chisq=numpy.sum( ((ydata-ymod)/sd)**2 )  
    return chisq  

fig = plt.figure(figsize=[30, 5])

plt.subplot(1,3,1)
corr_coeffs = np.zeros((50,50))
for i, qlim in enumerate(np.linspace(0.5, 5., 50)):
    for j, hlim in enumerate(np.linspace(-3.5, -1., 50)):
        mask = (sig_los != 0.) & (dist_ > 15.)
        corr_coeffs[j][i] = chisqg(np.array([1 if l < qlim else 0 for l in Qeff_min[mask].ravel()]), 
                                   np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()]))
        
xx, yy = np.meshgrid(np.linspace(0.5, 5., 50), np.linspace(-3.5, -1., 50))
quadmesh = plt.pcolormesh(xx, yy, corr_coeffs, cmap='sauron')
# quadmesh.set_clim(vmin=-0.5, vmax=0.5)
plt.colorbar()
plt.title(r'$Q_{min}$', fontsize=30)

plt.subplot(1,3,2)
corr_coeffs = np.zeros((50,50))
for i, qlim in enumerate(np.linspace(0.5, 5., 50)):
    for j, hlim in enumerate(np.linspace(-3.5, -1., 50)):
        mask = (sig_los != 0.) & (dist_ > 15.)
        corr_coeffs[j][i] = chisqg(np.array([1 if l < qlim else 0 for l in Qeff_max[mask].ravel()]), 
                                   np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()]))
        
xx, yy = np.meshgrid(np.linspace(0.5, 5., 50), np.linspace(-3.5, -1., 50))
quadmesh = plt.pcolormesh(xx, yy, corr_coeffs, cmap='sauron')
# quadmesh.set_clim(vmin=-0.5, vmax=0.5)
plt.colorbar()
plt.title(r'$Q_{max}$', fontsize=30)

plt.subplot(1,3,3)
corr_coeffs = np.zeros((50,50))
for i, qlim in enumerate(np.linspace(0.5, 5., 50)):
    for j, hlim in enumerate(np.linspace(-3.5, -1., 50)):
        mask = (sig_los != 0.) & (dist_ > 15.)
        corr_coeffs[j][i] = chisqg(np.array([1 if l < qlim else 0 for l in Qgs[mask].ravel()]), 
                                   np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()]))

xx, yy = np.meshgrid(np.linspace(0.5, 5., 50), np.linspace(-3.5, -1., 50))
quadmesh = plt.pcolormesh(xx, yy, corr_coeffs, cmap='sauron')
# quadmesh.set_clim(vmin=-0.5, vmax=0.5)
plt.colorbar()
plt.title(r'$Q_{g}$', fontsize=30);

ML metrics:

In [ ]:
%%time
fig, axes = plt.subplots(figsize=[30, 20], ncols=4, nrows=3)

mask = (sig_los != 0.) & (dist_ > 15.) & (dist_ < 140.)
xx, yy = np.meshgrid(np.linspace(1.0, 4., 50), np.linspace(-3.5, -1., 50))

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    coeffs = np.zeros((50,50,4))
    for i, qlim in enumerate(np.linspace(1.0, 4., 50)):
        for j, hlim in enumerate(np.linspace(-3.5, -1., 50)):
            true_signal = np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()])
            pred_signal = np.array([1 if l < qlim else 0 for l in Q[mask].ravel()])
            coeffs[j][i][0] = accuracy_score(true_signal, pred_signal)
            coeffs[j][i][1] = recall_score(true_signal, pred_signal, pos_label=1)
            coeffs[j][i][2] = precision_score(true_signal, pred_signal, pos_label=1)
            coeffs[j][i][3] = f1_score(true_signal, pred_signal, pos_label=1)

    for col in range(4):
        ax = axes[ind, col]
        quadmesh = ax.pcolormesh(xx, yy, coeffs[:, :, col], cmap='sauron')
        quadmesh.set_clim(vmin=0., vmax=1.)
        plt.colorbar(quadmesh, ax=ax)
        ax.set_title('{}:{}'.format(['Qeff_min', 'Qeff_max', 'Qg'][ind], ['accuracy', 'recall', 'precision', 'f1'][col]), fontsize=15)
        CS = ax.contour(xx, yy, coeffs[:, :, col], levels=np.linspace(0.1, 1., 10), colors=('black'), alpha=0.5)
        plt.clabel(CS, CS.levels, inline=True, fontsize=10)
        ax.axvline(x=3., ls='-', color='r', alpha=0.5)
        ax.grid()

fig.subplots_adjust(wspace=0.08, hspace=0.11)

In [ ]:
fig, axes = plt.subplots(figsize=[30, 8], ncols=4, nrows=1)

mask = (sig_los != 0.) & (dist_ > 15.) & (dist_ < 140.)
# xx, yy = np.meshgrid(np.linspace(1.0, 4., 50), np.linspace(-3.5, -1., 50))

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    coeffs = np.zeros((150,4))
    qlim = 3.0
    for j, hlim in enumerate(np.linspace(-3.5, -1., 150)):
        true_signal = np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()])
        pred_signal = np.array([1 if l < qlim else 0 for l in Q[mask].ravel()])
        coeffs[j][0] = accuracy_score(true_signal, pred_signal)
        coeffs[j][1] = recall_score(true_signal, pred_signal, pos_label=1)
        coeffs[j][2] = precision_score(true_signal, pred_signal, pos_label=1)
        coeffs[j][3] = f1_score(true_signal, pred_signal, pos_label=1)

    for col in range(4):
        ax = axes[col]
        ax.plot(np.linspace(-3.5, -1., 150), coeffs[:, col], label=['Qeff_min', 'Qeff_max', 'Qg'][ind])
#         ax.set_title('{}:{}'.format(['Qeff_min', 'Qeff_max', 'Qg'][ind], ['accuracy', 'recall', 'precision', 'f1'][col]), fontsize=15)
#         CS = ax.contour(xx, yy, coeffs[:, :, col], levels=np.linspace(0.1, 1., 10), colors=('black'), alpha=0.5)
#         plt.clabel(CS, CS.levels, inline=True, fontsize=10)
#         ax.axvline(x=3., ls='-', color='r', alpha=0.5)
        ax.grid()
        ax.set_title('{}'.format(['accuracy', 'recall', 'precision', 'f1'][col]))
        ax.axvline(x=-2.35, ls='--', alpha=0.3, color='grey')
        ax.set_ylim(0, 1.)
    plt.legend()

fig.subplots_adjust(wspace=0.08, hspace=0.11)

In [ ]:
# fig, axes = plt.subplots(figsize=[30, 8], ncols=4, nrows=1)

# mask = (sig_los != 0.) & (dist_ > 15.) & (dist_ < 140.)
# # xx, yy = np.meshgrid(np.linspace(1.0, 4., 50), np.linspace(-3.5, -1., 50))

# for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
#     coeffs = np.zeros((150,4))
#     qlim = 3.0
#     for j, hlim in enumerate(np.linspace(-3.5, -1., 150)):
#         true_signal = np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()])
#         pred_signal = np.array([1 if l < qlim else 0 for l in Q[mask].ravel()])
#         coeffs[j][0] = accuracy_score(true_signal, pred_signal)
#         coeffs[j][1] = recall_score(true_signal, pred_signal, pos_label=1)
#         coeffs[j][2] = precision_score(true_signal, pred_signal, pos_label=1)
#         coeffs[j][3] = f1_score(true_signal, pred_signal, pos_label=1)

#     for col in range(4):
#         ax = axes[col]
#         ax.plot(np.linspace(-3.5, -1., 150), coeffs[:, col], label=['Qeff_min', 'Qeff_max', 'Qg'][ind])
# #         ax.set_title('{}:{}'.format(['Qeff_min', 'Qeff_max', 'Qg'][ind], ['accuracy', 'recall', 'precision', 'f1'][col]), fontsize=15)
# #         CS = ax.contour(xx, yy, coeffs[:, :, col], levels=np.linspace(0.1, 1., 10), colors=('black'), alpha=0.5)
# #         plt.clabel(CS, CS.levels, inline=True, fontsize=10)
# #         ax.axvline(x=3., ls='-', color='r', alpha=0.5)
#         ax.grid()
#         ax.set_title('{}'.format(['accuracy', 'recall', 'precision', 'f1'][col]))
#         ax.axvline(x=-2.42, ls='--', alpha=0.3, color='grey')
#         ax.set_ylim(0, 1.)
#     plt.legend()

# fig.subplots_adjust(wspace=0.08, hspace=0.11)

картинка для статьи

In [ ]:
import matplotlib as mpl
mpl.style.use('classic')

In [ ]:
for ind, Q in enumerate([Qeff_min]):
    coeffsl = np.zeros((50,50))
    for i, qlim in enumerate(np.linspace(1.0, 4., 50)):
        for j, hlim in enumerate(np.linspace(-3.5, -1., 50)):
            true_signal = np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()])
            pred_signal = np.array([1 if l < qlim else 0 for l in Q[mask].ravel()])
#             coeffs[j][i][0] = accuracy_score(true_signal, pred_signal)
#             coeffs[j][i][1] = recall_score(true_signal, pred_signal, pos_label=1)
#             coeffs[j][i][2] = precision_score(true_signal, pred_signal, pos_label=1)
            coeffsl[j][i] = f1_score(true_signal, pred_signal, pos_label=1)

Удивительно, но точка похоже все время остается! Дергая Q или dist < R - все равно!

In [ ]:
fig = plt.figure(figsize=[21, 7])

ax2 = plt.subplot(131)

ax1 = plt.subplot(132)

ax3 = plt.subplot(133)

mask = (sig_los != 0.) & (dist_ > 15.) & (dist_ < 140.)

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    coeffs = np.zeros((150,4))
    qlim = 3.0
    for j, hlim in enumerate(np.linspace(-3.5, -1., 150)):
        true_signal = np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()])
        pred_signal = np.array([1 if l < qlim else 0 for l in Q[mask].ravel()])
        coeffs[j][0] = accuracy_score(true_signal, pred_signal)
        coeffs[j][1] = recall_score(true_signal, pred_signal, pos_label=1)
        coeffs[j][2] = precision_score(true_signal, pred_signal, pos_label=1)
        coeffs[j][3] = f1_score(true_signal, pred_signal, pos_label=1)
    
    color = ['r', 'g', 'b'][ind]
    ax3.plot(np.linspace(-3.5, -1., 150), coeffs[:, 3], 
             label=[r'$\mathrm{Q_{eff}^{min}}$', r'$\mathrm{Q_{eff}^{max}}$', 
                    r'$\mathrm{Q_{g}}$'][ind], color=color, lw=4)

    ax1.plot(np.linspace(-3.5, -1., 150), coeffs[:, 0], 
             label=['Qeff_min', 'Qeff_max', 'Qg'][ind], color=color, lw=4)
    ax1.plot(np.linspace(-3.5, -1., 150), coeffs[:, 1], '--', 
             label=['Qeff_min', 'Qeff_max', 'Qg'][ind], color=color, lw=4)

ax3.grid()
ax1.grid()
# ax3.axvline(x=-2.25, alpha=0.3, color='grey', lw=60)
# ax1.axvline(x=-2.25, alpha=0.3, color='grey', lw=60)
# ax2.axvline(x=-2.25, alpha=0.3, color='grey', lw=60)
ax3.set_ylim(0, 1.)
plt.legend(fontsize=30)

xx, yy = np.meshgrid(np.linspace(1.0, 4., 50), np.linspace(-3.5, -1., 50))
 
# quadmesh = ax2.pcolormesh(yy, xx, coeffsl[:, :], cmap='sauron')
# quadmesh.set_clim(vmin=0., vmax=1.)
# plt.colorbar(quadmesh, ax=ax)
CS = ax2.contour(yy, xx, coeffsl[:, :], levels=np.linspace(0.1, 1., 10), colors=('black'), alpha=0.8)
plt.clabel(CS, CS.levels, inline=True, fontsize=20, fmt='%.1f')


ax1.set_xlabel(r'$\log \Sigma_{\mathrm{SFR}}^{\mathrm{lim}}$', fontsize=30, rotation=0)
ax2.set_xlabel(r'$\log \Sigma_{\mathrm{SFR}}^{\mathrm{lim}}$', fontsize=30, rotation=0)
ax3.set_xlabel(r'$\log \Sigma_{\mathrm{SFR}}^{\mathrm{lim}}$', fontsize=30, rotation=0)

ax2.set_ylabel(r'$\mathrm{Q_{eff}^{min}}$', fontsize=35)
# ax2.yaxis.labelpad = -75

ax1.set_xticklabels(np.arange(-3.5, -0.5, 0.5), fontsize=20)
ax1.tick_params(direction='in', pad=7)
ax2.set_xticklabels(np.arange(-3.5, -0.5, 0.5), fontsize=20)
ax2.tick_params(direction='in', pad=7)
ax3.set_xticklabels(np.arange(-3.5, -0.5, 0.5), fontsize=20)
ax3.tick_params(direction='in', pad=7)

ax1.set_yticklabels(np.arange(0., 1.2, 0.2), fontsize=20)
ax2.set_yticklabels(np.arange(1., 4.5, 0.5), fontsize=20)
ax3.set_yticklabels(np.arange(0., 1.2, 0.2), fontsize=20)

ax1.set_title(r'$\mathrm{accuracy\,\,and\,\,recall}$', fontsize=25, y=1.01)
ax2.set_title(r'$F_1$', fontsize=25, y=1.01)
ax3.set_title(r'$F_1\, (\mathrm{Q_{lim}=3})$', fontsize=25, y=1.01)

# fig.subplots_adjust(wspace=0.08, hspace=0.11)

# plt.xlim(0, 200)
# plt.ylim(0, 300)
# plt.legend(loc=(0.1, 0.01), fontsize=24)

# plt.xlabel(r'$R,\,\mathrm{arcsec}$', fontsize=45, rotation=0)
# plt.ylabel(r'$v,\,\mathrm{km/s}$', fontsize=45)

# ax.xaxis.set_minor_locator(AutoMinorLocator(5))
# ax.yaxis.set_minor_locator(AutoMinorLocator(2))
# plt.grid(ls='-', alpha=0.5)

# # ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
# ax.set_yticklabels(range(0, 350, 50), fontsize=25)
# ax.set_xticklabels(range(0, 250, 50), fontsize=25)
# # if ind == 1:
# #     ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)

# ax.tick_params(axis='both', length=7, width=2)
# ax.tick_params(axis='both', length=4, width=1, which='minor')


plt.savefig(paper_imgs_dir3 +'match.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'match.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'match.pdf', format='pdf', dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure(figsize=[21, 7])

ax2 = plt.subplot(131)

ax1 = plt.subplot(132)

ax3 = plt.subplot(133)

mask = (sig_los != 0.) & (dist_ > 15.) & (dist_ < 140.)

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    coeffs = np.zeros((150,4))
    qlim = 3.0
    for j, hlim in enumerate(np.linspace(-3.5, -1., 150)):
        true_signal = np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()])
        pred_signal = np.array([1 if l < qlim else 0 for l in Q[mask].ravel()])
        coeffs[j][0] = accuracy_score(true_signal, pred_signal)
        coeffs[j][1] = recall_score(true_signal, pred_signal, pos_label=1)
        coeffs[j][2] = precision_score(true_signal, pred_signal, pos_label=1)
        coeffs[j][3] = f1_score(true_signal, pred_signal, pos_label=1)
    
    color = ['r', 'g', 'b'][ind]
    ax3.plot(np.linspace(-3.5, -1., 150), coeffs[:, 3], color=color, lw=4)
    
    ax3.plot([-5., np.linspace(-3.5, -1., 150)[0]], [coeffs[:, 3][0], coeffs[:, 3][0]], ['d-', 's-', 'o-'][ind],
             label=[r'$\mathrm{Q_{eff}^{min}}$', r'$\mathrm{Q_{eff}^{max}}$', 
                    r'$\mathrm{Q_{g}}$'][ind], color=color, ms=20, lw=4)

    ax1.plot(np.linspace(-3.5, -1., 150), coeffs[:, 0], color=color, lw=4)
    
    ax1.plot(np.linspace(-3.5, -1., 150)[0], coeffs[:, 0][0], ['d-', 's-', 'o-'][ind], color=color, ms=20)
    ax1.plot(np.linspace(-3.5, -1., 150)[-1], coeffs[:, 0][-1], ['d-', 's-', 'o-'][ind], color=color, ms=20)
    
    ax1.plot(np.linspace(-3.5, -1., 150), coeffs[:, 1], '--', 
             label=['Qeff_min', 'Qeff_max', 'Qg'][ind], color=color, lw=4)

ax3.grid()
ax1.grid()
# ax3.axvline(x=-2.25, alpha=0.3, color='grey', lw=60)
# ax1.axvline(x=-2.25, alpha=0.3, color='grey', lw=60)
# ax2.axvline(x=-2.25, alpha=0.3, color='grey', lw=60)
ax3.set_ylim(0, 1.)
plt.legend(fontsize=30, numpoints=1)

xx, yy = np.meshgrid(np.linspace(1.0, 4., 50), np.linspace(-3.5, -1., 50))
 
# quadmesh = ax2.pcolormesh(yy, xx, coeffsl[:, :], cmap='sauron')
# quadmesh.set_clim(vmin=0., vmax=1.)
# plt.colorbar(quadmesh, ax=ax)
CS = ax2.contour(yy, xx, coeffsl[:, :], levels=np.linspace(0.1, 1., 10), colors=('black'), alpha=0.8)
plt.clabel(CS, CS.levels, inline=True, fontsize=20, fmt='%.1f')


ax1.set_xlabel(r'$\log \Sigma_{\mathrm{SFR}}^{\mathrm{lim}}$', fontsize=30, rotation=0)
ax2.set_xlabel(r'$\log \Sigma_{\mathrm{SFR}}^{\mathrm{lim}}$', fontsize=30, rotation=0)
ax3.set_xlabel(r'$\log \Sigma_{\mathrm{SFR}}^{\mathrm{lim}}$', fontsize=30, rotation=0)

ax2.set_ylabel(r'$\mathrm{Q_{eff}^{min}}$', fontsize=35)
# ax2.yaxis.labelpad = -75

ax3.set_xlim(-3.5, -1.)
ax1.set_xlim(-3.5, -1.)

ax1.set_xticklabels(np.arange(-3.5, -0.5, 0.5), fontsize=20)
ax1.tick_params(direction='in', pad=7)
ax2.set_xticklabels(np.arange(-3.5, -0.5, 0.5), fontsize=20)
ax2.tick_params(direction='in', pad=7)
ax3.set_xticklabels(np.arange(-3.5, -0.5, 0.5), fontsize=20)
ax3.tick_params(direction='in', pad=7)

ax1.set_yticklabels(np.arange(0., 1.2, 0.2), fontsize=20)
ax2.set_yticklabels(np.arange(1., 4.5, 0.5), fontsize=20)
ax3.set_yticklabels(np.arange(0., 1.2, 0.2), fontsize=20)

ax1.set_title(r'$\mathrm{accuracy\,\,and\,\,recall}$', fontsize=25, y=1.01)
ax2.set_title(r'$F_1$', fontsize=25, y=1.01)
ax3.set_title(r'$F_1\, (\mathrm{Q_{lim}=3})$', fontsize=25, y=1.01)

# fig.subplots_adjust(wspace=0.08, hspace=0.11)

# plt.xlim(0, 200)
# plt.ylim(0, 300)
# plt.legend(loc=(0.1, 0.01), fontsize=24)

# plt.xlabel(r'$R,\,\mathrm{arcsec}$', fontsize=45, rotation=0)
# plt.ylabel(r'$v,\,\mathrm{km/s}$', fontsize=45)

# ax.xaxis.set_minor_locator(AutoMinorLocator(5))
# ax.yaxis.set_minor_locator(AutoMinorLocator(2))
# plt.grid(ls='-', alpha=0.5)

# # ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
# ax.set_yticklabels(range(0, 350, 50), fontsize=25)
# ax.set_xticklabels(range(0, 250, 50), fontsize=25)
# # if ind == 1:
# #     ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)

# ax.tick_params(axis='both', length=7, width=2)
# ax.tick_params(axis='both', length=4, width=1, which='minor')


plt.savefig(paper_imgs_dir3 +'match_colorless.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'match_colorless.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'match_colorless.pdf', format='pdf', dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
fig= plt.figure(figsize=[30,10])

ax = plt.gca()
q_level = 3.0
h_level = -2.25
ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.3)
CS1 = ax.contour(X, Y, dist_, levels=[90., 110., 120.], colors=('r'), alpha=0.3)
#         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
#         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[60,30], ncols=5, nrows=5)

for i in range(0, 5):
    for j in range(0, 5):
        ax = axes[j][i]
        q_level = range(1, 6)[i]
        h_level = np.linspace(-3.5, -1., 5)[::-1][j]
        ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
        CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.1)
#         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
#         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
        CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
        CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
        mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

fig.subplots_adjust(wspace=0.01, hspace=0.06)
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[60,30], ncols=5, nrows=5)

for i in range(0, 5):
    for j in range(0, 5):
        ax = axes[j][i]
        q_level = range(1, 6)[i]
        h_level = np.linspace(-3.5, -1., 5)[::-1][j]
        ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
        CS = ax.contourf(X, Y, Qgs, levels=[0., q_level], colors=('m'), alpha=0.1)
#         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
#         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
        CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
        CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
        
        mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

fig.subplots_adjust(wspace=0.01, hspace=0.06)
plt.show()

In [ ]:
def chisqg(ydata,ymod,sd=None):  
    """  
    Returns the chi-square error statistic as the sum of squared errors between  
    Ydata(i) and Ymodel(i). If individual standard deviations (array sd) are supplied,   
    then the chi-square error statistic is computed as the sum of squared errors  
    divided by the standard deviations.     Inspired on the IDL procedure linfit.pro.  
    See http://en.wikipedia.org/wiki/Goodness_of_fit for reference.  

    x,y,sd assumed to be Numpy arrays. a,b scalars.  
    Returns the float chisq with the chi-square statistic.  

    Rodrigo Nemmen  
    http://goo.gl/8S1Oo  
    """  
    # Chi-square statistic (Bevington, eq. 6.9)  
    if sd==None:  
        chisq=numpy.sum((ydata-ymod)**2)  
    else:  
        chisq=numpy.sum( ((ydata-ymod)/sd)**2 )  
    return chisq  

In [ ]:
fig, axes = plt.subplots(figsize=[50,25], ncols=5, nrows=5)

for i in range(0, 5):
    for j in range(0, 5):
        mask = (sig_los != 0.) & (dist_ > 15.)
        ax = axes[j][i]
        q_level = range(1, 6)[i]
        h_level = np.linspace(-3.5, -1., 5)[::-1][j]
        chi = chisqg(np.array([1 if l < q_level else 0 for l in Qeff_min[mask].ravel()]), 
                                   np.array([1 if l > h_level else 0 for l in log_Halpha_SFR_[mask].ravel()]))
        ax.set_title('q={:3.2f}, h={:3.2f}, chi={:3.2f}'.format(q_level, h_level, chi), fontsize=19)
        signal_match = np.array([0 if ((l[0] < q_level) & (l[1] > h_level)) | (l[0] > q_level) & (l[1] < h_level) else 1 for l in 
                                 zip(Qeff_min.ravel(), log_Halpha_SFR_.ravel())]).reshape((Ny, Nx))
        quadmesh = ax.pcolormesh(X, Y, signal_match, cmap=cm.hot)
        quadmesh.set_clim(vmin=0, vmax=1.)
#         CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.1)
#         CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.1)
# #         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
# #         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
#         CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
#         CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
        mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])
        add_SFR_contours(ax=ax, colors=('red'))

        circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='m', fill=False, lw=6, ls='--')
        ax.add_artist(circle2)

        add_spirals(ax, colors=['m', 'm'])

fig.subplots_adjust(wspace=0.01, hspace=0.15)
plt.show()

Выясним где точно минимум лучший самый:

In [ ]:
def chisqg(ydata,ymod,sd=None):  
    """  
    Returns the chi-square error statistic as the sum of squared errors between  
    Ydata(i) and Ymodel(i). If individual standard deviations (array sd) are supplied,   
    then the chi-square error statistic is computed as the sum of squared errors  
    divided by the standard deviations.     Inspired on the IDL procedure linfit.pro.  
    See http://en.wikipedia.org/wiki/Goodness_of_fit for reference.  

    x,y,sd assumed to be Numpy arrays. a,b scalars.  
    Returns the float chisq with the chi-square statistic.  

    Rodrigo Nemmen  
    http://goo.gl/8S1Oo  
    """  
    # Chi-square statistic (Bevington, eq. 6.9)  
    if sd==None:  
        chisq=numpy.sum((ydata-ymod)**2)  
    else:  
        chisq=numpy.sum( ((ydata-ymod)/sd)**2 )  
    return chisq  

fig = plt.figure(figsize=[30, 5])

mask = (sig_los != 0.) & (dist_ > 15.) & (dist_ < 140.)

plt.subplot(1,3,1)
corr_coeffs = np.zeros((100,100))
for i, qlim in enumerate(np.linspace(2.2, 3.2, 100)):
    for j, hlim in enumerate(np.linspace(-2.5, -1.7, 100)):
        corr_coeffs[j][i] = chisqg(np.array([1 if l < qlim else 0 for l in Qeff_min[mask].ravel()]), 
                                   np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()]))
        
quadmesh = plt.pcolormesh(xx, yy, corr_coeffs, cmap='sauron')
quadmesh.set_clim(vmin=500, vmax=3700)
plt.colorbar()
CS = plt.contour(xx, yy, corr_coeffs, levels=np.linspace(1000., 3000., 10), colors=('black'), alpha=0.5)
plt.clabel(CS, CS.levels, inline=True, fontsize=10)
plt.title(r'$Q_{min}$', fontsize=30)

plt.subplot(1,3,2)
corr_coeffs = np.zeros((100,100))
for i, qlim in enumerate(np.linspace(2.2, 3.2, 100)):
    for j, hlim in enumerate(np.linspace(-2.5, -1.7, 100)):
        corr_coeffs[j][i] = chisqg(np.array([1 if l < qlim else 0 for l in Qeff_max[mask].ravel()]), 
                                   np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()]))
        
quadmesh = plt.pcolormesh(xx, yy, corr_coeffs, cmap='sauron')
quadmesh.set_clim(vmin=500, vmax=3700)
plt.colorbar()
CS = plt.contour(xx, yy, corr_coeffs, levels=np.linspace(1000., 3000., 10), colors=('black'), alpha=0.5)
plt.clabel(CS, CS.levels, inline=True, fontsize=10)
plt.title(r'$Q_{max}$', fontsize=30)

plt.subplot(1,3,3)
corr_coeffs = np.zeros((100,100))
for i, qlim in enumerate(np.linspace(2.2, 3.2, 100)):
    for j, hlim in enumerate(np.linspace(-2.5, -1.7, 100)):
        corr_coeffs[j][i] = chisqg(np.array([1 if l < qlim else 0 for l in Qgs[mask].ravel()]), 
                                   np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()]))

quadmesh = plt.pcolormesh(xx, yy, corr_coeffs, cmap='sauron')
quadmesh.set_clim(vmin=500, vmax=3700)
plt.colorbar()
CS = plt.contour(xx, yy, corr_coeffs, levels=np.linspace(1000., 3000., 10), colors=('black'), alpha=0.5)
plt.clabel(CS, CS.levels, inline=True, fontsize=10)
plt.title(r'$Q_{g}$', fontsize=30);

In [ ]:
fig = plt.figure(figsize=[10, 5])

mask = (sig_los != 0.) & (dist_ > 15.) & (dist_ < 130.)

corr_coeffs = np.zeros((100,100))
for i, qlim in enumerate(np.linspace(2.2, 3.2, 100)):
    for j, hlim in enumerate(np.linspace(-2.5, -1.7, 100)):
        corr_coeffs[j][i] = chisqg(np.array([1 if l < qlim else 0 for l in Qeff_min[mask].ravel()]), 
                                   np.array([1 if l > hlim else 0 for l in log_Halpha_SFR_[mask].ravel()]))
        
quadmesh = plt.pcolormesh(xx, yy, corr_coeffs, cmap='sauron')
quadmesh.set_clim(vmin=500, vmax=3700)
plt.colorbar()
CS = plt.contour(xx, yy, corr_coeffs, levels=np.linspace(1000., 3000., 30), colors=('black'), alpha=0.5)
plt.clabel(CS, CS.levels, inline=True, fontsize=10)
plt.title(r'$Q_{min}$', fontsize=30)
plt.grid()

In [ ]:
fig= plt.figure(figsize=[30,10])

ax = plt.gca()
q_level = 2.8
h_level = -2.07
ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.3)
# CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.3)
#         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
#         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[50,25], ncols=5, nrows=5)

for i in range(0, 5):
    for j in range(0, 5):
        ax = axes[j][i]
        q_level = np.linspace(2.2, 3.2, 5)[i]
        h_level = np.linspace(-2.5, -1.9, 5)[::-1][j]
        chi = chisqg(np.array([1 if l < q_level else 0 for l in Qeff_min[mask].ravel()]), 
                                   np.array([1 if l > h_level else 0 for l in log_Halpha_SFR_[mask].ravel()]))
        ax.set_title('q={:3.2f}, h={:3.2f}, chi={:3.2f}'.format(q_level, h_level, chi), fontsize=19)
        CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.1)
        CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.1)
#         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
#         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
        CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
        CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
        mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

fig.subplots_adjust(wspace=0.01, hspace=0.15)
plt.show()

In [ ]:
fig= plt.figure(figsize=[30,10])

ax = plt.gca()
q_level = 2.5
h_level = -2.15
ax.set_title('Qg-Qeff_min', fontsize=19)
# signal_match = np.array([0 if ((l[0] < q_level) & (l[1] > h_level)) | (l[0] > q_level) & (l[1] < h_level) else 1 for l in zip(Qeff_min.ravel(), log_Halpha_SFR_.ravel())]).reshape((Ny, Nx))
quadmesh = plt.pcolormesh(X, Y, Qgs-Qeff_min, cmap=cm.hot)
quadmesh.set_clim(vmin=-0.5, vmax=2.)
plt.colorbar()
mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])
add_SFR_contours(ax=ax, colors=('b'))

plt.show()

In [ ]:
fig= plt.figure(figsize=[30,10])

ax = plt.gca()
q_level = 2.5
h_level = -2.15
ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.3)
CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.3)
#         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
#         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

plt.show()

In [ ]:
fig= plt.figure(figsize=[30,10])

ax = plt.gca()
q_level = 2.5
h_level = -2.15
signal_match = np.array([0 if ((l[0] < q_level) & (l[1] > h_level)) | (l[0] > q_level) & (l[1] < h_level) else 1 for l in zip(Qeff_min.ravel(), log_Halpha_SFR_.ravel())]).reshape((Ny, Nx))
quadmesh = plt.pcolormesh(X, Y, signal_match, cmap=cm.hot)
quadmesh.set_clim(vmin=0, vmax=1.)
plt.colorbar()
ax.set_title('q={:3.2f}, h={:3.2f} error={:3.2f}'.format(q_level, h_level, np.sum(signal_match.ravel())), fontsize=19)
mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

plt.show()

In [ ]:
fig= plt.figure(figsize=[30,10])

ax = plt.gca()
q_level = 3.
h_level = -2.3
ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.3)
CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.3)
#         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
#         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

plt.show()

match/not match map

In [ ]:
fig= plt.figure(figsize=[30,10])

ax = plt.gca()
q_level = 3.
h_level = -2.3
signal_match = np.array([0 if ((l[0] < q_level) & (l[1] > h_level)) | (l[0] > q_level) & (l[1] < h_level) else 1 for l in zip(Qeff_min.ravel(), log_Halpha_SFR_.ravel())]).reshape((Ny, Nx))
quadmesh = plt.pcolormesh(X, Y, signal_match, cmap=cm.hot)
quadmesh.set_clim(vmin=0, vmax=1.)
ax.set_title('q={:3.2f}, h={:3.2f} error={:3.2f}'.format(q_level, h_level, np.sum(signal_match.ravel())), fontsize=19)
mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

plt.show()

In [ ]:
np.sum(signal_match[dist_ < 140.].ravel())

In [ ]:
fig= plt.figure(figsize=[30,10])

ax = plt.gca()
q_level = 3.
h_level = -2.3
ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
CS = ax.contourf(X, Y, Qgs, levels=[0., q_level], colors=('m'), alpha=0.3)
#         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
#         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

plt.show()

In [ ]:
fig= plt.figure(figsize=[30,10])

ax = plt.gca()
q_level = 3.
h_level = -2.3
signal_match = np.array([0 if ((l[0] < q_level) & (l[1] > h_level)) | (l[0] > q_level) & (l[1] < h_level) else 1 for l in zip(Qgs.ravel(), log_Halpha_SFR_.ravel())]).reshape((Ny, Nx))
quadmesh = plt.pcolormesh(X, Y, signal_match, cmap=cm.hot)
quadmesh.set_clim(vmin=0, vmax=1.)
ax.set_title('q={:3.2f}, h={:3.2f} error={:3.2f}'.format(q_level, h_level, np.sum(signal_match.ravel())), fontsize=19)
mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])

plt.show()

In [ ]:
np.sum(signal_match[dist_ < 140.].ravel())

## Romeo

Посмотрим трехкомпонентную модель c толщиной и сколько там получается:

In [ ]:
G = 4.32
Qeff_max_rom = np.zeros(dist_.shape)
Qeff_min_rom = np.zeros(dist_.shape)
# rom_comp_min = np.empty(dist_.shape, dtype="S10")
# rom_comp_max = np.empty(dist_.shape, dtype="S10")
rom_comp_min = np.zeros(dist_.shape)
rom_comp_max = np.zeros(dist_.shape)

step = 0
errors = 0
for i, row in enumerate(dist_):
    for j, col in enumerate(dist_[0]):
        step+=1
        if step % 1000 == 0:
            print step
        c = c_g_[i][j]
        Sigma = 1.36*(Sigma_HI_[i][j] + Sigma_HERACLES_CO_[i][j])
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        Sigma_s_ = Sigma_s_Leroy_[i][j]
        s_R_max = sig_R_max[i][j]
        s_R_min = sig_R_min[i][j]
                
        try:
            rom_max, comp = romeo_Qinv(r=None, epicycl=kappa, sound_vel_CO=c, sound_vel_HI=c, 
                            sigma_R=s_R_max, 
                            star_density=Sigma_s_, HI_density=Sigma_HI_[i][j], CO_density=Sigma_HERACLES_CO_[i][j], 
                            alpha=0.3, thin=False, verbose=False, He_corr=False)
        except ValueError:
            errors+=1
            rom_max = 0.01
            comp = 'error'
        Qeff_max_rom[i][j] = 1./rom_max
        if comp == 'star':
            marker = 1
        elif comp == 'HI':
            marker = 2
        elif comp == 'H2':
            marker = 3
        elif comp == 'error':
            marker = 0
        rom_comp_max[i][j] = marker
        
        try:
            rom_min, comp = romeo_Qinv(r=None, epicycl=kappa, sound_vel_CO=c, sound_vel_HI=c, 
                            sigma_R=s_R_min, 
                            star_density=Sigma_s_, HI_density=Sigma_HI_[i][j], CO_density=Sigma_HERACLES_CO_[i][j], 
                            alpha=0.7, thin=False, verbose=False, He_corr=False)
        except ValueError:
            errors+=1
            rom_min = 0.01
            comp = 'error'
        Qeff_min_rom[i][j] = 1./rom_min
        if comp == 'star':
            marker = 1
        elif comp == 'HI':
            marker = 2
        elif comp == 'H2':
            marker = 3
        elif comp == 'error':
            marker = 0
        rom_comp_min[i][j] = marker

        
print 'Errors:{}'.format(errors)

In [ ]:
np.save('Qeff_min_romeo.npy', Qeff_min_rom)

In [ ]:
numpy.unique(rom_comp_min, return_counts=True)

In [ ]:
plt.figure(figsize=[30, 32])

plt.subplot(4,1,1)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, Qeff_min_rom, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_min_rom, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,min,rom}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((512, 512), 10., color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
# circle = plt.Circle((Nx/2, Ny/2), 15*1553./(157.3+175.92), color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
add_SFR_contours()


plt.subplot(4,1,2)
quadmesh = plt.pcolormesh(X, Y, Qeff_max_rom, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_max_rom, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,max,rom}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((Nx/2, Ny/2), 15*1553./(157.3+175.92), color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
add_SFR_contours()


plt.subplot(4,1,3)
quadmesh = plt.pcolormesh(X, Y, rom_comp_min, cmap=cm.Set1)
# quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
# CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'rom_comp_min', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(colors=['b'])

plt.subplot(4,1,4)
quadmesh = plt.pcolormesh(X, Y, rom_comp_max, cmap=cm.Set1)
# quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
# CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'rom_comp_max', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(colors=['b'])

plt.show()

Т.е. целиком почти определяется звездами и $H_2$, наверху - только звездами.

Для тонкого диска:


In [ ]:
G = 4.32
Qeff_max_rom = np.zeros(dist_.shape)
Qeff_min_rom = np.zeros(dist_.shape)
# rom_comp_min = np.empty(dist_.shape, dtype="S10")
# rom_comp_max = np.empty(dist_.shape, dtype="S10")
rom_comp_min = np.zeros(dist_.shape)
rom_comp_max = np.zeros(dist_.shape)

step = 0
errors = 0
for i, row in enumerate(dist_):
    for j, col in enumerate(dist_[0]):
        step+=1
        if step % 1000 == 0:
            print step
        c = c_g_[i][j]
        Sigma = 1.36*(Sigma_HI_[i][j] + Sigma_HERACLES_CO_[i][j])
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        Sigma_s_ = Sigma_s_Leroy_[i][j]
        s_R_max = sig_R_max[i][j]
        s_R_min = sig_R_min[i][j]
                
        try:
            rom_max, comp = romeo_Qinv(r=None, epicycl=kappa, sound_vel_CO=c, sound_vel_HI=c, 
                            sigma_R=s_R_max, 
                            star_density=Sigma_s_, HI_density=Sigma_HI_[i][j], CO_density=Sigma_HERACLES_CO_[i][j], 
                            alpha=0.3, thin=True, verbose=False, He_corr=False)
        except ValueError:
            errors+=1
            rom_max = 0.01
            comp = 'error'
        Qeff_max_rom[i][j] = 1./rom_max
        if comp == 'star':
            marker = 1
        elif comp == 'HI':
            marker = 2
        elif comp == 'H2':
            marker = 3
        elif comp == 'error':
            marker = 0
        rom_comp_max[i][j] = marker
        
        try:
            rom_min, comp = romeo_Qinv(r=None, epicycl=kappa, sound_vel_CO=c, sound_vel_HI=c, 
                            sigma_R=s_R_min, 
                            star_density=Sigma_s_, HI_density=Sigma_HI_[i][j], CO_density=Sigma_HERACLES_CO_[i][j], 
                            alpha=0.7, thin=True, verbose=False, He_corr=False)
        except ValueError:
            errors+=1
            rom_min = 0.01
            comp = 'error'
        Qeff_min_rom[i][j] = 1./rom_min
        if comp == 'star':
            marker = 1
        elif comp == 'HI':
            marker = 2
        elif comp == 'H2':
            marker = 3
        elif comp == 'error':
            marker = 0
        rom_comp_min[i][j] = marker

        
print 'Errors:{}'.format(errors)

In [ ]:
plt.figure(figsize=[30, 32])

plt.subplot(4,1,1)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, Qeff_min_rom, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_min_rom, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,min,rom}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((512, 512), 10., color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
# circle = plt.Circle((Nx/2, Ny/2), 15*1553./(157.3+175.92), color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
add_SFR_contours()


plt.subplot(4,1,2)
quadmesh = plt.pcolormesh(X, Y, Qeff_max_rom, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_max_rom, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,max,rom}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((Nx/2, Ny/2), 15*1553./(157.3+175.92), color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
add_SFR_contours()


plt.subplot(4,1,3)
quadmesh = plt.pcolormesh(X, Y, rom_comp_min, cmap=cm.Set1)
# quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
# CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'rom_comp_min', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(colors=['b'])

plt.subplot(4,1,4)
quadmesh = plt.pcolormesh(X, Y, rom_comp_max, cmap=cm.Set1)
# quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
# CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'rom_comp_max', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(colors=['b'])

plt.show()

Дисперсии $\sigma_{HI} = 1.5\sigma_{H_2}$

In [ ]:
G = 4.32
Qeff_max_rom = np.zeros(dist_.shape)
Qeff_min_rom = np.zeros(dist_.shape)
# rom_comp_min = np.empty(dist_.shape, dtype="S10")
# rom_comp_max = np.empty(dist_.shape, dtype="S10")
rom_comp_min = np.zeros(dist_.shape)
rom_comp_max = np.zeros(dist_.shape)

step = 0
errors = 0
for i, row in enumerate(dist_):
    for j, col in enumerate(dist_[0]):
        step+=1
        if step % 1000 == 0:
            print step
        c = c_g_[i][j]
        Sigma = 1.36*(Sigma_HI_[i][j] + Sigma_HERACLES_CO_[i][j])
        kappa = epicyclicFreq_real(gas_approx, dist_[i][j], scale)
        Sigma_s_ = Sigma_s_Leroy_[i][j]
        s_R_max = sig_R_max[i][j]
        s_R_min = sig_R_min[i][j]
                
        try:
            rom_max, comp = romeo_Qinv(r=None, epicycl=kappa, sound_vel_CO=c/1.5, sound_vel_HI=c, 
                            sigma_R=s_R_max, 
                            star_density=Sigma_s_, HI_density=Sigma_HI_[i][j], CO_density=Sigma_HERACLES_CO_[i][j], 
                            alpha=0.3, thin=False, verbose=False, He_corr=False)
        except ValueError:
            errors+=1
            rom_max = 0.01
            comp = 'error'
        Qeff_max_rom[i][j] = 1./rom_max
        if comp == 'star':
            marker = 1
        elif comp == 'HI':
            marker = 2
        elif comp == 'H2':
            marker = 3
        elif comp == 'error':
            marker = 0
        rom_comp_max[i][j] = marker
        
        try:
            rom_min, comp = romeo_Qinv(r=None, epicycl=kappa, sound_vel_CO=c/1.5, sound_vel_HI=c, 
                            sigma_R=s_R_min, 
                            star_density=Sigma_s_, HI_density=Sigma_HI_[i][j], CO_density=Sigma_HERACLES_CO_[i][j], 
                            alpha=0.7, thin=False, verbose=False, He_corr=False)
        except ValueError:
            errors+=1
            rom_min = 0.01
            comp = 'error'
        Qeff_min_rom[i][j] = 1./rom_min
        if comp == 'star':
            marker = 1
        elif comp == 'HI':
            marker = 2
        elif comp == 'H2':
            marker = 3
        elif comp == 'error':
            marker = 0
        rom_comp_min[i][j] = marker

        
print 'Errors:{}'.format(errors)

In [ ]:
plt.figure(figsize=[30, 32])

plt.subplot(4,1,1)
# X,Y = np.meshgrid(np.linspace(512. - 175.92/1.5, 512.+157.3/1.5, 1553), np.linspace(512. - 53.82/1.5, 512. + 55.42/1.5, 513))
quadmesh = plt.pcolormesh(X, Y, Qeff_min_rom, cmap=cm.gist_yarg)
# CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_min_rom, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,min,rom}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((512, 512), 10., color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
# circle = plt.Circle((Nx/2, Ny/2), 15*1553./(157.3+175.92), color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
add_SFR_contours()


plt.subplot(4,1,2)
quadmesh = plt.pcolormesh(X, Y, Qeff_max_rom, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, Qeff_max_rom, levels=[2.5], colors=('g'))
plt.title(r'$Q_{eff,max,rom}$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
#bulge
# circle = plt.Circle((Nx/2, Ny/2), 15*1553./(157.3+175.92), color='b', fill=False, lw=10)
# plt.gca().add_artist(circle)
add_SFR_contours()


plt.subplot(4,1,3)
quadmesh = plt.pcolormesh(X, Y, rom_comp_min, cmap=cm.Set1)
# quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
# CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'rom_comp_min', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(colors=['b'])

plt.subplot(4,1,4)
quadmesh = plt.pcolormesh(X, Y, rom_comp_max, cmap=cm.Set1)
# quadmesh.set_clim(vmin=2.0, vmax=4.)
plt.colorbar()
# CS = plt.contour(X, Y, Qgs, levels=[2.5], colors=('g'))
plt.title(r'rom_comp_max', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(colors=['b'])

plt.show()

Естественно молекулярный стал посильнее, но результат тот же.

## depletion time

In [ ]:
# НАДО ЛИ ИСПРАВЛЯТЬ ЗА 1.36?
tdep = np.divide(Sigma_HERACLES_CO_, np.power(10., log_Halpha_SFR_))/1000.

In [ ]:
fig = plt.figure(figsize=[30, 10])

quadmesh = plt.pcolormesh(X, Y, tdep, cmap=cm.gist_yarg)
quadmesh.set_clim(vmin=0.0, vmax=4.)
plt.colorbar()
CS = plt.contour(X, Y, tdep, levels=[2.], colors=('g'))
plt.title(r'$\tau_{dep}, Gyr$', fontsize=30)
# add_SFR_contours(levels=[-2.25], colors=('red'), X=None, Y=None)
mark_stars(colors=('y'), levels=[0.999, 1.001])
add_SFR_contours(colors=('red'))

# circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='m', fill=False, lw=6, ls='--')
# plt.gca().add_artist(circle2)

# add_spirals(plt.gca(), colors=['m', 'm'])

Да, два миллиарда лет как и обычно. 

Картинка для статьи:

In [ ]:
import matplotlib as mpl
mpl.style.use('classic')

In [ ]:
fig = plt.figure(figsize=[30, 9])

ax = plt.gca()

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))

CS = plt.contour(X, Y, tdep, levels=[2.], colors=('g'))

CS2 = ax.contour(X, Y, tdep, levels=[2.], colors=('grey'), alpha=0.5)
CS3 = ax.contourf(X, Y, tdep, levels=[0., 2.], colors=('none'), hatches=['.'], alpha=0.)

# CS = plt.contourf(X, Y, lambda_kpc, levels=[0., 1., 2.0, 3.0], 
#             colors=(cm.rainbow(np.linspace(0,1,8))), alpha=0.4)
# plt.colorbar()
# plt.title(r'$\lambda\:[kpc]$', fontsize=30)
# mark_stars(colors=('y'), levels=[0.999, 1.001])
   
# masked_array = np.ma.array(lambda_kpc, mask=sig_los == 0)
# imcmap = matplotlib.cm.gray_r
# imcmap.set_bad('y')
# quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=0., vmax=3., interpolation='none')
# quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
# quadmesh.set_clim(vmin=0.0, vmax=3.)
# CS = plt.contourf(X, Y, masked_array, levels=[0., 1., 2.0, 3.0], 
#                   colors=(cm.rainbow(np.linspace(0,1,8))), alpha=0.4)
# plt.colorbar()

# cbar = plt.colorbar(CS, pad=0.01)
# cbar.ax.tick_params(labelsize=20) 

# def add_SFR_contours(levels=[-2.], colors=('red', 'blue'), X=None, Y=None, ax=None, **kwargs):
#     if X is None and Y is None:
#         X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))
#     if ax is not None:
#         CS = ax.contour(X, Y, log_Halpha_SFR_, levels=levels, colors=colors, **kwargs)
#     else:
#         CS = plt.contour(X, Y, log_Halpha_SFR_, levels=levels, colors=colors, **kwargs)

# add_SFR_contours(levels=[h_level], colors=('red'), X=None, Y=None, linewidths=(4.5))

CS = plt.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -1.], colors=('red'), linewidths=(4.5), alpha=0.3)
# ax.set_facecolor("white")
    
ax.set_xlim(0, Nx-1)
ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
ax.tick_params(axis='both', length=8, width=3)
ax.tick_params(axis='both', length=4, width=2, which='minor')
ax.xaxis.set_minor_locator(AutoMinorLocator(3))
ax.yaxis.set_minor_locator(AutoMinorLocator(2))

ax.text(6, 46.5, r'$\tau_{\mathrm{dep}}$', fontsize=80, color='black')
plt.plot([6, 6+1./calc_scale(8.6) * (Nx/350.)], [6, 6], '-', lw=4, color='black')

# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

plt.savefig(paper_imgs_dir3 +'depl.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'depl.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'depl.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

# Картинки для статьи

In [ ]:
import matplotlib as mpl
mpl.style.use('classic')

In [ ]:
Qeff_min = np.load('Qeff_min.npy')
bark = np.load('bark.npy')
Qeff_max = np.load('Qeff_max.npy')
bark = np.load('bark_max.npy')
Qgs = np.load('Qgs.npy')
Qss_min = np.load('Qss_min.npy')
Qss_max = np.load('Qss_max.npy')

Qeff_min_rom = np.load('Qeff_min_romeo.npy')

X,Y = np.meshgrid(np.arange(0.,Nx,1), np.arange(0.,Ny,1))

In [ ]:
fig, axes = plt.subplots(figsize=[30, 27], ncols=1, nrows=3, sharex=True)

h_limit = -2.23
mask = dist_ < 140.

for ind, Q in enumerate([Qeff_min, Qeff_max, Qgs]):
    ax = axes[ind]
#     quadmesh = ax.pcolormesh(X, Y, Q, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)
    
    mask = (sig_los == 0) | (dist_ > 140.)
    masked_array = np.ma.array(Q, mask=mask)
    imcmap = matplotlib.cm.gist_yarg
    imcmap.set_bad('y',alpha=0.5)
#     quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=2., vmax=4., interpolation='none')
    quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
    quadmesh.set_clim(vmin=2.0, vmax=4.)
    
    # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
    
#     ax.add_colorbar()
    # CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
#     ax.set_title([r'$\mathrm{Q_{eff}^{min}}$', r'$\mathrm{Q_{eff}^{max}}$', r'$\mathrm{Q_{g}}$'][ind], fontsize=30)
    
    ax.text(1, 46.5, [r'$\mathrm{Q_{eff}^{min}}$', r'$\mathrm{Q_{eff}^{max}}$', r'$\,\,\mathrm{Q_{g}}$'][ind], fontsize=70, color='r')

#     CS = ax.contour(X, Y, dist_, levels=[140.])
   
#     add_SFR_contours(colors=('red'), levels=[h_limit], ax=ax, linewidths=(2.))
    log_Halpha_SFR_tmp = log_Halpha_SFR_.copy()
    log_Halpha_SFR_tmp[dist_ > 140.] = -3.5
    mpl.rcParams['contour.negative_linestyle'] = 'solid'
    CS_SFR = ax.contour(X, Y, log_Halpha_SFR_tmp, levels=[h_limit], colors=('red'), linewidths=(3.7))

    circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='m', fill=False, lw=10, ls='--')
    ax.add_artist(circle2)

    add_spirals(ax, colors=['m', 'm'], lw=10, alpha=0.5)
    
    ax.set_facecolor("y")
    plt.xlim(0, Nx-1)
    plt.ylim(0, Ny-1)
    
#     ax.set_yticklabels(ax.get_yticks(), fontsize=30)
#     ax.yaxis.set_ticks([(l*Ny/120.)+28. for l in np.arange(-40., 60, 20)], ['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
    ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
    ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
    if ind == 1:
        ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
    
    ax.tick_params(axis='both', color='r', length=10, width=5)
    ax.tick_params(axis='both', color='r', length=6, width=2, which='minor')
    
    ax.xaxis.set_minor_locator(AutoMinorLocator(3))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))


# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

ax.text(3, 6., r'$1\,\mathrm{kpc}$', fontsize=45, color='red')
plt.plot([2, 2+1./calc_scale(8.6) * (Nx/350.)], [5, 5], '-|', lw=4, color='red', ms=20, mew=6)

cbar_ax = fig.add_axes([0.905, 0.10, 0.02, 0.8])
fig.colorbar(quadmesh, cax=cbar_ax, ticks=np.arange(2., 4.5, 0.5))
cbar_ax.set_yticklabels(['< 2.0', '2.5', ' ', '3.5', '> 4.0'], fontsize=35)
plt.ylabel(r'$\mathrm{Q}$', fontsize=60, rotation=0)
plt.gca().yaxis.set_label_coords(1.75, 0.5)


fig.subplots_adjust(wspace=0.2, hspace=0.01)
plt.savefig(paper_imgs_dir3 +'Q_results.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'Q_results.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'Q_results.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

Вторая:

In [ ]:
# fig= plt.figure(figsize=[30,10])

# ax = plt.gca()
# q_level = 3.
# h_level = -2.3
# ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
# CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.3)
# CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.3)
# #         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
# #         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
# CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
# CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
# mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])
# plt.show()


fig, axes = plt.subplots(figsize=[30, 18], ncols=1, nrows=2, sharex=True)

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

for ind, Q in enumerate([Qeff_min, Qgs]):
    ax = axes[ind]
#     quadmesh = ax.pcolormesh(X, Y, Q, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)
    
    masked_array = np.ma.array(Q, mask=sig_los == 0)
    imcmap = matplotlib.cm.gist_yarg
    imcmap.set_bad('y',alpha=0.5)
#     quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=2., vmax=4., interpolation='none')
#     quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)

    CS = ax.contourf(X, Y, Q, levels=[0., q_level], colors=(['g', 'm'][ind]), alpha=0.3)
    if ind == 0: 
        CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.3)
        CS1 = ax.contourf(X, Y, Qeff_min_rom, levels=[0., q_level], colors=('y'), alpha=0.3)
    else:   
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{min}}$', lw=20, alpha=0.3)
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{max}}$', lw=20, alpha=0.6)
        ax.plot([-1, -2], [-1, -2], '-', color=[181/256., 208/256., 124/256.], 
                label=r'$\mathrm{Q_{eff}^{RF}}$', lw=20, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color='m', label=r'$\mathrm{Q_{g}}$', lw=20, alpha=0.3)
    
        legend = ax.legend(loc=(0.845, 0.7), shadow=False, fontsize=45)
        legend.get_frame().set_facecolor('#FFFFFF')

    CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.5)
    CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
    
    # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
    
#     ax.add_colorbar()
    # CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
    
    ax.text(2, 45, [r'$\mathrm{Q_{eff}}$', r'$\mathrm{Q_{g}}$'][ind], fontsize=70, color='r')

    CS = ax.contour(X, Y, dist_, levels=[140.])

#     mark_stars(colors=('y'), levels=[0.999, 1.001], ax=ax)
    
#     add_SFR_contours(colors=('red'), levels=[h_limit], ax=ax, linewidths=(2.))

    circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='b', fill=False, lw=6, ls='--')
    ax.add_artist(circle2)

#     add_spirals(ax, colors=['m', 'm'], lw=10, alpha=0.2)
    
    ax.set_xlim(0, Nx-1)
    ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
    ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
    ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
    ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
    ax.tick_params(axis='both', length=8, width=3)
    ax.tick_params(axis='both', length=4, width=2, which='minor')
    ax.xaxis.set_minor_locator(AutoMinorLocator(3))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))


# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

# cbar_ax = fig.add_axes([0.905, 0.10, 0.02, 0.8])
# fig.colorbar(quadmesh, cax=cbar_ax, ticks=np.arange(2., 4.5, 0.5))
# cbar_ax.set_yticklabels(['< 2.0', '2.5', '3.0', '3.5', '> 4.0'], fontsize=25)
# # plt.ylabel(r'$\mathrm{Q}$', fontsize=30, rotation=0)


fig.subplots_adjust(wspace=0.2, hspace=0.0)
plt.savefig(paper_imgs_dir3 +'threshold_results.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'threshold_results.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'threshold_results.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# fig= plt.figure(figsize=[30,10])

# ax = plt.gca()
# q_level = 3.
# h_level = -2.3
# ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
# CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.3)
# CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.3)
# #         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
# #         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
# CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
# CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
# mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])
# plt.show()


fig, axes = plt.subplots(figsize=[30, 18], ncols=1, nrows=2, sharex=True)

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

for ind, Q in enumerate([Qeff_min, Qgs]):
    ax = axes[ind]
#     quadmesh = ax.pcolormesh(X, Y, Q, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)
    
    masked_array = np.ma.array(Q, mask=sig_los == 0)
    imcmap = matplotlib.cm.gist_yarg
    imcmap.set_bad('y',alpha=0.5)
#     quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=2., vmax=4., interpolation='none')
#     quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)

    CS = ax.contourf(X, Y, Q, levels=[0., q_level], colors=(['g', 'm'][ind]), alpha=0.3)
    if ind == 0: 
        CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.3)
        CS1 = ax.contourf(X, Y, Qeff_min_rom, levels=[0., q_level], colors=('g'), alpha=0.35)
    else:   
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{min}}$', lw=20, alpha=0.3)
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{max}}$', lw=20, alpha=0.6)
        ax.plot([-1, -2], [-1, -2], '-', color=[181/256., 208/256., 124/256.], 
                label=r'$\mathrm{Q_{eff}^{RF}}$', lw=20, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color='m', label=r'$\mathrm{Q_{g}}$', lw=20, alpha=0.3)
    
        legend = ax.legend(loc=(0.845, 0.7), shadow=False, fontsize=45)
        legend.get_frame().set_facecolor('#FFFFFF')

    CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.5)
    CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
    
    # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
    
#     ax.add_colorbar()
    # CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
    
    ax.text(2, 45, [r'$\mathrm{Q_{eff}}$', r'$\mathrm{Q_{g}}$'][ind], fontsize=70, color='r')

    CS = ax.contour(X, Y, dist_, levels=[140.])

#     mark_stars(colors=('y'), levels=[0.999, 1.001], ax=ax)
    
#     add_SFR_contours(colors=('red'), levels=[h_limit], ax=ax, linewidths=(2.))

    circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='b', fill=False, lw=6, ls='--')
    ax.add_artist(circle2)

#     add_spirals(ax, colors=['m', 'm'], lw=10, alpha=0.2)
    
    ax.set_xlim(0, Nx-1)
    ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
    ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
    ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
    ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
    ax.tick_params(axis='both', length=8, width=3)
    ax.tick_params(axis='both', length=4, width=2, which='minor')
    ax.xaxis.set_minor_locator(AutoMinorLocator(3))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))


# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

# cbar_ax = fig.add_axes([0.905, 0.10, 0.02, 0.8])
# fig.colorbar(quadmesh, cax=cbar_ax, ticks=np.arange(2., 4.5, 0.5))
# cbar_ax.set_yticklabels(['< 2.0', '2.5', '3.0', '3.5', '> 4.0'], fontsize=25)
# # plt.ylabel(r'$\mathrm{Q}$', fontsize=30, rotation=0)


fig.subplots_adjust(wspace=0.2, hspace=0.0)
# plt.savefig(paper_imgs_dir3 +'threshold_results.eps', format='eps', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'threshold_results.png', format='png', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'threshold_results.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# fig= plt.figure(figsize=[30,10])

# ax = plt.gca()
# q_level = 3.
# h_level = -2.3
# ax.set_title('q={:3.2f}, h={:3.2f}'.format(q_level, h_level), fontsize=19)
# CS = ax.contourf(X, Y, Qeff_min, levels=[0., q_level], colors=('g'), alpha=0.3)
# CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.3)
# #         plt.title(r'$Qeff,min={}$'.format(q_level), fontsize=30)
# #         add_SFR_contours(levels=[h_level], colors=('red'), ax=ax)
# CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.3)
# CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
# mark_stars(ax=ax, colors=('b'), levels=[0.999, 1.001])
# plt.show()


fig, axes = plt.subplots(figsize=[30, 18], ncols=1, nrows=2, sharex=True)

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

for ind, Q in enumerate([Qeff_min, Qgs]):
    ax = axes[ind]
#     quadmesh = ax.pcolormesh(X, Y, Q, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)
    
    masked_array = np.ma.array(Q, mask=sig_los == 0)
    imcmap = matplotlib.cm.gist_yarg
    imcmap.set_bad('y',alpha=0.5)
#     quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=2., vmax=4., interpolation='none')
#     quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)

    CS = ax.contourf(X, Y, Q, levels=[0., q_level], colors=(['g', 'm'][ind]), alpha=0.9)
    if ind == 0: 
        CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('b'), alpha=0.9)
        CS1 = ax.contourf(X, Y, Qeff_min_rom, levels=[0., q_level], colors=('m'), alpha=0.9)
    else:   
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{min}}$', lw=20, alpha=0.3)
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{max}}$', lw=20, alpha=0.6)
        ax.plot([-1, -2], [-1, -2], '-', color=[181/256., 208/256., 124/256.], 
                label=r'$\mathrm{Q_{eff}^{RF}}$', lw=20, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color='m', label=r'$\mathrm{Q_{g}}$', lw=20, alpha=0.3)
    
        legend = ax.legend(loc=(0.845, 0.7), shadow=False, fontsize=45)
        legend.get_frame().set_facecolor('#FFFFFF')

    CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=1.)
    CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
    
    # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
    
#     ax.add_colorbar()
    # CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
    
    ax.text(2, 45, [r'$\mathrm{Q_{eff}}$', r'$\mathrm{Q_{g}}$'][ind], fontsize=70, color='r')

    CS = ax.contour(X, Y, dist_, levels=[140.])

#     mark_stars(colors=('y'), levels=[0.999, 1.001], ax=ax)
    
#     add_SFR_contours(colors=('red'), levels=[h_limit], ax=ax, linewidths=(2.))

    circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='b', fill=False, lw=6, ls='--')
    ax.add_artist(circle2)

#     add_spirals(ax, colors=['m', 'm'], lw=10, alpha=0.2)
    
    ax.set_xlim(0, Nx-1)
    ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
    ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
    ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
    ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
    ax.tick_params(axis='both', length=8, width=3)
    ax.tick_params(axis='both', length=4, width=2, which='minor')
    ax.xaxis.set_minor_locator(AutoMinorLocator(3))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))


# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

# cbar_ax = fig.add_axes([0.905, 0.10, 0.02, 0.8])
# fig.colorbar(quadmesh, cax=cbar_ax, ticks=np.arange(2., 4.5, 0.5))
# cbar_ax.set_yticklabels(['< 2.0', '2.5', '3.0', '3.5', '> 4.0'], fontsize=25)
# # plt.ylabel(r'$\mathrm{Q}$', fontsize=30, rotation=0)


fig.subplots_adjust(wspace=0.2, hspace=0.0)
# plt.savefig(paper_imgs_dir3 +'threshold_results.eps', format='eps', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'threshold_results.png', format='png', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'threshold_results.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[30, 18], ncols=1, nrows=2, sharex=True)

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

Q_tmp = Qeff_min.copy()
Q_tmp[Qeff_max < q_level] = q_level+1
Q_tmp[Qeff_min_rom < q_level] = q_level+1

Qeff_max_tmp = Qeff_max.copy()
Qeff_max_tmp[Qeff_min_rom < q_level] = q_level+1

for ind, Q in enumerate([Q_tmp, Qgs]):
    ax = axes[ind]
#     quadmesh = ax.pcolormesh(X, Y, Q, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)
    
    masked_array = np.ma.array(Q, mask=sig_los == 0)
    imcmap = matplotlib.cm.gist_yarg
    imcmap.set_bad('y',alpha=0.5)
#     quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=2., vmax=4., interpolation='none')
#     quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)

    CS = ax.contourf(X, Y, Q, levels=[0., q_level], colors=(['g', 'm'][ind]), alpha=0.3)
    if ind == 0: 
        CS1 = ax.contourf(X, Y, Qeff_max_tmp, levels=[0., q_level], colors=('b'), alpha=0.3)
        CS1 = ax.contourf(X, Y, Qeff_min_rom, levels=[0., q_level], colors=('m'), alpha=0.3)
    else:   
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{min}}$', lw=20, alpha=0.3)
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{max}}$', lw=20, alpha=0.6)
        ax.plot([-1, -2], [-1, -2], '-', color=[181/256., 208/256., 124/256.], 
                label=r'$\mathrm{Q_{eff}^{RF}}$', lw=20, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color='m', label=r'$\mathrm{Q_{g}}$', lw=20, alpha=0.3)
    
        legend = ax.legend(loc=(0.845, 0.7), shadow=False, fontsize=45)
        legend.get_frame().set_facecolor('#FFFFFF')

    CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=1.)
    CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
    
    # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
    
#     ax.add_colorbar()
    # CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
    
    ax.text(2, 45, [r'$\mathrm{Q_{eff}}$', r'$\mathrm{Q_{g}}$'][ind], fontsize=70, color='r')

    CS = ax.contour(X, Y, dist_, levels=[140.])

#     mark_stars(colors=('y'), levels=[0.999, 1.001], ax=ax)
    
#     add_SFR_contours(colors=('red'), levels=[h_limit], ax=ax, linewidths=(2.))

    circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='b', fill=False, lw=6, ls='--')
    ax.add_artist(circle2)

#     add_spirals(ax, colors=['m', 'm'], lw=10, alpha=0.2)
    
    ax.set_xlim(0, Nx-1)
    ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
    ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
    ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
    ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
    ax.tick_params(axis='both', length=8, width=3)
    ax.tick_params(axis='both', length=4, width=2, which='minor')
    ax.xaxis.set_minor_locator(AutoMinorLocator(3))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))


# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

# cbar_ax = fig.add_axes([0.905, 0.10, 0.02, 0.8])
# fig.colorbar(quadmesh, cax=cbar_ax, ticks=np.arange(2., 4.5, 0.5))
# cbar_ax.set_yticklabels(['< 2.0', '2.5', '3.0', '3.5', '> 4.0'], fontsize=25)
# # plt.ylabel(r'$\mathrm{Q}$', fontsize=30, rotation=0)


fig.subplots_adjust(wspace=0.2, hspace=0.0)
# plt.savefig(paper_imgs_dir3 +'threshold_results.eps', format='eps', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'threshold_results.png', format='png', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'threshold_results.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[30, 18], ncols=1, nrows=2, sharex=True)

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

Q_tmp = Qeff_min.copy()
Q_tmp[Q_tmp > q_level] = q_level + 10
Q_tmp[Qeff_max < q_level] = q_level+1
Q_tmp[Qeff_min_rom < q_level] = q_level+2

# Qeff_max_tmp = Qeff_max.copy()
# Qeff_max_tmp[Qeff_min_rom < q_level] = q_level+1

for ind, Q in enumerate([Q_tmp, Qgs]):
    ax = axes[ind]
#     quadmesh = ax.pcolormesh(X, Y, Q, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)
    
    masked_array = np.ma.array(Q, mask=sig_los == 0)
    imcmap = matplotlib.cm.gist_yarg
    imcmap.set_bad('y',alpha=0.5)
#     quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=2., vmax=4., interpolation='none')
#     quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)

    CS = ax.contourf(X, Y, Q, levels=[0., q_level], colors=(['g', 'm'][ind]), alpha=0.9)
    if ind == 0: 
        CS1 = ax.contourf(X, Y, Q_tmp, levels=[q_level, q_level+1], colors=('b'), alpha=0.9)
        CS1 = ax.contourf(X, Y, Q_tmp, levels=[q_level+1, q_level+2], colors=('m'), alpha=0.9)
    else:   
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{min}}$', lw=20, alpha=0.3)
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{max}}$', lw=20, alpha=0.6)
        ax.plot([-1, -2], [-1, -2], '-', color=[181/256., 208/256., 124/256.], 
                label=r'$\mathrm{Q_{eff}^{RF}}$', lw=20, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color='m', label=r'$\mathrm{Q_{g}}$', lw=20, alpha=0.3)
    
        legend = ax.legend(loc=(0.845, 0.7), shadow=False, fontsize=45)
        legend.get_frame().set_facecolor('#FFFFFF')

    CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=1.)
    CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['...'], alpha=0.)
    
    # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
    
#     ax.add_colorbar()
    # CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
    
    ax.text(2, 45, [r'$\mathrm{Q_{eff}}$', r'$\mathrm{Q_{g}}$'][ind], fontsize=70, color='r')

    CS = ax.contour(X, Y, dist_, levels=[140.])

#     mark_stars(colors=('y'), levels=[0.999, 1.001], ax=ax)
    
#     add_SFR_contours(colors=('red'), levels=[h_limit], ax=ax, linewidths=(2.))

    circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='b', fill=False, lw=6, ls='--')
    ax.add_artist(circle2)

#     add_spirals(ax, colors=['m', 'm'], lw=10, alpha=0.2)
    
    ax.set_xlim(0, Nx-1)
    ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
    ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
    ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
    ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
    ax.tick_params(axis='both', length=8, width=3)
    ax.tick_params(axis='both', length=4, width=2, which='minor')
    ax.xaxis.set_minor_locator(AutoMinorLocator(3))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))


# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

# cbar_ax = fig.add_axes([0.905, 0.10, 0.02, 0.8])
# fig.colorbar(quadmesh, cax=cbar_ax, ticks=np.arange(2., 4.5, 0.5))
# cbar_ax.set_yticklabels(['< 2.0', '2.5', '3.0', '3.5', '> 4.0'], fontsize=25)
# # plt.ylabel(r'$\mathrm{Q}$', fontsize=30, rotation=0)


fig.subplots_adjust(wspace=0.2, hspace=0.0)
# plt.savefig(paper_imgs_dir3 +'threshold_results.eps', format='eps', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'threshold_results.png', format='png', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'threshold_results.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[30, 18], ncols=1, nrows=2, sharex=True)

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

for ind, Q in enumerate([Qeff_min, Qgs]):
    ax = axes[ind]
#     quadmesh = ax.pcolormesh(X, Y, Q, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)
    
    masked_array = np.ma.array(Q, mask=sig_los == 0)
    imcmap = matplotlib.cm.gist_yarg
    imcmap.set_bad('y',alpha=0.5)
#     quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=2., vmax=4., interpolation='none')
#     quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)

    CS = ax.contourf(X, Y, Q, levels=[0., q_level], colors=(['g', 'm'][ind]), alpha=0.3)
    if ind == 0: 
        CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.3)
        CS1 = ax.contourf(X, Y, Qeff_min_rom, levels=[0., q_level], colors=('r'), alpha=0.3)
    else:   
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{min}}$', lw=20, alpha=0.3)
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{max}}$', lw=20, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color=[181/256., 208/256., 124/256.], 
                label=r'$\mathrm{Q_{eff}^{RF}}$', lw=20, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color='m', label=r'$\mathrm{Q_{g}}$', lw=20, alpha=0.3)
    
        legend = ax.legend(loc=(0.845, 0.7), shadow=False, fontsize=45)
        legend.get_frame().set_facecolor('#FFFFFF')

    CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=0.5)
    CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], colors=('none'), hatches=['.'], alpha=0.)
    
    # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
    
#     ax.add_colorbar()
    # CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
    
    ax.text(2, 45, [r'$\mathrm{Q_{eff}}$', r'$\mathrm{Q_{g}}$'][ind], fontsize=70, color='r')

    CS = ax.contour(X, Y, dist_, levels=[140.])

#     mark_stars(colors=('y'), levels=[0.999, 1.001], ax=ax)
    
#     add_SFR_contours(colors=('red'), levels=[h_limit], ax=ax, linewidths=(2.))

    circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='b', fill=False, lw=6, ls='--')
    ax.add_artist(circle2)

#     add_spirals(ax, colors=['m', 'm'], lw=10, alpha=0.2)
    
    ax.set_xlim(0, Nx-1)
    ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
    ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
    ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
    ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
    ax.tick_params(axis='both', length=8, width=3)
    ax.tick_params(axis='both', length=4, width=2, which='minor')
    ax.xaxis.set_minor_locator(AutoMinorLocator(3))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))


# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

# cbar_ax = fig.add_axes([0.905, 0.10, 0.02, 0.8])
# fig.colorbar(quadmesh, cax=cbar_ax, ticks=np.arange(2., 4.5, 0.5))
# cbar_ax.set_yticklabels(['< 2.0', '2.5', '3.0', '3.5', '> 4.0'], fontsize=25)
# # plt.ylabel(r'$\mathrm{Q}$', fontsize=30, rotation=0)


fig.subplots_adjust(wspace=0.2, hspace=0.0)
# plt.savefig(paper_imgs_dir3 +'threshold_results.eps', format='eps', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'threshold_results.png', format='png', bbox_inches='tight')
# plt.savefig(paper_imgs_dir3 +'threshold_results.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[30, 18], ncols=1, nrows=2, sharex=True)

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

for ind, Q in enumerate([Qeff_min, Qgs]):
    ax = axes[ind]
#     quadmesh = ax.pcolormesh(X, Y, Q, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)
    
    masked_array = np.ma.array(Q, mask=sig_los == 0)
    imcmap = matplotlib.cm.gist_yarg
    imcmap.set_bad('y',alpha=0.5)
#     quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=2., vmax=4., interpolation='none')
#     quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)

    CS = ax.contourf(X, Y, Q, levels=[0., q_level], colors=(['g', 'm'][ind]), alpha=0.3)
    if ind == 0: 
        CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('g'), alpha=0.4)
        CS1 = ax.contour(X, Y, Qeff_min_rom, levels=[0., q_level], colors=('r'), alpha=0.7, linewidths=(2.))
    else:   
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{min}}$', lw=20, alpha=0.3)
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{max}}$', lw=20, alpha=0.7)
#         ax.plot([-1, -2], [-1, -2], '-', color=[181/256., 208/256., 124/256.], 
#                 label=r'$\mathrm{Q_{eff}^{RF}}$', lw=20, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color='r', 
                label=r'$\mathrm{Q_{eff}^{RF}}$', lw=2, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color='m', label=r'$\mathrm{Q_{g}}$', lw=20, alpha=0.3)
    
        legend = ax.legend(loc=(0.845, 0.7), shadow=False, fontsize=45)
        legend.get_frame().set_facecolor('#FFFFFF')

#     CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=1.)
    CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], 
                      colors=('none'), hatches=['.'], alpha=0.)
    
    # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
    
#     ax.add_colorbar()
    # CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
    
    ax.text(2, 45, [r'$\mathrm{Q_{eff}}$', r'$\mathrm{Q_{g}}$'][ind], fontsize=70, color='black', 
           bbox=dict(facecolor='white', edgecolor='none'))

    CS = ax.contour(X, Y, dist_, levels=[140.])

#     mark_stars(colors=('y'), levels=[0.999, 1.001], ax=ax)
    
#     add_SFR_contours(colors=('red'), levels=[h_limit], ax=ax, linewidths=(2.))

    circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='b', fill=False, lw=6, ls='--')
    ax.add_artist(circle2)

#     add_spirals(ax, colors=['m', 'm'], lw=10, alpha=0.2)
    
    ax.set_xlim(0, Nx-1)
    ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
    ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
    ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
    ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
    ax.tick_params(axis='both', length=8, width=3)
    ax.tick_params(axis='both', length=4, width=2, which='minor')
    ax.xaxis.set_minor_locator(AutoMinorLocator(3))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))


# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

# cbar_ax = fig.add_axes([0.905, 0.10, 0.02, 0.8])
# fig.colorbar(quadmesh, cax=cbar_ax, ticks=np.arange(2., 4.5, 0.5))
# cbar_ax.set_yticklabels(['< 2.0', '2.5', '3.0', '3.5', '> 4.0'], fontsize=25)
# # plt.ylabel(r'$\mathrm{Q}$', fontsize=30, rotation=0)


fig.subplots_adjust(wspace=0.2, hspace=0.0)
plt.savefig(paper_imgs_dir3 +'threshold_results2.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'threshold_results2.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'threshold_results2.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
mpl.rcParams['hatch.linewidth'] = 0.1  # previous pdf hatch linewidth
mpl.rcParams['hatch.linewidth'] = 1.8  # previous svg hatch linewidth

In [ ]:
fig, axes = plt.subplots(figsize=[30, 18], ncols=1, nrows=2, sharex=True)

h_level = -2.23
q_level = 3.0
mask = dist_ < 140.

for ind, Q in enumerate([Qeff_min, Qgs]):
    ax = axes[ind]
#     quadmesh = ax.pcolormesh(X, Y, Q, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)
    
    masked_array = np.ma.array(Q, mask=sig_los == 0)
    imcmap = matplotlib.cm.gist_yarg
    imcmap.set_bad('y',alpha=0.5)
#     quadmesh = ax.imshow(masked_array[::-1, :], cmap=imcmap, vmin=2., vmax=4., interpolation='none')
#     quadmesh = ax.pcolormesh(X, Y, masked_array, cmap=cm.gist_yarg)
#     quadmesh.set_clim(vmin=2.0, vmax=4.)

    CS = ax.contourf(X, Y, Q, levels=[0., q_level], colors=(['g', 'm'][ind]), alpha=0.4)
    if ind == 0: 
        CS1 = ax.contourf(X, Y, Qeff_max, levels=[0., q_level], colors=('b'), alpha=0.3)
        CS1 = ax.contour(X, Y, Qeff_min_rom, levels=[0., q_level], colors=('r'), alpha=0.7, linewidths=(2.))
    else:   
        ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{min}}$', lw=20, alpha=0.4)
#         ax.plot([-1, -2], [-1, -2], '-', color='g', label=r'$\mathrm{Q_{eff}^{max}}$', lw=20, alpha=0.7)
        ax.plot([-1, -2], [-1, -2], '-', color=[106/256., 142/256., 183/256.], 
                label=r'$\mathrm{Q_{eff}^{max}}$', lw=20, alpha=0.9)
#         ax.plot([-1, -2], [-1, -2], '-', color=[181/256., 208/256., 124/256.], 
#                 label=r'$\mathrm{Q_{eff}^{RF}}$', lw=20, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color='r', 
                label=r'$\mathrm{Q_{eff}^{RF}}$', lw=2, alpha=0.9)
        ax.plot([-1, -2], [-1, -2], '-', color='m', label=r'$\mathrm{Q_{g}}$', lw=20, alpha=0.3)
    
        legend = ax.legend(loc=(0.845, 0.7), shadow=False, fontsize=45)
        legend.get_frame().set_facecolor('#FFFFFF')

#     CS2 = ax.contour(X, Y, log_Halpha_SFR_, levels=[h_level], colors=('grey'), alpha=1.)
    CS3 = ax.contourf(X, Y, log_Halpha_SFR_, levels=[h_level, -0.99], 
                      colors=('none'), hatches=['.'], alpha=0.)
    
    # CS = plt.contour(X, Y, Sigma_HI, levels=[5., 10.], aspect='auto', alpha=0.5)
    
#     ax.add_colorbar()
    # CS = plt.contour(X, Y, Qeff_min, levels=[2.5], colors=('g'))
    
    ax.text(2, 45, [r'$\mathrm{Q_{eff}}$', r'$\mathrm{Q_{g}}$'][ind], fontsize=70, color='black', 
           bbox=dict(facecolor='white', edgecolor='none'))

    CS = ax.contour(X, Y, dist_, levels=[140.])

#     mark_stars(colors=('y'), levels=[0.999, 1.001], ax=ax)
    
#     add_SFR_contours(colors=('red'), levels=[h_limit], ax=ax, linewidths=(2.))

    circle2 = plt.Circle((4+Nx/2, 1+Ny/2), 7., color='b', fill=False, lw=6, ls='--')
    ax.add_artist(circle2)

#     add_spirals(ax, colors=['m', 'm'], lw=10, alpha=0.2)
    
    ax.set_xlim(0, Nx-1)
    ax.set_ylim(0, Ny-1)
#     ax.set_yticklabels(ax.get_yticks(), fontsize=20)
    ax.set_yticks([(l*Ny/108.)+28. for l in np.arange(-40., 60, 20)])
    ax.set_yticklabels(['{:2.0f}'.format(l) for l in np.arange(-40., 60, 20)], fontsize=30)
    ax.set_ylabel(r'$\Delta Y, \mathrm{arcsec}$', fontsize=45)
    ax.tick_params(axis='both', length=8, width=3)
    ax.tick_params(axis='both', length=4, width=2, which='minor')
    ax.xaxis.set_minor_locator(AutoMinorLocator(3))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))


# cbar = fig.colorbar(quadmesh, ticks=[2, 3, 4], orientation='horizontal')
# cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  # vertically oriented colorbar

plt.xticks([(l*Nx/328.1)+81. for l in np.arange(-150., 180, 30)], 
           ['{:2.0f}'.format(l) for l in np.arange(-150., 180, 30)], fontsize=30)
plt.xlabel(r'$\Delta X, \mathrm{arcsec}$', fontsize=45)

# cbar_ax = fig.add_axes([0.905, 0.10, 0.02, 0.8])
# fig.colorbar(quadmesh, cax=cbar_ax, ticks=np.arange(2., 4.5, 0.5))
# cbar_ax.set_yticklabels(['< 2.0', '2.5', '3.0', '3.5', '> 4.0'], fontsize=25)
# # plt.ylabel(r'$\mathrm{Q}$', fontsize=30, rotation=0)


fig.subplots_adjust(wspace=0.2, hspace=0.0)
plt.savefig(paper_imgs_dir3 +'threshold_results_c.eps', format='eps', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'threshold_results_c.png', format='png', bbox_inches='tight')
plt.savefig(paper_imgs_dir3 +'threshold_results_c.pdf', format='pdf', dpi=150, bbox_inches='tight')
plt.show()

# Appendix